# Introduction

This notebook introduces the library SPAN, a small library designed to make using setools 4 simple in a Jupyter notebook such as this one.

Jupyter notebooks are an interactive environment that let's us write text (in Markdown) and code together. What's powerful is that the code is executable (unless you are viewing this on the web in a read-only mode). That let's you write queries and text together at the same time. You can get a feel for what's possible in this awesome notebook on [Regex Golf from XKCD](http://nbviewer.jupyter.org/url/norvig.com/ipython/xkcd1313.ipynb). There is also the more official (and boring) [introduction](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/).

Using setools within Jupyter notebook is an amazingly productive way to do policy analysis. It becomes simple to keep notes alongside any queries you do or, almost more importantly, write simple scripts that allow you to do more powerful policy analysis.

To get started, let's import the library and load a Fedora 25 binary and source policy as an example:

In [1]:
# Import span - this complication is just to handle running this in the development tree
try:
    import span as se
except:
    import os
    path = os.path.dirname(os.getcwd())
    import sys
    sys.path.insert(0, path)
    import span as se
p = se.load_policy("fedora-25-policy.30")
ps = se.load_policy_source("serefpolicy-fedora-25")

# Example - Protecting Passwords

We'll get to the details of how to use the library soon. But first, let's start with an example to demonstrate some of the power that we get from this environment.

Let's do that by answering a common security question: what applications can write to the shadow file and are any of those applications controllable by users?

But first, did I mention that we can include images in these notebooks?

<img src="https://i.imgur.com/D5LidQ1.jpg">

## Domains That Can Write /etc/shadow

Anyway, for the first part of that question, we can do a simple search for rules using the method terules_query:

In [2]:
passwd_writers = p.terules_query(target="shadow_t", tclass=["file"], perms=["write", "append", "relabelto"])
passwd_writers

,source,target,tclass,perms,cond
0,cockpit_session_t,shadow_t,file,"{setattr, getattr, write, append, rename, ioctl, create, read, open, link, unlink, lock}",None
1,files_unconfined_type,file_type,file,"{setattr, relabelfrom, getattr, write, append, rename, swapon, mounton, ioctl, create, quotaon, read, audit_access, open, execute, relabelto, execute_no_trans, link, unlink, lock}",None
2,groupadd_t,shadow_t,file,"{setattr, relabelfrom, getattr, write, append, rename, ioctl, create, read, open, relabelto, link, unlink, lock}",None
3,passwd_t,shadow_t,file,"{setattr, relabelfrom, getattr, write, append, rename, ioctl, create, read, open, relabelto, link, unlink, lock}",None
4,pegasus_openlmi_account_t,shadow_t,file,"{setattr, relabelfrom, getattr, write, append, rename, ioctl, create, read, open, relabelto, link, unlink, lock}",None
5,rpm_script_t,file_type,file,"{setattr, relabelfrom, getattr, write, append, rename, ioctl, create, read, open, relabelto, link, unlink, lock}",None
6,rpm_script_t,shadow_t,file,{relabelto},None
7,rpm_t,file_type,file,"{setattr, relabelfrom, getattr, write, append, rename, ioctl, create, read, open, relabelto, link, unlink, lock}",None
8,rpm_t,shadow_t,file,{relabelto},None
9,sandbox_domain,sandbox_typeattr_1,file,"{getattr, write, append, ioctl, read, lock}",None


A few things to note about this if you are new to Jupyter notebook. First, by default Jupyter will display the output from the last expression, which is why just putting the variable we assigned the results to on it's own line caused the display. If we didn't need to use the output later we could have just omitted assigning the output to a variable.

Next, you'll notice that the output is a nicely formatted table. The results are actually in a Pandas [DataFrame](http://pandas.pydata.org/pandas-docs/stable/dsintro.html). [Pandas](http://pandas.pydata.org) is a common and very powerful data analysis tool for Python. Here it lets us display the data nicely, including allowing sorting (try clicking on the column titles to sort by that column).

It also let's us further search. For example, let's see which domains where allowed access directly to `shadow_t` rather than to an attribute that includes `shadow_t`.

In [3]:
passwd_writers[passwd_writers.target == "shadow_t"]

,source,target,tclass,perms,cond
0,cockpit_session_t,shadow_t,file,"{setattr, getattr, write, append, rename, ioctl, create, read, open, link, unlink, lock}",None
2,groupadd_t,shadow_t,file,"{setattr, relabelfrom, getattr, write, append, rename, ioctl, create, read, open, relabelto, link, unlink, lock}",None
3,passwd_t,shadow_t,file,"{setattr, relabelfrom, getattr, write, append, rename, ioctl, create, read, open, relabelto, link, unlink, lock}",None
4,pegasus_openlmi_account_t,shadow_t,file,"{setattr, relabelfrom, getattr, write, append, rename, ioctl, create, read, open, relabelto, link, unlink, lock}",None
6,rpm_script_t,shadow_t,file,{relabelto},None
8,rpm_t,shadow_t,file,{relabelto},None
11,secadm_t,shadow_t,file,"{relabelfrom, getattr, relabelto}",None
13,setfiles_mac_t,shadow_t,file,{relabelto},None
15,setfiles_t,shadow_t,file,{relabelto},None
16,sysadm_passwd_t,shadow_t,file,"{setattr, relabelfrom, getattr, write, append, rename, ioctl, create, read, open, relabelto, link, unlink, lock}",None


Just to make certain this is the right type, let's check the filecontexts from the source:

In [4]:
print(ps.file_contexts("shadow_t"))

./system/authlogin.fc:11:/etc/gshadow.*		--	gen_context(system_u:object_r:shadow_t,s0)
./system/authlogin.fc:12:/etc/nshadow.*		--	gen_context(system_u:object_r:shadow_t,s0)
./system/authlogin.fc:13:/etc/shadow.*		--	gen_context(system_u:object_r:shadow_t,s0)
./system/authlogin.fc:14:/etc/security/opasswd	--	gen_context(system_u:object_r:shadow_t,s0)
./system/authlogin.fc:15:/etc/security/opasswd\.old	--	gen_context(system_u:object_r:shadow_t,s0)
./system/authlogin.fc:55:/var/db/shadow.*	--	gen_context(system_u:object_r:shadow_t,s0)



That looks right . . . I guess. I wonder what the heck `/etc/security/opasswd` is??!? Oh well, it covers good, old-fashioned `/etc/shadow`, so let's move on.

## Finding Domain Transitions from Login Domains

Now we have all of the domains that can write to shadow, so let's answer the second part by determining whether any of the login domains can transition to these domains. Just to keep this short, I'm going to just check for the 3 standard login types (normally you would need to figure out whether there were more).

So let's make a list of those types:

In [5]:
login_domains = ["user_t", "sysadm_t", "secadm_t"]

# Just verify that these are in the policy
for domain in login_domains:
    print(p.lookup_type(domain))

user_t
sysadm_t
secadm_t


Now we can find the domain transitions that are allowed:

In [6]:
user_transitions = p.terules_query(source="user_t", tclass=["process"], perms=["transition"])
user_transitions

,source,target,tclass,perms,cond
0,domain,abrt_helper_t,process,{transition},None
1,user_t,bluetooth_helper_t,process,"{getattr, signull, signal, sigchld, sigstop, transition, sigkill}",None
2,user_t,cdrecord_t,process,"{getattr, signull, signal, sigchld, sigstop, transition, sigkill}",None
3,user_t,chfn_t,process,{transition},None
4,user_t,chkpwd_t,process,"{getattr, transition}",None
5,user_t,chrome_sandbox_t,process,"{getattr, signull, signal, sigchld, sigstop, transition, sigkill}",None
6,user_t,crontab_t,process,"{getattr, signull, signal, sigchld, sigstop, transition, sigkill}",None
7,user_t,exim_t,process,{transition},None
8,user_t,gconfd_t,process,"{getattr, signull, signal, sigchld, sigstop, transition, sigkill}",None
9,user_t,git_session_t,process,{transition},git_session_users


## Putting It Together - Accessible Domains That Can Write /etc/shadow
And now we can see if `user_t` is allowed to transition to any of the domains that can write shadow passwords. To do this, we are going to leverage the built-in sets from Python (which are fantastic). You can get a single column from a DataFrame with `DataFrame.column_name` and, because that is iterable, build a set from that. So we build a set from the targets of the user transitions and the source of the password writers.

Once you have the sets it's simple to perform set intersection (with the `&` operator) to find the types that are both sets.

In [7]:
user_passwd_writers = set(user_transitions.target) & set(passwd_writers.source)
user_passwd_writers

{passwd_t, sandbox_domain, updpwd_t}

So, as expected, `passwd_t` `updpwd_t` can write to shadow and is accessible from `user_t`. `sandbox_domain` is more surprising.

Let's see what types are in that attribute.

In [8]:
p.types_in_attribute("sandbox_domain")

[sandbox_t]

## Automating Checks for All Login Domains

Wait, you should be saying, we're only answering the question for one login domain! Let's build a simple function to do this for all of the login domains (because that nicely shows the power of having a full programming language right here for analysis).

In [9]:
# Yeah for stupidly long function names
def check_accessible_domains_that_can_write_shadow(login_domains):
    for login_domain in login_domains:
        # These are the same queries we did above, just using the passed in login domain types as appropriate.
        writers = p.terules_query(target="shadow_t", tclass=["file"], perms=["write", "append", "relabelto"])
        accessible_domains = p.terules_query(source=login_domain, tclass=["process"], perms=["transition"])
        ad_set = set(accessible_domains.target)
        # Add the login domain to see if it has direct access.
        #
        # Since we are doing comparisons it must be the object here and _not_ the
        # string for the type. These kinds of issues crop up occasionally so keep an eye out for them.
        ad_set.add(p.lookup_type(login_domain))
        print("Shadow writers accessible by " + login_domain + ":")
        print(ad_set & set(writers.source))
        
# Notice how I'm referring to a list that we created way up the page?
check_accessible_domains_that_can_write_shadow(login_domains)

Shadow writers accessible by user_t:
{passwd_t, updpwd_t, sandbox_domain}
Shadow writers accessible by sysadm_t:
{updpwd_t, setfiles_t, sysadm_t, groupadd_t, passwd_t, sysadm_passwd_t, rpm_t, rpm_script_t, useradd_t, sandbox_domain}
Shadow writers accessible by secadm_t:
{secadm_t, updpwd_t, setfiles_t}


Look at that - both `secadm_t` and `sysadm_t` have direct access. Let's see what that looks like.

In [10]:
passwd_writers[passwd_writers.source == "sysadm_t"]

,source,target,tclass,perms,cond
17,sysadm_t,file_type,file,"{relabelfrom, getattr, relabelto}",None
18,sysadm_t,shadow_t,file,"{relabelfrom, getattr, relabelto}",None


And the moral of the story, kids, is never forget about broad relabeling privileges. Being able to relabel a file to a type has the same security implications as writing the same type. Though, come to think of it, you have to also be able to create a file in `/etc` with the correct name. Let's check that.

In [11]:
p.terules_query(source="sysadm_t", target="etc_t", perms=["add_name"])

,source,target,tclass,perms,cond
0,sysadm_t,etc_t,dir,"{getattr, write, ioctl, add_name, read, remove_name, open, search, lock}",None
1,sysadm_t,non_security_file_type,dir,"{setattr, relabelfrom, getattr, write, rename, ioctl, add_name, create, read, remove_name, reparent, rmdir, relabelto, search, link, open, unlink, lock}",None


## Checking Automatic Transitions

Notice that we only checked for _allowed_ transitions. We didn't see if any where automatic. Let's do that now.

In [12]:
automatic_transitions = p.transrules_query(source="user_t", tclass=["process"])
automatic_transitions

,source,target,tclass,default,filename,cond
0,user_t,abrt_helper_exec_t,process,abrt_helper_t,None,None
1,user_t,bluetooth_helper_exec_t,process,bluetooth_helper_t,None,None
2,user_t,cdrecord_exec_t,process,cdrecord_t,None,None
3,user_t,chfn_exec_t,process,chfn_t,None,None
4,user_t,chkpwd_exec_t,process,chkpwd_t,None,None
5,user_t,chrome_sandbox_exec_t,process,chrome_sandbox_t,None,None
6,user_t,crontab_exec_t,process,crontab_t,None,None
7,user_t,dbusd_exec_t,process,user_dbusd_t,None,None
8,user_t,exim_exec_t,process,exim_t,None,None
9,user_t,fusermount_exec_t,process,mount_t,None,None


And using the same approach with sets, we can see if any of those transitions are automatic (just for `user_t` for now).

In [13]:
set(automatic_transitions.default) & user_passwd_writers

{passwd_t, updpwd_t}

Hmmm - wasn't one of those an attribute? And the default type of a transition rule can't be an attribute.

Let's try that again, but expand attributes.

In [14]:
expanded_writers = p.expand_attributes(user_passwd_writers)
print("with attributes expanded: " + str(expanded_writers))
        
set(automatic_transitions.default) & expanded_writers

with attributes expanded: {passwd_t, updpwd_t, sandbox_t}


{passwd_t, updpwd_t}

Well - doesn't look like that's an automatic transition (which isn't surprising). But I included this example to remind you to be careful about attributes. The rule searching will check attributes for you by default, but you have to be careful in your own code.

## Wrapping Up - Entrypoints and Userspace Checks

Two last things.

First - let's see the entrypoints for these domains. I'm including this because I _never_ get tired of bringing this up. It's critical to know what code runs in a domain because that's how you know whether you should trust that code with the access granted to the domain.

In [15]:
# This will handle the sandbox_domain attribute for us automatically in that the domain will be matched as well
# as any types with that attribute with rules explicitly referencing them.
p.terules_query(source=user_passwd_writers, tclass=["file"], perms=["entrypoint"])

,source,target,tclass,perms,cond
0,passwd_t,passwd_exec_t,file,"{getattr, ioctl, read, open, execute, execute_no_trans, entrypoint, lock}",None
1,sandbox_domain,exec_type,file,"{getattr, ioctl, read, open, execute, execute_no_trans, entrypoint, lock}",None
2,sandbox_domain,mountpoint,file,{entrypoint},None
3,updpwd_t,updpwd_exec_t,file,"{getattr, ioctl, read, open, execute, execute_no_trans, entrypoint, lock}",None


Well - sandbox_domain is certainly concerning. I'm certain that the magic of containers is all good though.

Let's check the file contexts for the normal ones.

In [16]:
print(ps.file_contexts("passwd_exec_t"))
print(ps.file_contexts("updpwd_exec_t"))

./admin/usermanage.fc:2:/bin/passwd		--	gen_context(system_u:object_r:passwd_exec_t,s0)
./admin/usermanage.fc:5:/usr/bin/chage		--	gen_context(system_u:object_r:passwd_exec_t,s0)
./admin/usermanage.fc:9:/usr/bin/passwd		--	gen_context(system_u:object_r:passwd_exec_t,s0)
./admin/usermanage.fc:10:/usr/bin/vigr		--	gen_context(system_u:object_r:admin_passwd_exec_t,s0)
./admin/usermanage.fc:11:/usr/bin/vipw		--	gen_context(system_u:object_r:admin_passwd_exec_t,s0)
./admin/usermanage.fc:21:/usr/sbin/grpconv	--	gen_context(system_u:object_r:admin_passwd_exec_t,s0)
./admin/usermanage.fc:22:/usr/sbin/grpunconv	--	gen_context(system_u:object_r:admin_passwd_exec_t,s0)
./admin/usermanage.fc:24:/usr/sbin/pwconv	--	gen_context(system_u:object_r:admin_passwd_exec_t,s0)
./admin/usermanage.fc:25:/usr/sbin/pwunconv	--	gen_context(system_u:object_r:admin_passwd_exec_t,s0)
./admin/usermanage.fc:29:/usr/sbin/vigr		--	gen_context(system_u:object_r:admin_passwd_exec_t,s0)
./admin/usermanage.fc:30:/usr/sbin/

Last thing - there is a userspace permission for changing _other_ users' passwords (which is what really matters here). I know that /bin/passwd checks this, but I'm not certain about things like /sbin/unix_update. But that's what this analysis is for - it tells us what code to go off an audit for trustworthiness.

So let's check the userspace permission for our login types.

In [17]:
p.terules_query(source=login_domains, tclass=["passwd"])

,source,target,tclass,perms,cond
0,secadm_t,secadm_t,passwd,"{passwd, chfn, chsh, rootok}",None
1,sysadm_t,sysadm_t,passwd,"{chfn, crontab, chsh, rootok, passwd}",None


Just what we would expect - `user_t` is not allowed to change other users' passwords.

So at the end of this things are basically what I would have expected with the exception of sandbox_domain (which I'm pretty sure is fine, but I don't understand well enough to know for sure).

# Reference Documentation

Some documentation on what's possible. This isn't exhaustive - mainly because it doesn't cover everything that Setools offers. One important note is that the policy object returned by `se.load_policy` is a subclass of the Setools policy object. All of the public methods from that class are available - you can see them at https://github.com/TresysTechnology/setools/blob/master/setools/policyrep/selinuxpolicy.py.


## Type and Attribute Searching
Find types by name

In [18]:
p.lookup_type("smbd_t")

smbd_t

Find types by regex

In [19]:
p.types_re("smbd")

[smbd_client_packet_t,
 smbd_exec_t,
 smbd_keytab_t,
 smbd_port_t,
 smbd_server_packet_t,
 smbd_t,
 smbd_tmp_t,
 smbd_tmpfs_t,
 smbd_var_run_t]

The return from these functions is an object (even though it is rendered as a string here). You can, for example, show the attributes for a type by calling a method on the returned object.

In [20]:
sorted(p.lookup_type("smbd_t").attributes())

[corenet_unlabeled_type,
 daemon,
 domain,
 kernel_system_state_reader,
 netlabel_peer_type,
 nsswitch_domain,
 pcmcia_typeattr_1,
 pcmcia_typeattr_2,
 pcmcia_typeattr_3,
 pcmcia_typeattr_4,
 pcmcia_typeattr_5,
 pcmcia_typeattr_6,
 pcmcia_typeattr_7,
 syslog_client_type]

Notice that the output was sorted. This is both because it's nice for the output to be sorted, but also because most of the results from setools return generator functions instead of lists. That makes their output less convenient for use in jupyter notebook. For example, this is the output from the previous example not sorted.

In [21]:
p.lookup_type("smbd_t").attributes()

<generator object Type.attributes at 0x1090ae468>

Because of this, we provide some convenience fucntions that simply make the output niecer. For example, find the attributes for a types:

In [22]:
p.attributes_for_type("smbd_t")

[corenet_unlabeled_type,
 daemon,
 domain,
 kernel_system_state_reader,
 netlabel_peer_type,
 nsswitch_domain,
 pcmcia_typeattr_1,
 pcmcia_typeattr_2,
 pcmcia_typeattr_3,
 pcmcia_typeattr_4,
 pcmcia_typeattr_5,
 pcmcia_typeattr_6,
 pcmcia_typeattr_7,
 syslog_client_type]

Find all of the types in an attribute:

In [23]:
p.types_in_attribute("files_unconfined_type")

[abrt_handle_event_t,
 anaconda_t,
 authconfig_t,
 bacula_unconfined_script_t,
 boinc_project_t,
 bootloader_t,
 certmonger_unconfined_t,
 cinder_api_t,
 cinder_backup_t,
 cinder_scheduler_t,
 cinder_volume_t,
 cloud_init_t,
 cluster_t,
 clvmd_t,
 condor_startd_t,
 conman_unconfined_script_t,
 crond_t,
 depmod_t,
 devicekit_disk_t,
 devicekit_power_t,
 devicekit_t,
 dirsrvadmin_unconfined_script_t,
 firstboot_t,
 fsadm_t,
 fwupd_t,
 httpd_unconfined_script_t,
 inetd_child_t,
 inetd_t,
 initrc_t,
 insmod_t,
 install_t,
 kdumpctl_t,
 keepalived_unconfined_script_t,
 kernel_t,
 livecd_t,
 lvm_t,
 mount_t,
 nagios_eventhandler_plugin_t,
 nagios_unconfined_plugin_t,
 openshift_initrc_t,
 openvpn_unconfined_script_t,
 openwsman_t,
 pegasus_openlmi_logicalfile_t,
 pegasus_openlmi_unconfined_t,
 pki_tomcat_script_t,
 prelink_t,
 preupgrade_t,
 puppetagent_t,
 realmd_consolehelper_t,
 realmd_t,
 rolekit_t,
 rpm_script_t,
 rpm_t,
 rtas_errd_t,
 samba_unconfined_net_t,
 samba_unconfined_script_t,

Attributes by regex

In [24]:
p.attributes_re("unconfined")

[corenet_unconfined_type,
 dbusd_unconfined,
 devices_unconfined_type,
 files_unconfined_type,
 filesystem_unconfined_type,
 kern_unconfined,
 selinux_unconfined_type,
 sepgsql_unconfined_type,
 storage_unconfined_type,
 unconfined_domain_type,
 unconfined_login_domain,
 unconfined_usertype,
 xserver_unconfined_type]

Lookup an attribute by name

In [25]:
p.lookup_typeattr("domain")

domain

Expand attribtutes in a list (this will be really long). The list can contain both types and attributes - it just returns the types unchanged into the output list.

In [26]:
p.expand_attributes([p.lookup_type("smbd_t"), p.lookup_typeattr("domain")])

{NetworkManager_t,
 abrt_dump_oops_t,
 abrt_handle_event_t,
 abrt_helper_t,
 abrt_retrace_coredump_t,
 abrt_retrace_worker_t,
 abrt_t,
 abrt_upload_watch_t,
 abrt_watch_log_t,
 accountsd_t,
 acct_t,
 admin_crontab_t,
 afs_bosserver_t,
 afs_fsserver_t,
 afs_kaserver_t,
 afs_ptserver_t,
 afs_t,
 afs_vlserver_t,
 aiccu_t,
 aide_t,
 ajaxterm_ssh_t,
 ajaxterm_t,
 alsa_t,
 amanda_recover_t,
 amanda_t,
 amtu_t,
 anaconda_t,
 anon_sftpd_t,
 antivirus_t,
 apcupsd_cgi_script_t,
 apcupsd_t,
 apm_t,
 apmd_t,
 arpwatch_t,
 asterisk_t,
 audisp_remote_t,
 audisp_t,
 auditadm_screen_t,
 auditadm_su_t,
 auditadm_sudo_t,
 auditadm_t,
 auditctl_t,
 auditd_t,
 authconfig_t,
 automount_t,
 avahi_t,
 awstats_script_t,
 awstats_t,
 bacula_admin_t,
 bacula_t,
 bacula_unconfined_script_t,
 bcfg2_t,
 bitlbee_t,
 blkmapd_t,
 blktap_t,
 blueman_t,
 bluetooth_helper_t,
 bluetooth_t,
 boinc_project_t,
 boinc_t,
 bootloader_t,
 brctl_t,
 brltty_t,
 bugzilla_script_t,
 bumblebee_t,
 cachefiles_kernel_t,
 cachefilesd_

Lookup types or attributes from a list of strings.

In [27]:
p.lookup_type_or_attrs(["smbd_t", "domain"])

[domain, smbd_t]

## Roles

Find roles - this is just a convenience wrapper around https://github.com/TresysTechnology/setools/blob/master/setools/rolequery.py.

In [28]:
p.roles_query(name="sysadm_r")

[sysadm_r]

In [29]:
p.types_in_role("sysadm_r")

['NetworkManager_t',
 'admin_crontab_t',
 'admin_crontab_tmp_t',
 'aide_t',
 'alsa_home_t',
 'amanda_recover_t',
 'antivirus_home_t',
 'audio_home_t',
 'auditctl_t',
 'auth_home_t',
 'bacula_admin_t',
 'bootloader_t',
 'brctl_t',
 'cache_home_t',
 'cardmgr_t',
 'cdcc_t',
 'certwatch_t',
 'checkpc_t',
 'checkpolicy_t',
 'chfn_t',
 'chkpwd_t',
 'chrome_sandbox_home_t',
 'chrome_sandbox_nacl_t',
 'chrome_sandbox_t',
 'config_home_t',
 'container_t',
 'cronjob_t',
 'cvs_home_t',
 'data_home_t',
 'dbus_home_t',
 'dcc_client_t',
 'dcc_dbclean_t',
 'depmod_t',
 'dmidecode_t',
 'exim_t',
 'fetchmail_home_t',
 'firstboot_t',
 'fsadm_t',
 'gconf_home_t',
 'git_session_t',
 'git_user_content_t',
 'gkeyringd_gnome_home_t',
 'gnome_home_t',
 'gpg_secret_t',
 'groupadd_t',
 'gstreamer_home_t',
 'home_bin_t',
 'home_cert_t',
 'hostname_t',
 'httpd_helper_t',
 'httpd_user_content_t',
 'httpd_user_htaccess_t',
 'httpd_user_ra_content_t',
 'httpd_user_rw_content_t',
 'httpd_user_script_exec_t',
 'hwcloc

In [30]:
p.roles_for_type("smbd_t")

['system_r', 'unconfined_r']

## Rule Searching

These two methods are wrappers around an implementation that matches the API for Setools TERuleQuery, so the best documentation is at https://github.com/TresysTechnology/setools/blob/master/setools/terulequery.py.

One major API difference is that source and target paramaters can take a single type/attribute, string, or list.

The other difference is speed. This implementation is often 30x faster. It does fully pass the unit tests for the Setools implementation, so it is fast and API compliant. The speedup comes from the use of an index, so the first rule search after a policy is loaded will build the index (which can take a few seconds). Subsequent queries reuse the index.

In [31]:
p.terules_query(target="shadow_t", perms=se.file_w_perms)

,source,target,tclass,perms,cond
0,cockpit_session_t,shadow_t,file,"{setattr, getattr, write, append, rename, ioctl, create, read, open, link, unlink, lock}",None
1,files_unconfined_type,file_type,blk_file,"{setattr, relabelfrom, getattr, write, append, rename, swapon, mounton, ioctl, execmod, create, quotaon, read, audit_access, open, execute, relabelto, link, unlink, lock}",None
2,files_unconfined_type,file_type,chr_file,"{setattr, relabelfrom, getattr, write, append, rename, swapon, mounton, ioctl, create, quotaon, read, audit_access, open, execute, relabelto, execute_no_trans, link, unlink, lock}",None
3,files_unconfined_type,file_type,dir,"{setattr, ioctl, quotaon, open, relabelto, getattr, reparent, rmdir, unlink, write, append, rename, mounton, add_name, create, search, link, lock, execmod, relabelfrom, swapon, read, remove_name, audit_access, execute}",None
4,files_unconfined_type,file_type,fifo_file,"{setattr, relabelfrom, getattr, write, append, rename, swapon, mounton, ioctl, execmod, create, quotaon, read, audit_access, open, execute, relabelto, link, unlink, lock}",None
5,files_unconfined_type,file_type,file,"{setattr, relabelfrom, getattr, write, append, rename, swapon, mounton, ioctl, create, quotaon, read, audit_access, open, execute, relabelto, execute_no_trans, link, unlink, lock}",None
6,files_unconfined_type,file_type,filesystem,"{relabelfrom, getattr, unmount, mount, quotaget, associate, relabelto, remount, quotamod, transition}",None
7,files_unconfined_type,file_type,lnk_file,"{setattr, relabelfrom, getattr, write, append, rename, swapon, mounton, ioctl, execmod, create, quotaon, read, audit_access, open, execute, relabelto, link, unlink, lock}",None
8,files_unconfined_type,file_type,sock_file,"{setattr, relabelfrom, getattr, write, append, rename, swapon, mounton, ioctl, execmod, create, quotaon, read, audit_access, open, execute, relabelto, link, unlink, lock}",None
9,groupadd_t,shadow_t,file,"{setattr, relabelfrom, getattr, write, append, rename, ioctl, create, read, open, relabelto, link, unlink, lock}",None


In [32]:
# Search with a list for the target
p.terules_query(target=["ssh_home_t", "sshd_key_t"], perms=["write", "append"])

,source,target,tclass,perms,cond
0,abrt_dump_oops_t,non_security_file_type,dir,"{setattr, getattr, write, rename, ioctl, add_name, create, read, remove_name, reparent, rmdir, search, open, link, unlink, lock}",None
1,abrt_dump_oops_t,non_security_file_type,file,"{setattr, getattr, write, append, rename, ioctl, create, read, open, link, unlink, lock}",None
2,abrt_dump_oops_t,non_security_file_type,lnk_file,"{setattr, getattr, write, append, rename, ioctl, create, read, link, unlink, lock}",None
3,ajaxterm_ssh_t,ssh_home_t,dir,"{getattr, write, ioctl, add_name, read, remove_name, open, search, lock}",None
4,ajaxterm_ssh_t,ssh_home_t,file,"{setattr, getattr, write, append, rename, ioctl, create, read, open, link, unlink, lock}",None
5,ajaxterm_t,ssh_home_t,dir,"{getattr, write, ioctl, add_name, read, remove_name, open, search, lock}",None
6,ajaxterm_t,ssh_home_t,file,"{setattr, getattr, write, append, rename, ioctl, create, read, open, link, unlink, lock}",None
7,ajaxterm_t,ssh_home_t,lnk_file,"{setattr, getattr, write, append, rename, ioctl, create, read, link, unlink, lock}",None
8,ajaxterm_t,ssh_home_t,sock_file,"{setattr, getattr, write, append, rename, ioctl, create, read, open, link, unlink, lock}",None
9,automount_t,non_security_file_type,dir,"{setattr, getattr, write, rename, ioctl, add_name, create, read, remove_name, reparent, rmdir, search, open, link, unlink, lock}",None


In [33]:
p.transrules_query(source="initrc_t", default="smbd_t", tclass=["process"])

,source,target,tclass,default,filename,cond
0,initrc_t,smbd_exec_t,process,smbd_t,None,None


## Information Flow

The information flow queries allow you to focus more on the types and object classes without worrying so much about the details of the permissions. You can, instead, think in terms of read, write, or both.

For exampe, `domain_info_flow` shows all of the object types that a domain can read, write, or both.

In [34]:
# By default this shows writes.
p.domain_info_flow("smbd_t", tclass=["file"])

,type,conditional,perms
0,abrt_var_cache_t,None,"{getattr, append}"
1,afs_cache_t,None,"{read, write}"
2,auth_cache_t,None,"{setattr, getattr, write, rename, append, ioctl, create, read, open, link, unlink, lock}"
3,ctdbd_var_lib_t,None,"{setattr, getattr, write, rename, append, ioctl, create, read, open, link, unlink, lock}"
4,faillog_t,None,"{getattr, write, append, ioctl, read, open, lock}"
5,glusterd_var_lib_t,None,"{getattr, write, append, ioctl, read, open, lock}"
6,initrc_tmp_t,None,"{getattr, write, append, ioctl, read, lock}"
7,initrc_var_run_t,None,"{getattr, write, append, ioctl, read, open, lock}"
8,logfile,None,"{ioctl, getattr, append, lock}"
9,nmbd_var_run_t,None,"{getattr, write, append, ioctl, read, open, lock}"


In [35]:
# Show reads instead
p.domain_info_flow("smbd_t", tclass=["file"], direction="r")

,type,conditional,perms
0,abrt_helper_exec_t,None,"{read, getattr, open, execute}"
1,abrt_t,None,"{getattr, ioctl, read, open, lock}"
2,abrt_var_run_t,None,"{getattr, ioctl, read, open, lock}"
3,afs_cache_t,None,"{read, write}"
4,anon_inodefs_t,None,"{getattr, ioctl, read, open, lock}"
5,auth_cache_t,None,"{setattr, getattr, write, rename, append, ioctl, create, read, open, link, unlink, lock}"
6,base_ro_file_type,None,"{getattr, ioctl, read, open, execute, execute_no_trans, lock}"
7,bin_t,None,"{getattr, ioctl, read, open, execute, execute_no_trans, lock}"
8,cert_t,None,"{getattr, ioctl, read, open, lock}"
9,chkpwd_exec_t,None,"{read, getattr, open, execute}"


The information flow is weighted by bandwidth on a scale from 1 to 10. 10 would be something like `read` or `write`, while lower bandwidth permissions, like `getattr` would be lower.

Here we set the minimum weight lower and show the additional types that returns.

In [36]:
set(p.domain_info_flow("NetworkManager_t", min_weight=1).type) - set(p.domain_info_flow("NetworkManager_t").type)

{dhcpc_state_t, dhcpc_var_run_t, dnsmasq_var_run_t}

You can also look from the other direction - the perspective of the object.

In [37]:
p.object_info_flow("bin_t")

,type,conditional,perms
0,abrt_dump_oops_t,None,"{setattr, write, rename, append, ioctl, create, open, link, lock, getattr, read, unlink}"
1,files_unconfined_type,None,"{setattr, ioctl, quotaon, open, relabelto, execute_no_trans, getattr, unlink, write, append, rename, mounton, create, link, lock, execmod, relabelfrom, swapon, read, audit_access, execute}"
2,pegasus_openlmi_logicalfile_t,None,"{setattr, write, rename, append, ioctl, create, open, link, lock, getattr, read, unlink}"
3,prelink_t,None,"{setattr, write, rename, append, ioctl, create, open, relabelto, link, lock, relabelfrom, getattr, read, execute, unlink}"
4,restorecond_t,None,"{relabelfrom, getattr, ioctl, read, open, relabelto, lock}"
5,rpm_script_t,None,"{setattr, write, append, rename, ioctl, create, open, relabelto, link, lock, relabelfrom, getattr, read, unlink}"
6,rpm_t,None,"{setattr, write, append, rename, ioctl, create, open, relabelto, link, lock, relabelfrom, getattr, read, unlink}"
7,secadm_t,None,"{relabelfrom, getattr, relabelto}"
8,setfiles_mac_t,None,"{relabelfrom, getattr, relabelto}"
9,setfiles_t,None,"{relabelfrom, getattr, read, relabelto}"


The concept of read and write works for non-file-like object classes as well.

In [38]:
p.domain_info_flow("smbd_t", tclass=["packet"])

,type,conditional,perms
0,dns_client_packet_t,None,"{send, recv}"
1,ldap_client_packet_t,None,"{send, recv}"
2,unlabeled_t,None,"{send, recv}"
3,client_packet_t,nis_enabled,{send}
4,kerberos_client_packet_t,kerberos_enabled,{send}
5,ocsp_client_packet_t,kerberos_enabled,{send}
6,packet_type,samba_load_libgfapi,{send}
7,portmap_client_packet_t,nis_enabled,{send}
8,server_packet_t,nis_enabled,{send}


You can see which permissions are included with `info_flow_perms`.

In [39]:
p.info_flow_perms(tclass=["dir"], min_weight=1)

['setattr',
 'write',
 'append',
 'rename',
 'mounton',
 'swapon',
 'add_name',
 'create',
 'quotaon',
 'remove_name',
 'rmdir',
 'reparent',
 'relabelto',
 'link',
 'unlink']

In [40]:
p.info_flow_perms(tclass=["dir"], min_weight=10)

['write', 'append', 'relabelto']

## Summaries

These are a quick way to gather related information about something in the policy.

In [41]:
p.types_summary(p.types_re("smb"))

,name,attributes
0,smbcontrol_exec_t,"[application_exec_type, entry_type, exec_type, file_type, non_auth_file_type, non_security_file_type]"
1,smbcontrol_t,"[application_domain_type, corenet_unlabeled_type, domain, pcmcia_typeattr_1, pcmcia_typeattr_2, pcmcia_typeattr_3, pcmcia_typeattr_4, pcmcia_typeattr_5, pcmcia_typeattr_6, pcmcia_typeattr_7]"
2,smbd_client_packet_t,"[client_packet_type, packet_type]"
3,smbd_exec_t,"[direct_init_entry, entry_type, exec_type, file_type, non_auth_file_type, non_security_file_type]"
4,smbd_keytab_t,"[file_type, non_auth_file_type, non_security_file_type, sandbox_typeattr_1, sandbox_typeattr_2, sandbox_typeattr_3, sandbox_typeattr_4]"
5,smbd_port_t,"[defined_port_type, port_type, reserved_port_type]"
6,smbd_server_packet_t,"[packet_type, server_packet_type]"
7,smbd_t,"[corenet_unlabeled_type, daemon, domain, kernel_system_state_reader, netlabel_peer_type, nsswitch_domain, pcmcia_typeattr_1, pcmcia_typeattr_2, pcmcia_typeattr_3, pcmcia_typeattr_4, pcmcia_typeattr_5, pcmcia_typeattr_6, pcmcia_typeattr_7, syslog_client_type]"
8,smbd_tmp_t,"[file_type, non_auth_file_type, non_security_file_type, polymember, sandbox_typeattr_1, sandbox_typeattr_2, sandbox_typeattr_3, sandbox_typeattr_4, tmpfile]"
9,smbd_tmpfs_t,"[file_type, non_auth_file_type, non_security_file_type, sandbox_typeattr_1, sandbox_typeattr_2, sandbox_typeattr_3, sandbox_typeattr_4, tmpfsfile]"


In [42]:
p.domain_summary("httpd_t")


### Domain Summary: httpd_t

####  Type Attributes

  * `can_change_object_identity`
  * `corenet_unlabeled_type`
  * `daemon`
  * `domain`
  * `kernel_system_state_reader`
  * `netlabel_peer_type`
  * `nsswitch_domain`
  * `pcmcia_typeattr_1`
  * `pcmcia_typeattr_2`
  * `pcmcia_typeattr_3`
  * `pcmcia_typeattr_4`
  * `pcmcia_typeattr_5`
  * `pcmcia_typeattr_6`
  * `pcmcia_typeattr_7`
  * `sepgsql_client_type`
  * `syslog_client_type`


#### Domain Transitions In

  * `certwatch_t`
  * `cobblerd_t`
  * `crond_t`
  * `dirsrvadmin_t`
  * `httpd_t`
  * `initrc_domain`
  * `logrotate_t`
  * `piranha_pulse_t`
  * `svc_run_t`
  * `system_cronjob_t`
  * `unconfined_t`


#### Domain Transitions Out

  * `abrt_helper_t`
  * `abrt_retrace_worker_t`
  * `antivirus_t`
  * `apcupsd_cgi_script_t`
  * `awstats_script_t`
  * `bugzilla_script_t`
  * `chkpwd_t`
  * `collectd_script_t`
  * `cvs_script_t`
  * `dirsrvadmin_script_t`
  * `dirsrvadmin_unconfined_script_t`
  * `dspam_script_t`
  * `git_script_t`
  * `gpg_web_t`
  * `httpd_helper_t`
  * `httpd_passwd_t`
  * `httpd_php_t`
  * `httpd_rotatelogs_t`
  * `httpd_suexec_t`
  * `httpd_sys_script_t`
  * `httpd_sys_script_t`
  * `httpd_sys_script_t`
  * `httpd_sys_script_t`
  * `httpd_sys_script_t`
  * `httpd_sys_script_t`
  * `httpd_t`
  * `httpd_unconfined_script_t`
  * `httpd_user_script_t`
  * `ipa_helper_t`
  * `keystone_cgi_script_t`
  * `mailman_cgi_t`
  * `man2html_script_t`
  * `mediawiki_script_t`
  * `mojomojo_script_t`
  * `munin_script_t`
  * `mythtv_script_t`
  * `nagios_script_t`
  * `nutups_cgi_script_t`
  * `openshift_script_t`
  * `passenger_t`
  * `preupgrade_t`
  * `prewikka_script_t`
  * `pwauth_t`
  * `smokeping_cgi_script_t`
  * `spamc_t`
  * `squid_script_t`
  * `system_mail_t`
  * `updpwd_t`
  * `w3c_validator_script_t`
  * `webalizer_script_t`
  * `winbind_helper_t`
  * `zoneminder_script_t`
  * `sepgsql_ranged_proc_t`
  * `sepgsql_trusted_proc_t`


#### Entrypoints

  * `httpd_exec_t`


#### Non-Process Transitions


```
source
target
tclass
default
filename
cond
```
        

#### Roles

  * `system_r`


#### Capability Permissions


```
allow httpd_t httpd_t:capability audit_write; [ httpd_mod_auth_pam ]:True
allow httpd_t httpd_t:capability net_bind_service; [ httpd_enable_ftp_server ]:True
allow httpd_t httpd_t:capability sys_resource; [ httpd_setrlimit ]:True
allow httpd_t httpd_t:capability sys_resource; [ httpd_setrlimit ]:True
allow httpd_t httpd_t:capability { chown dac_override kill net_bind_service setgid setuid sys_chroot sys_nice sys_tty_config };
allow httpd_t httpd_t:capability { fowner fsetid sys_resource }; [ httpd_run_stickshift ]:True
```
        

#### File Read Permissions


```
allow httpd_t abrt_etc_t:file { getattr ioctl lock open read };
allow httpd_t abrt_retrace_spool_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t abrt_retrace_spool_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t abrt_retrace_spool_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t abrt_retrace_worker_exec_t:file { execute getattr open read };
allow httpd_t anon_inodefs_t:file { append getattr ioctl lock open read write };
allow httpd_t antivirus_exec_t:file { execute getattr open read };
allow httpd_t apcupsd_cgi_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t apcupsd_cgi_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t apcupsd_cgi_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t apcupsd_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t apcupsd_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t apcupsd_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t apcupsd_cgi_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t apcupsd_cgi_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t apcupsd_cgi_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t application_exec_type:file { execute execute_no_trans getattr ioctl lock open read };
allow httpd_t autofs_t:dir { getattr ioctl lock open read search }; [ httpd_use_nfs ]:True
allow httpd_t autofs_t:dir { getattr ioctl lock open read search }; [ use_nfs_home_dirs && httpd_enable_homedirs ]:True
allow httpd_t awstats_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t base_ro_file_type:file { execute execute_no_trans getattr ioctl lock open read };
allow httpd_t bin_t:dir { getattr ioctl lock open read search };
allow httpd_t bin_t:dir { getattr ioctl lock open read search }; [ httpd_ssi_exec ]:True
allow httpd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow httpd_t bin_t:lnk_file { getattr read };
allow httpd_t bin_t:lnk_file { getattr read }; [ httpd_can_sendmail ]:True
allow httpd_t bin_t:lnk_file { getattr read }; [ httpd_mod_auth_pam ]:True
allow httpd_t bin_t:lnk_file { getattr read }; [ httpd_run_preupgrade ]:False
allow httpd_t bin_t:lnk_file { getattr read }; [ httpd_run_preupgrade ]:True
allow httpd_t bin_t:lnk_file { getattr read }; [ httpd_ssi_exec ]:True
allow httpd_t bugzilla_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t calamaris_www_t:dir { getattr ioctl lock open read search };
allow httpd_t calamaris_www_t:file { getattr ioctl lock open read };
allow httpd_t calamaris_www_t:lnk_file { getattr read };
allow httpd_t cert_t:dir { getattr ioctl lock open read search };
allow httpd_t cert_t:file { getattr ioctl lock open read };
allow httpd_t cert_t:lnk_file { getattr read };
allow httpd_t cgroup_t:dir { getattr ioctl lock open read search };
allow httpd_t cgroup_t:file { getattr ioctl lock open read };
allow httpd_t cgroup_t:lnk_file { getattr read };
allow httpd_t chkpwd_exec_t:file { execute getattr open read }; [ httpd_mod_auth_pam ]:True
allow httpd_t cifs_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_use_cifs ]:True
allow httpd_t cifs_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_use_cifs ]:True
allow httpd_t cifs_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_use_cifs ]:True
allow httpd_t cifs_t:dir { getattr ioctl lock open read search }; [ git_system_use_cifs ]:True
allow httpd_t cifs_t:dir { getattr ioctl lock open read search }; [ git_system_use_cifs ]:True
allow httpd_t cifs_t:dir { getattr ioctl lock open read search }; [ use_samba_home_dirs && httpd_enable_homedirs ]:True
allow httpd_t cifs_t:dir { getattr ioctl lock open read search }; [ use_samba_home_dirs && httpd_enable_homedirs ]:True
allow httpd_t cifs_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_use_cifs ]:True
allow httpd_t cifs_t:file { execute getattr open read }; [ httpd_use_cifs && httpd_enable_cgi ]:True
allow httpd_t cifs_t:file { getattr ioctl lock open read }; [ git_system_use_cifs ]:True
allow httpd_t cifs_t:file { getattr ioctl lock open read }; [ use_samba_home_dirs && httpd_enable_homedirs ]:True
allow httpd_t cifs_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_use_cifs ]:True
allow httpd_t cifs_t:lnk_file { getattr read }; [ use_samba_home_dirs && httpd_enable_homedirs ]:True
allow httpd_t cluster_conf_t:file { getattr ioctl lock open read };
allow httpd_t cobbler_etc_t:dir { getattr ioctl lock open read search };
allow httpd_t cobbler_etc_t:file { getattr ioctl lock open read };
allow httpd_t cobbler_var_lib_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_serve_cobbler_files ]:True
allow httpd_t cobbler_var_lib_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_serve_cobbler_files ]:True
allow httpd_t cobbler_var_lib_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_serve_cobbler_files ]:True
allow httpd_t cobbler_var_lib_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_serve_cobbler_files ]:True
allow httpd_t cobbler_var_lib_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_serve_cobbler_files ]:True
allow httpd_t cobbler_var_lib_t:file { getattr ioctl lock open read }; [ httpd_serve_cobbler_files ]:False
allow httpd_t cobbler_var_lib_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_serve_cobbler_files ]:True
allow httpd_t cobbler_var_lib_t:lnk_file { getattr read }; [ httpd_serve_cobbler_files ]:False
allow httpd_t collectd_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t collectd_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t collectd_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t collectd_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t collectd_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t collectd_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t collectd_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t collectd_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t collectd_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t crypt_device_t:chr_file { append getattr ioctl lock open read write };
allow httpd_t cvs_data_t:dir { getattr ioctl lock open read search };
allow httpd_t cvs_data_t:file { getattr ioctl lock open read };
allow httpd_t cvs_data_t:lnk_file { getattr read };
allow httpd_t cvs_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t cvs_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t cvs_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t cvs_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t cvs_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t cvs_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t cvs_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t cvs_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t cvs_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t dbusd_etc_t:dir { getattr ioctl lock open read search };
allow httpd_t dbusd_etc_t:file { getattr ioctl lock open read };
allow httpd_t device_t:dir { getattr ioctl lock open read search };
allow httpd_t device_t:lnk_file { getattr read };
allow httpd_t dirsrv_config_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t dirsrv_config_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t dirsrv_share_t:dir { getattr ioctl lock open read search };
allow httpd_t dirsrv_share_t:file { getattr ioctl lock open read };
allow httpd_t dirsrv_share_t:lnk_file read;
allow httpd_t dirsrv_var_log_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t dirsrv_var_log_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t dirsrv_var_run_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t dirsrv_var_run_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t dirsrvadmin_config_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t dirsrvadmin_config_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t dirsrvadmin_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t dirsrvadmin_tmp_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t dirsrvadmin_tmp_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t dirsrvadmin_unconfined_script_exec_t:file { execute getattr open read };
allow httpd_t dspam_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t dspam_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t dspam_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t dspam_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t dspam_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t dspam_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t dspam_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t dspam_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t dspam_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t etc_runtime_t:file { getattr ioctl lock open read };
allow httpd_t etc_runtime_t:lnk_file { getattr read };
allow httpd_t etc_t:dir { getattr ioctl lock open read search };
allow httpd_t file_context_t:dir { getattr ioctl lock open read search };
allow httpd_t file_context_t:dir { getattr ioctl lock open read search }; [ kerberos_enabled ]:True
allow httpd_t file_context_t:file { getattr ioctl lock open read };
allow httpd_t file_context_t:file { getattr ioctl lock open read }; [ kerberos_enabled ]:True
allow httpd_t file_context_t:lnk_file { getattr read };
allow httpd_t file_context_t:lnk_file { getattr read }; [ kerberos_enabled ]:True
allow httpd_t fonts_cache_t:dir { getattr ioctl lock open read search };
allow httpd_t fonts_cache_t:file { getattr ioctl lock open read };
allow httpd_t fonts_cache_t:lnk_file { getattr read };
allow httpd_t fonts_t:dir { getattr ioctl lock open read search };
allow httpd_t fonts_t:file { getattr ioctl lock open read };
allow httpd_t fonts_t:lnk_file { getattr read };
allow httpd_t fusefs_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_use_fusefs ]:True
allow httpd_t fusefs_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_use_fusefs ]:True
allow httpd_t fusefs_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_use_fusefs ]:True
allow httpd_t fusefs_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_use_fusefs ]:True
allow httpd_t fusefs_t:file { execute getattr open read }; [ httpd_use_fusefs && httpd_enable_cgi ]:True
allow httpd_t fusefs_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_use_fusefs ]:True
allow httpd_t git_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t git_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t git_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t git_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t git_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t git_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t git_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t git_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t git_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t git_sys_content_t:dir { getattr ioctl lock open read search };
allow httpd_t git_sys_content_t:file { getattr ioctl lock open read };
allow httpd_t git_sys_content_t:lnk_file { getattr read };
allow httpd_t gitosis_var_lib_t:dir { getattr ioctl lock open read search };
allow httpd_t gitosis_var_lib_t:file { getattr ioctl lock open read };
allow httpd_t gitosis_var_lib_t:lnk_file { getattr read };
allow httpd_t gpg_exec_t:file { execute getattr open read }; [ httpd_use_gpg && httpd_enable_cgi ]:True
allow httpd_t home_root_t:dir { getattr ioctl lock open read search }; [ httpd_enable_homedirs ]:True
allow httpd_t home_root_t:lnk_file { getattr read };
allow httpd_t home_root_t:lnk_file { getattr read }; [ httpd_enable_homedirs ]:True
allow httpd_t home_root_t:lnk_file { getattr read }; [ httpd_read_user_content ]:True
allow httpd_t httpd_cache_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t httpd_cache_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t httpd_cache_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t httpd_config_t:dir { getattr ioctl lock open read search };
allow httpd_t httpd_config_t:file { getattr ioctl lock open read };
allow httpd_t httpd_config_t:lnk_file { getattr read };
allow httpd_t httpd_content_type:dir { getattr ioctl lock open read search }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_content_type:file { getattr ioctl lock open read };
allow httpd_t httpd_content_type:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_content_type:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_exec_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow httpd_t httpd_helper_exec_t:file { execute getattr open read };
allow httpd_t httpd_keytab_t:file { getattr ioctl lock open read };
allow httpd_t httpd_lock_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t httpd_log_t:file { append create getattr ioctl lock open read setattr };
allow httpd_t httpd_log_t:lnk_file { getattr read };
allow httpd_t httpd_modules_t:dir { getattr ioctl lock open read search };
allow httpd_t httpd_modules_t:file { execute getattr ioctl lock open read };
allow httpd_t httpd_modules_t:lnk_file { getattr read };
allow httpd_t httpd_passwd_exec_t:file { execute getattr open read };
allow httpd_t httpd_php_exec_t:file { execute getattr open read };
allow httpd_t httpd_rotatelogs_exec_t:file { execute getattr open read };
allow httpd_t httpd_script_exec_type:dir { getattr ioctl lock open read search };
allow httpd_t httpd_script_exec_type:file { getattr ioctl lock open read };
allow httpd_t httpd_script_exec_type:lnk_file { getattr read };
allow httpd_t httpd_squirrelmail_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t httpd_squirrelmail_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t httpd_squirrelmail_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t httpd_suexec_exec_t:file { execute getattr open read };
allow httpd_t httpd_suexec_t:file { getattr ioctl lock open read };
allow httpd_t httpd_sys_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpd_sys_content_t:dir { getattr ioctl lock open read search };
allow httpd_t httpd_sys_content_t:file { getattr ioctl lock open read };
allow httpd_t httpd_sys_content_t:lnk_file { getattr read };
allow httpd_t httpd_sys_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_sys_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_sys_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpd_sys_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_sys_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_sys_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_sys_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_sys_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpd_sys_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_sys_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpd_sys_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_sys_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t httpd_t:dir { getattr ioctl lock open read search };
allow httpd_t httpd_t:file { append getattr ioctl lock open read write };
allow httpd_t httpd_t:lnk_file { getattr ioctl lock read };
allow httpd_t httpd_tmp_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t httpd_tmp_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t httpd_tmp_t:file { execute execute_no_trans getattr ioctl lock open read }; [ httpd_builtin_scripting && httpd_tmp_exec ]:True
allow httpd_t httpd_tmp_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t httpd_tmpfs_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t httpd_tmpfs_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t httpd_tmpfs_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t httpd_unconfined_script_exec_t:file { execute getattr open read };
allow httpd_t httpd_user_content_type:file { getattr ioctl lock open read }; [ httpd_enable_homedirs ]:True
allow httpd_t httpd_user_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_user_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_user_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_user_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_user_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_user_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_user_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_user_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_user_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t httpd_var_lib_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t httpd_var_lib_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t httpd_var_lib_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t httpd_var_run_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t httpd_var_run_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t httpdcontent:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:dir { add_name getattr ioctl lock open read remove_name search write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:dir { add_name getattr ioctl lock open read remove_name search write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:dir { add_name getattr ioctl lock open read remove_name search write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:dir { add_name getattr ioctl lock open read remove_name search write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:dir { add_name getattr ioctl lock open read remove_name search write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:dir { add_name getattr ioctl lock open read remove_name search write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:file { execute getattr open read }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t hugetlbfs_t:file { append getattr ioctl lock open read write };
allow httpd_t init_t:file { getattr ioctl lock open read };
allow httpd_t init_t:lnk_file { getattr read };
allow httpd_t init_var_run_t:dir { getattr ioctl lock open read search };
allow httpd_t inotifyfs_t:dir { getattr ioctl lock open read search };
allow httpd_t ipa_helper_exec_t:file { execute getattr open read }; [ httpd_run_ipa ]:True
allow httpd_t iso9660_t:dir { getattr ioctl lock open read search };
allow httpd_t iso9660_t:file { getattr ioctl lock open read };
allow httpd_t iso9660_t:lnk_file { getattr read };
allow httpd_t jetty_cache_t:dir { add_name create getattr ioctl link lock open read relabelfrom relabelto remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t jetty_cache_t:file { append create getattr ioctl link lock open read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_cache_t:lnk_file { append create getattr ioctl link lock read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_log_t:dir { add_name create getattr ioctl link lock open read relabelfrom relabelto remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t jetty_log_t:file { append create getattr ioctl link lock open read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_log_t:lnk_file { append create getattr ioctl link lock read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_t:dir { getattr ioctl lock open read search };
allow httpd_t jetty_t:file { getattr ioctl lock open read };
allow httpd_t jetty_t:lnk_file { getattr read };
allow httpd_t jetty_tmp_t:dir { add_name create getattr ioctl link lock open read relabelfrom relabelto remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t jetty_tmp_t:file { append create getattr ioctl link lock open read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_tmp_t:lnk_file { append create getattr ioctl link lock read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_unit_file_t:dir { add_name create getattr ioctl link lock open read relabelfrom relabelto remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t jetty_unit_file_t:file { append create getattr ioctl link lock open read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_unit_file_t:lnk_file { append create getattr ioctl link lock read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_var_lib_t:dir { add_name create getattr ioctl link lock open read relabelfrom relabelto remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t jetty_var_lib_t:file { append create getattr ioctl link lock open read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_var_lib_t:lnk_file { append create getattr ioctl link lock read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_var_run_t:dir { add_name create getattr ioctl link lock open read relabelfrom relabelto remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t jetty_var_run_t:file { append create getattr ioctl link lock open read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_var_run_t:lnk_file { append create getattr ioctl link lock read relabelfrom relabelto rename setattr unlink write };
allow httpd_t keystone_cgi_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t keystone_cgi_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t keystone_cgi_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t keystone_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t keystone_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t keystone_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t keystone_cgi_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t keystone_cgi_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t keystone_cgi_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t keystone_log_t:file { getattr ioctl lock open read }; [ httpd_use_openstack ]:True
allow httpd_t krb5_conf_t:dir { getattr ioctl lock open read search };
allow httpd_t krb5_conf_t:file { getattr ioctl lock open read };
allow httpd_t krb5_host_rcache_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t krb5_host_rcache_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ kerberos_enabled ]:True
allow httpd_t krb5_host_rcache_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t krb5_host_rcache_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ kerberos_enabled ]:True
allow httpd_t krb5_keytab_t:file { getattr ioctl lock open read };
allow httpd_t lib_t:dir { getattr ioctl lock open read search };
allow httpd_t lib_t:file { execute execute_no_trans getattr ioctl lock open read };
allow httpd_t lib_t:lnk_file { getattr read };
allow httpd_t mailman_archive_t:dir { getattr ioctl lock open read search };
allow httpd_t mailman_archive_t:file { getattr ioctl lock open read };
allow httpd_t mailman_archive_t:lnk_file { getattr read };
allow httpd_t mailman_cgi_exec_t:file { execute getattr open read };
allow httpd_t mailman_data_t:dir { getattr ioctl lock open read search };
allow httpd_t mailman_data_t:file { getattr ioctl lock open read };
allow httpd_t mailman_data_t:lnk_file { getattr read };
allow httpd_t man2html_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t man2html_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t man2html_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t man2html_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t man2html_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t man2html_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t man2html_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t man2html_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t man2html_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t mediawiki_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t mediawiki_tmp_t:file { getattr ioctl lock open read unlink };
allow httpd_t mediawiki_tmp_t:lnk_file { getattr read };
allow httpd_t memcached_var_run_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_manage_ipa ]:True
allow httpd_t memcached_var_run_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_manage_ipa ]:True
allow httpd_t mirrormanager_log_t:file { getattr ioctl lock open read };
allow httpd_t mirrormanager_var_lib_t:dir { getattr ioctl lock open read search };
allow httpd_t mirrormanager_var_lib_t:file { getattr ioctl lock open read };
allow httpd_t mirrormanager_var_run_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t mirrormanager_var_run_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t mnt_t:dir { getattr ioctl lock open read search };
allow httpd_t mnt_t:lnk_file { getattr read };
allow httpd_t mojomojo_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t mojomojo_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t mojomojo_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mojomojo_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mojomojo_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mojomojo_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mojomojo_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mojomojo_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mojomojo_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t mta_exec_type:file { execute getattr open read }; [ httpd_can_sendmail ]:True
allow httpd_t mta_exec_type:lnk_file { getattr read }; [ httpd_can_sendmail ]:True
allow httpd_t munin_etc_t:dir { getattr ioctl lock open read search };
allow httpd_t munin_etc_t:file { getattr ioctl lock open read };
allow httpd_t munin_etc_t:lnk_file { getattr read };
allow httpd_t munin_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t mysqld_etc_t:dir { getattr ioctl lock open read search };
allow httpd_t mysqld_etc_t:file { getattr ioctl lock open read };
allow httpd_t mysqld_etc_t:lnk_file { getattr read };
allow httpd_t mythtv_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t nagios_etc_t:dir { getattr ioctl lock open read search };
allow httpd_t nagios_etc_t:file { getattr ioctl lock open read };
allow httpd_t nagios_log_t:file { getattr ioctl lock open read };
allow httpd_t nagios_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t nagios_var_lib_t:dir { getattr ioctl lock open read search };
allow httpd_t nagios_var_lib_t:file { getattr ioctl lock open read };
allow httpd_t nfs_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_use_nfs ]:True
allow httpd_t nfs_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_use_nfs ]:True
allow httpd_t nfs_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_use_nfs ]:True
allow httpd_t nfs_t:dir { getattr ioctl lock open read search }; [ git_system_use_nfs ]:True
allow httpd_t nfs_t:dir { getattr ioctl lock open read search }; [ git_system_use_nfs ]:True
allow httpd_t nfs_t:dir { getattr ioctl lock open read search }; [ use_nfs_home_dirs && httpd_enable_homedirs ]:True
allow httpd_t nfs_t:dir { getattr ioctl lock open read search }; [ use_nfs_home_dirs && httpd_enable_homedirs ]:True
allow httpd_t nfs_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_use_nfs ]:True
allow httpd_t nfs_t:file { execute getattr open read }; [ httpd_use_nfs && httpd_enable_cgi ]:True
allow httpd_t nfs_t:file { getattr ioctl lock open read }; [ git_system_use_nfs ]:True
allow httpd_t nfs_t:file { getattr ioctl lock open read }; [ use_nfs_home_dirs && httpd_enable_homedirs ]:True
allow httpd_t nfs_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_use_nfs ]:True
allow httpd_t nfs_t:lnk_file { getattr read }; [ use_nfs_home_dirs && httpd_enable_homedirs ]:True
allow httpd_t nutups_cgi_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t nutups_cgi_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t nutups_cgi_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t nutups_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t nutups_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t nutups_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t nutups_cgi_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t nutups_cgi_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t nutups_cgi_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t oddjob_t:dir { getattr ioctl lock open read search };
allow httpd_t oddjob_t:file { getattr ioctl lock open read };
allow httpd_t oddjob_t:lnk_file { getattr read };
allow httpd_t openshift_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t passenger_exec_t:file { execute execute_no_trans getattr ioctl lock open read };
allow httpd_t passenger_exec_t:file { execute getattr open read }; [ httpd_run_stickshift ]:False
allow httpd_t passenger_tmp_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_run_stickshift ]:False
allow httpd_t passenger_tmp_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_run_stickshift ]:False
allow httpd_t passenger_tmp_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_run_stickshift ]:False
allow httpd_t passenger_tmp_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_run_stickshift ]:False
allow httpd_t passenger_var_lib_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_run_stickshift ]:True
allow httpd_t passenger_var_lib_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_run_stickshift ]:True
allow httpd_t passenger_var_lib_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_run_stickshift ]:True
allow httpd_t passenger_var_lib_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_run_stickshift ]:True
allow httpd_t passenger_var_lib_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_run_stickshift ]:True
allow httpd_t passenger_var_lib_t:file { getattr ioctl lock open read }; [ httpd_run_stickshift ]:False
allow httpd_t passenger_var_lib_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_run_stickshift ]:True
allow httpd_t passenger_var_lib_t:lnk_file { getattr read }; [ httpd_run_stickshift ]:False
allow httpd_t passenger_var_run_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t passenger_var_run_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t pcscd_var_run_t:file { getattr ioctl lock open read };
allow httpd_t pki_apache_config:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t pki_apache_config:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t pki_apache_var_lib:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t pki_apache_var_lib:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t pki_apache_var_lib:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t pki_apache_var_log:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t pki_apache_var_log:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t pki_apache_var_run:file { getattr ioctl lock open read };
allow httpd_t pki_tomcat_cert_t:file { getattr ioctl lock open read };
allow httpd_t pki_tomcat_cert_t:lnk_file { getattr read };
allow httpd_t postfix_spool_t:file { append getattr ioctl lock open read write }; [ httpd_can_sendmail ]:True
allow httpd_t preupgrade_data_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_run_preupgrade ]:True
allow httpd_t preupgrade_data_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_run_preupgrade ]:True
allow httpd_t preupgrade_data_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_run_preupgrade ]:True
allow httpd_t preupgrade_data_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_run_preupgrade ]:True
allow httpd_t preupgrade_data_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_run_preupgrade ]:True
allow httpd_t preupgrade_data_t:file { getattr ioctl lock open read }; [ httpd_run_preupgrade ]:False
allow httpd_t preupgrade_data_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_run_preupgrade ]:True
allow httpd_t preupgrade_data_t:lnk_file { getattr read }; [ httpd_run_preupgrade ]:False
allow httpd_t preupgrade_exec_t:file { execute execute_no_trans getattr ioctl lock open read }; [ httpd_run_preupgrade ]:False
allow httpd_t preupgrade_exec_t:file { execute getattr open read }; [ httpd_run_preupgrade ]:True
allow httpd_t prewikka_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t prewikka_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t prewikka_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t prewikka_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t prewikka_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t prewikka_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t prewikka_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t prewikka_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t prewikka_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t proc_net_t:dir { getattr ioctl lock open read search };
allow httpd_t proc_net_t:file { getattr ioctl lock open read };
allow httpd_t proc_net_t:lnk_file { getattr read };
allow httpd_t public_content_rw_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_anon_write ]:True
allow httpd_t public_content_rw_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_anon_write ]:True
allow httpd_t public_content_rw_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_anon_write ]:True
allow httpd_t public_content_rw_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_anon_write ]:True
allow httpd_t public_content_rw_t:dir { getattr ioctl lock open read search };
allow httpd_t public_content_rw_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_anon_write ]:True
allow httpd_t public_content_rw_t:file { getattr ioctl lock open read };
allow httpd_t public_content_rw_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_anon_write ]:True
allow httpd_t public_content_rw_t:lnk_file { getattr read };
allow httpd_t public_content_t:dir { getattr ioctl lock open read search };
allow httpd_t public_content_t:file { getattr ioctl lock open read };
allow httpd_t public_content_t:lnk_file { getattr read };
allow httpd_t puppet_var_lib_t:file { getattr ioctl lock open read };
allow httpd_t pwauth_exec_t:file { execute getattr open read };
allow httpd_t random_device_t:chr_file { getattr ioctl lock open read };
allow httpd_t security_t:dir { getattr ioctl lock open read search }; [ kerberos_enabled ]:True
allow httpd_t security_t:file { append getattr ioctl lock open read write }; [ kerberos_enabled ]:True
allow httpd_t security_t:lnk_file { getattr read }; [ kerberos_enabled ]:True
allow httpd_t shell_exec_t:file { execute execute_no_trans getattr ioctl lock open read };
allow httpd_t shell_exec_t:file { execute getattr open read }; [ httpd_ssi_exec ]:True
allow httpd_t slapd_cert_t:dir { getattr ioctl lock open read search };
allow httpd_t slapd_cert_t:file { getattr ioctl lock open read };
allow httpd_t slapd_cert_t:lnk_file { getattr read };
allow httpd_t smokeping_cgi_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t smokeping_cgi_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t smokeping_cgi_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t smokeping_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t smokeping_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t smokeping_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t smokeping_cgi_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t smokeping_cgi_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t smokeping_cgi_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t smokeping_var_lib_t:file { getattr ioctl lock open read };
allow httpd_t smokeping_var_run_t:file { getattr ioctl lock open read };
allow httpd_t spamc_exec_t:file { execute getattr open read }; [ httpd_can_check_spam && httpd_can_sendmail ]:True
allow httpd_t squid_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t squirrelmail_spool_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t squirrelmail_spool_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t squirrelmail_spool_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t sssd_public_t:dir { getattr ioctl lock open read search };
allow httpd_t sssd_public_t:file { getattr ioctl lock open read };
allow httpd_t sysctl_kernel_t:dir { getattr ioctl lock open read search };
allow httpd_t sysctl_kernel_t:file { getattr ioctl lock open read };
allow httpd_t sysfs_t:dir { getattr ioctl lock open read search };
allow httpd_t sysfs_t:file { getattr ioctl lock open read };
allow httpd_t sysfs_t:lnk_file { getattr read };
allow httpd_t system_dbusd_var_lib_t:file { getattr ioctl lock open read };
allow httpd_t systemd_logind_var_run_t:dir { getattr ioctl lock open read search };
allow httpd_t systemd_logind_var_run_t:file { getattr ioctl lock open read };
allow httpd_t systemd_passwd_agent_exec_t:file { execute execute_no_trans getattr ioctl lock open read };
allow httpd_t systemd_passwd_var_run_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t systemd_passwd_var_run_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t systemd_systemctl_exec_t:file { execute execute_no_trans getattr ioctl lock open read };
allow httpd_t systemd_unit_file_type:dir { getattr ioctl lock open read search };
allow httpd_t tetex_data_t:dir { getattr ioctl lock open read search };
allow httpd_t tetex_data_t:file { getattr ioctl lock open read };
allow httpd_t tetex_data_t:lnk_file { getattr read };
allow httpd_t tmp_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t tmp_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ kerberos_enabled ]:True
allow httpd_t tmp_t:lnk_file { getattr read };
allow httpd_t tmp_t:lnk_file { getattr read }; [ httpd_run_stickshift ]:False
allow httpd_t tmp_t:lnk_file { getattr read }; [ kerberos_enabled ]:True
allow httpd_t tmp_t:lnk_file { getattr read }; [ kerberos_enabled ]:True
allow httpd_t tmpfs_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t udev_var_run_t:dir { getattr ioctl lock open read search };
allow httpd_t udev_var_run_t:file { getattr ioctl lock open read };
allow httpd_t udev_var_run_t:lnk_file { getattr read };
allow httpd_t updpwd_exec_t:file { execute getattr open read }; [ httpd_mod_auth_pam ]:True
allow httpd_t urandom_device_t:chr_file { getattr ioctl lock open read };
allow httpd_t user_devpts_t:chr_file { append getattr ioctl lock read write }; [ httpd_tty_comm ]:True
allow httpd_t user_home_dir_t:dir { getattr ioctl lock open read search }; [ httpd_read_user_content ]:True
allow httpd_t user_home_dir_t:lnk_file { getattr read }; [ httpd_enable_homedirs ]:True
allow httpd_t user_home_dir_t:lnk_file { getattr read }; [ httpd_read_user_content ]:True
allow httpd_t user_home_type:dir { getattr ioctl lock open read search }; [ httpd_read_user_content ]:True
allow httpd_t user_home_type:file { getattr ioctl lock open read }; [ httpd_read_user_content ]:True
allow httpd_t user_home_type:lnk_file { getattr read }; [ httpd_enable_homedirs ]:True
allow httpd_t user_tmp_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t user_tty_device_t:chr_file { append getattr ioctl lock read write }; [ httpd_tty_comm ]:True
allow httpd_t usr_t:dir { getattr ioctl lock open read search };
allow httpd_t usr_t:file { execute execute_no_trans getattr ioctl open read };
allow httpd_t usr_t:lnk_file { getattr read };
allow httpd_t var_lib_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t var_lib_t:file { getattr ioctl lock open read };
allow httpd_t var_lib_t:lnk_file { getattr read };
allow httpd_t var_lock_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t var_lock_t:lnk_file { getattr read };
allow httpd_t var_log_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t var_run_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t var_run_t:lnk_file { getattr read };
allow httpd_t var_run_t:lnk_file { getattr read }; [ httpd_manage_ipa ]:True
allow httpd_t var_run_t:lnk_file { getattr read }; [ httpd_use_sasl ]:True
allow httpd_t var_run_t:lnk_file { getattr read }; [ kerberos_enabled ]:True
allow httpd_t var_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t var_t:lnk_file { getattr read };
allow httpd_t var_t:lnk_file { getattr read }; [ httpd_manage_ipa ]:True
allow httpd_t var_t:lnk_file { getattr read }; [ httpd_use_sasl ]:True
allow httpd_t var_t:lnk_file { getattr read }; [ kerberos_enabled ]:True
allow httpd_t w3c_validator_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t w3c_validator_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t w3c_validator_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t w3c_validator_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t w3c_validator_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t w3c_validator_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t w3c_validator_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t w3c_validator_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t w3c_validator_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t webalizer_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t winbind_helper_exec_t:file { execute getattr open read }; [ httpd_mod_auth_ntlm_winbind ]:True
allow httpd_t zarafa_var_lib_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t zarafa_var_lib_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t zarafa_var_lib_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t zoneminder_exec_t:file { execute execute_no_trans getattr ioctl lock open read };
allow httpd_t zoneminder_ra_content_t:file { getattr ioctl lock open read }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_ra_content_t:lnk_file { getattr read }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_script_exec_t:file { execute getattr open read }; [ httpd_enable_cgi ]:True
allow httpd_t zoneminder_var_lib_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t zoneminder_var_lib_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
```
        

#### File Write Permissions


```
allow httpd_t abrt_retrace_spool_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t abrt_retrace_spool_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t abrt_retrace_spool_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t anon_inodefs_t:file { append getattr ioctl lock open read write };
allow httpd_t apcupsd_cgi_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t apcupsd_cgi_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t apcupsd_cgi_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t apcupsd_cgi_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t apcupsd_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t apcupsd_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t apcupsd_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t apcupsd_cgi_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t apcupsd_cgi_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t awstats_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t bugzilla_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t cifs_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_use_cifs ]:True
allow httpd_t cifs_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_use_cifs ]:True
allow httpd_t cifs_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_use_cifs ]:True
allow httpd_t cifs_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_use_cifs ]:True
allow httpd_t cifs_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_use_cifs ]:True
allow httpd_t cobbler_var_lib_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_serve_cobbler_files ]:True
allow httpd_t cobbler_var_lib_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_serve_cobbler_files ]:True
allow httpd_t cobbler_var_lib_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_serve_cobbler_files ]:True
allow httpd_t cobbler_var_lib_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_serve_cobbler_files ]:True
allow httpd_t cobbler_var_lib_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_serve_cobbler_files ]:True
allow httpd_t cobbler_var_lib_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_serve_cobbler_files ]:True
allow httpd_t collectd_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t collectd_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t collectd_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t collectd_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t collectd_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t collectd_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t collectd_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t collectd_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t collectd_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t crypt_device_t:chr_file { append getattr ioctl lock open read write };
allow httpd_t cvs_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t cvs_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t cvs_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t cvs_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t cvs_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t cvs_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t cvs_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t cvs_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t cvs_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrv_config_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t dirsrv_config_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t dirsrv_var_log_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t dirsrv_var_log_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t dirsrv_var_run_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t dirsrv_var_run_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t dirsrvadmin_config_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t dirsrvadmin_config_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t dirsrvadmin_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t dirsrvadmin_tmp_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t dirsrvadmin_tmp_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t dspam_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t dspam_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t dspam_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t dspam_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t dspam_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t dspam_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t dspam_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t dspam_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t dspam_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t fusefs_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_use_fusefs ]:True
allow httpd_t fusefs_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_use_fusefs ]:True
allow httpd_t fusefs_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_use_fusefs ]:True
allow httpd_t fusefs_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_use_fusefs ]:True
allow httpd_t fusefs_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_use_fusefs ]:True
allow httpd_t git_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t git_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t git_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t git_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t git_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t git_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t git_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t git_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t git_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_cache_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t httpd_cache_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t httpd_cache_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t httpd_lock_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t httpd_log_t:dir { add_name create getattr ioctl lock open search setattr write };
allow httpd_t httpd_log_t:file { append create getattr ioctl lock open read setattr };
allow httpd_t httpd_squirrelmail_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t httpd_squirrelmail_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t httpd_squirrelmail_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t httpd_sys_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpd_sys_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_sys_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_sys_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_sys_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpd_sys_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_sys_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_sys_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_sys_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_sys_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpd_sys_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_sys_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpd_sys_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_t:file { append getattr ioctl lock open read write };
allow httpd_t httpd_tmp_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t httpd_tmp_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t httpd_tmp_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t httpd_tmpfs_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t httpd_tmpfs_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t httpd_tmpfs_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t httpd_user_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_user_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_user_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_user_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_user_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_user_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_user_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_user_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_user_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t httpd_var_lib_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t httpd_var_lib_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t httpd_var_lib_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t httpd_var_run_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t httpd_var_run_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t httpdcontent:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:dir { add_name getattr ioctl lock open read remove_name search write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:dir { add_name getattr ioctl lock open read remove_name search write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:dir { add_name getattr ioctl lock open read remove_name search write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:dir { add_name getattr ioctl lock open read remove_name search write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:dir { add_name getattr ioctl lock open read remove_name search write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:dir { add_name getattr ioctl lock open read remove_name search write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpdcontent:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t hugetlbfs_t:file { append getattr ioctl lock open read write };
allow httpd_t jetty_cache_t:dir { add_name create getattr ioctl link lock open read relabelfrom relabelto remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t jetty_cache_t:file { append create getattr ioctl link lock open read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_cache_t:lnk_file { append create getattr ioctl link lock read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_log_t:dir { add_name create getattr ioctl link lock open read relabelfrom relabelto remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t jetty_log_t:file { append create getattr ioctl link lock open read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_log_t:lnk_file { append create getattr ioctl link lock read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_tmp_t:dir { add_name create getattr ioctl link lock open read relabelfrom relabelto remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t jetty_tmp_t:file { append create getattr ioctl link lock open read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_tmp_t:lnk_file { append create getattr ioctl link lock read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_unit_file_t:dir { add_name create getattr ioctl link lock open read relabelfrom relabelto remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t jetty_unit_file_t:file { append create getattr ioctl link lock open read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_unit_file_t:lnk_file { append create getattr ioctl link lock read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_var_lib_t:dir { add_name create getattr ioctl link lock open read relabelfrom relabelto remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t jetty_var_lib_t:file { append create getattr ioctl link lock open read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_var_lib_t:lnk_file { append create getattr ioctl link lock read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_var_run_t:dir { add_name create getattr ioctl link lock open read relabelfrom relabelto remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t jetty_var_run_t:file { append create getattr ioctl link lock open read relabelfrom relabelto rename setattr unlink write };
allow httpd_t jetty_var_run_t:lnk_file { append create getattr ioctl link lock read relabelfrom relabelto rename setattr unlink write };
allow httpd_t keystone_cgi_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t keystone_cgi_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t keystone_cgi_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t keystone_cgi_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t keystone_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t keystone_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t keystone_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t keystone_cgi_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t keystone_cgi_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t krb5_host_rcache_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t krb5_host_rcache_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ kerberos_enabled ]:True
allow httpd_t krb5_host_rcache_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t krb5_host_rcache_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ kerberos_enabled ]:True
allow httpd_t man2html_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t man2html_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t man2html_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t man2html_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t man2html_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t man2html_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t man2html_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t man2html_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t man2html_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mediawiki_tmp_t:dir { getattr ioctl lock open remove_name search write };
allow httpd_t memcached_var_run_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_manage_ipa ]:True
allow httpd_t memcached_var_run_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_manage_ipa ]:True
allow httpd_t mirrormanager_var_run_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t mirrormanager_var_run_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t mojomojo_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mojomojo_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mojomojo_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t mojomojo_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mojomojo_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mojomojo_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mojomojo_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mojomojo_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mojomojo_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t munin_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t mythtv_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t nagios_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t nfs_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_use_nfs ]:True
allow httpd_t nfs_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_use_nfs ]:True
allow httpd_t nfs_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_use_nfs ]:True
allow httpd_t nfs_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_use_nfs ]:True
allow httpd_t nfs_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_use_nfs ]:True
allow httpd_t nutups_cgi_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t nutups_cgi_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t nutups_cgi_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t nutups_cgi_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t nutups_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t nutups_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t nutups_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t nutups_cgi_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t nutups_cgi_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t openshift_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t passenger_tmp_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_run_stickshift ]:False
allow httpd_t passenger_tmp_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_run_stickshift ]:False
allow httpd_t passenger_tmp_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_run_stickshift ]:False
allow httpd_t passenger_tmp_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_run_stickshift ]:False
allow httpd_t passenger_var_lib_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_run_stickshift ]:True
allow httpd_t passenger_var_lib_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_run_stickshift ]:True
allow httpd_t passenger_var_lib_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_run_stickshift ]:True
allow httpd_t passenger_var_lib_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_run_stickshift ]:True
allow httpd_t passenger_var_lib_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_run_stickshift ]:True
allow httpd_t passenger_var_lib_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_run_stickshift ]:True
allow httpd_t passenger_var_run_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t passenger_var_run_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t pki_apache_config:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t pki_apache_config:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t pki_apache_var_lib:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t pki_apache_var_lib:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t pki_apache_var_lib:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t pki_apache_var_log:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t pki_apache_var_log:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t postfix_spool_t:file { append getattr ioctl lock open read write }; [ httpd_can_sendmail ]:True
allow httpd_t preupgrade_data_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_run_preupgrade ]:True
allow httpd_t preupgrade_data_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_run_preupgrade ]:True
allow httpd_t preupgrade_data_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_run_preupgrade ]:True
allow httpd_t preupgrade_data_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_run_preupgrade ]:True
allow httpd_t preupgrade_data_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_run_preupgrade ]:True
allow httpd_t preupgrade_data_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_run_preupgrade ]:True
allow httpd_t prewikka_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t prewikka_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t prewikka_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t prewikka_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t prewikka_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t prewikka_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t prewikka_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t prewikka_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t prewikka_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t public_content_rw_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_anon_write ]:True
allow httpd_t public_content_rw_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_anon_write ]:True
allow httpd_t public_content_rw_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_anon_write ]:True
allow httpd_t public_content_rw_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_anon_write ]:True
allow httpd_t public_content_rw_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_anon_write ]:True
allow httpd_t public_content_rw_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_anon_write ]:True
allow httpd_t security_t:file { append getattr ioctl lock open read write }; [ kerberos_enabled ]:True
allow httpd_t smokeping_cgi_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t smokeping_cgi_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t smokeping_cgi_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t smokeping_cgi_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t smokeping_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t smokeping_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t smokeping_cgi_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t smokeping_cgi_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t smokeping_cgi_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t squid_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t squirrelmail_spool_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t squirrelmail_spool_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t squirrelmail_spool_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t systemd_passwd_var_run_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t systemd_passwd_var_run_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t tmp_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t tmp_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ kerberos_enabled ]:True
allow httpd_t tmpfs_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t user_devpts_t:chr_file { append getattr ioctl lock read write }; [ httpd_tty_comm ]:True
allow httpd_t user_tmp_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t user_tty_device_t:chr_file { append getattr ioctl lock read write }; [ httpd_tty_comm ]:True
allow httpd_t var_lib_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t var_lock_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t var_log_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t var_run_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t var_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow httpd_t w3c_validator_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t w3c_validator_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t w3c_validator_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t w3c_validator_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t w3c_validator_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t w3c_validator_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t w3c_validator_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t w3c_validator_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t w3c_validator_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t webalizer_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t zarafa_var_lib_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t zarafa_var_lib_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
allow httpd_t zarafa_var_lib_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow httpd_t zoneminder_log_t:file { append getattr ioctl lock open };
allow httpd_t zoneminder_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_ra_content_t:dir { add_name getattr ioctl lock open search write }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_ra_content_t:file { append getattr ioctl lock open }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_rw_content_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_rw_content_t:dir { add_name getattr ioctl lock open read remove_name search write }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_rw_content_t:file { append create getattr ioctl link lock open read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_rw_content_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write }; [ httpd_builtin_scripting ]:True
allow httpd_t zoneminder_var_lib_t:dir { add_name create getattr ioctl link lock open read remove_name rename reparent rmdir search setattr unlink write };
allow httpd_t zoneminder_var_lib_t:file { append create getattr ioctl link lock open read rename setattr unlink write };
```
        

#### Packet Permissions


```
allow httpd_t ftp_client_packet_t:packet recv; [ httpd_can_network_relay ]:True
allow httpd_t ftp_client_packet_t:packet send; [ httpd_can_network_relay ]:True
allow httpd_t gopher_client_packet_t:packet recv; [ httpd_can_network_relay ]:True
allow httpd_t gopher_client_packet_t:packet send; [ httpd_can_network_relay ]:True
allow httpd_t http_cache_client_packet_t:packet recv; [ httpd_can_network_relay ]:True
allow httpd_t http_cache_client_packet_t:packet send; [ httpd_can_network_relay ]:True
allow httpd_t http_client_packet_t:packet recv; [ httpd_can_network_relay ]:True
allow httpd_t http_client_packet_t:packet send; [ httpd_can_network_relay ]:True
allow httpd_t http_server_packet_t:packet { recv send };
allow httpd_t kerberos_client_packet_t:packet recv; [ kerberos_enabled ]:True
allow httpd_t kerberos_client_packet_t:packet send; [ kerberos_enabled ]:True
allow httpd_t mssql_client_packet_t:packet recv; [ httpd_can_network_connect_db ]:True
allow httpd_t mssql_client_packet_t:packet send; [ httpd_can_network_connect_db ]:True
allow httpd_t mysqld_client_packet_t:packet recv; [ httpd_can_network_connect_db ]:True
allow httpd_t mysqld_client_packet_t:packet send; [ httpd_can_network_connect_db ]:True
allow httpd_t ocsp_client_packet_t:packet recv; [ kerberos_enabled ]:True
allow httpd_t ocsp_client_packet_t:packet send; [ kerberos_enabled ]:True
allow httpd_t oracle_client_packet_t:packet recv; [ httpd_can_network_connect_db ]:True
allow httpd_t oracle_client_packet_t:packet send; [ httpd_can_network_connect_db ]:True
allow httpd_t pop_client_packet_t:packet recv; [ httpd_can_sendmail ]:True
allow httpd_t pop_client_packet_t:packet send; [ httpd_can_sendmail ]:True
allow httpd_t postgresql_client_packet_t:packet recv; [ httpd_can_network_connect_db ]:True
allow httpd_t postgresql_client_packet_t:packet send; [ httpd_can_network_connect_db ]:True
allow httpd_t smtp_client_packet_t:packet recv; [ httpd_can_sendmail ]:True
allow httpd_t smtp_client_packet_t:packet send; [ httpd_can_sendmail ]:True
allow httpd_t squid_client_packet_t:packet recv; [ httpd_can_network_relay ]:True
allow httpd_t squid_client_packet_t:packet send; [ httpd_can_network_relay ]:True
```
        

#### Process Permissions


```
allow httpd_t abrt_retrace_worker_t:process transition;
allow httpd_t antivirus_t:process transition;
allow httpd_t apcupsd_cgi_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t awstats_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t bugzilla_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t certwatch_t:process sigchld;
allow httpd_t chkpwd_t:process transition; [ httpd_mod_auth_pam ]:True
allow httpd_t cobblerd_t:process sigchld;
allow httpd_t collectd_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t crond_t:process sigchld;
allow httpd_t cvs_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t dirsrv_t:process { signal signull };
allow httpd_t dirsrvadmin_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t dirsrvadmin_t:process sigchld;
allow httpd_t dirsrvadmin_unconfined_script_t:process { sigchld sigkill signal signull sigstop transition };
allow httpd_t domain:process getpgid; [ httpd_run_stickshift ]:True
allow httpd_t dspam_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t git_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t gpg_web_t:process transition; [ httpd_use_gpg && httpd_enable_cgi ]:True
allow httpd_t httpd_helper_t:process transition;
allow httpd_t httpd_passwd_t:process transition;
allow httpd_t httpd_php_t:process transition;
allow httpd_t httpd_rotatelogs_t:process { sigchld sigkill signal signull sigstop transition };
allow httpd_t httpd_script_type:process { sigkill signal signull sigstop };
allow httpd_t httpd_suexec_t:process { signal signull transition };
allow httpd_t httpd_sys_script_t:process transition; [ ( httpd_builtin_scripting && httpd_unified && httpd_enable_cgi ) ]:True
allow httpd_t httpd_sys_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t httpd_sys_script_t:process transition; [ httpd_ssi_exec ]:True
allow httpd_t httpd_sys_script_t:process transition; [ httpd_use_cifs && httpd_enable_cgi ]:True
allow httpd_t httpd_sys_script_t:process transition; [ httpd_use_fusefs && httpd_enable_cgi ]:True
allow httpd_t httpd_sys_script_t:process transition; [ httpd_use_nfs && httpd_enable_cgi ]:True
allow httpd_t httpd_t:process setexec; [ httpd_run_stickshift ]:True
allow httpd_t httpd_t:process setfscreate; [ kerberos_enabled ]:True
allow httpd_t httpd_t:process setrlimit; [ httpd_setrlimit ]:True
allow httpd_t httpd_t:process setrlimit; [ httpd_setrlimit ]:True
allow httpd_t httpd_t:process { dyntransition fork getattr getcap getpgid getsched getsession noatsecure rlimitinh setcap setkeycreate setpgid setsched setsockcreate share sigchld siginh sigkill signal signull sigstop transition };
allow httpd_t httpd_t:process { execmem execstack }; [ httpd_execmem ]:True
allow httpd_t httpd_unconfined_script_t:process { sigchld sigkill signal signull sigstop transition };
allow httpd_t httpd_user_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t ipa_helper_t:process transition; [ httpd_run_ipa ]:True
allow httpd_t jetty_t:process ptrace; [ deny_ptrace ]:False
allow httpd_t jetty_t:process { getattr sigchld sigkill signal signull sigstop };
allow httpd_t keystone_cgi_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t logrotate_t:process sigchld;
allow httpd_t mailman_cgi_t:process { signal transition };
allow httpd_t man2html_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t mediawiki_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t mojomojo_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t munin_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t mythtv_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t nagios_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t newrole_t:process sigchld;
allow httpd_t nutups_cgi_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t oddjob_t:process getattr;
allow httpd_t openshift_initrc_t:process { signal signull };
allow httpd_t openshift_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t passenger_t:process sigkill;
allow httpd_t passenger_t:process transition; [ httpd_run_stickshift ]:False
allow httpd_t piranha_pulse_t:process sigchld;
allow httpd_t pki_apache_domain:process signal;
allow httpd_t preupgrade_t:process transition; [ httpd_run_preupgrade ]:True
allow httpd_t prewikka_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t pwauth_t:process transition;
allow httpd_t smokeping_cgi_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t spamc_t:process transition; [ httpd_can_check_spam && httpd_can_sendmail ]:True
allow httpd_t squid_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t svc_start_t:process sigchld;
allow httpd_t system_cronjob_t:process sigchld;
allow httpd_t system_mail_t:process signal; [ httpd_can_sendmail ]:True
allow httpd_t system_mail_t:process transition; [ httpd_can_sendmail ]:True
allow httpd_t updpwd_t:process transition; [ httpd_mod_auth_pam ]:True
allow httpd_t w3c_validator_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t webalizer_script_t:process transition; [ httpd_enable_cgi ]:True
allow httpd_t winbind_helper_t:process signal; [ httpd_mod_auth_ntlm_winbind ]:True
allow httpd_t winbind_helper_t:process transition; [ httpd_mod_auth_ntlm_winbind ]:True
allow httpd_t zoneminder_script_t:process transition; [ httpd_enable_cgi ]:True
```
        

#### Socket Permissions


```
allow httpd_t apcupsd_cgi_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t awstats_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t bugzilla_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t cobbler_port_t:tcp_socket name_connect; [ httpd_can_network_connect_cobbler ]:True
allow httpd_t collectd_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t commplex_main_port_t:tcp_socket name_bind; [ httpd_use_openstack ]:True
allow httpd_t cvs_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t dirsrv_t:unix_stream_socket connectto;
allow httpd_t dirsrvadmin_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t dspam_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t ephemeral_port_type:tcp_socket name_bind; [ httpd_enable_ftp_server ]:True
allow httpd_t ephemeral_port_type:tcp_socket name_connect; [ httpd_can_connect_ftp ]:True
allow httpd_t ephemeral_port_type:tcp_socket name_connect; [ httpd_can_network_relay ]:True
allow httpd_t ephemeral_port_type:tcp_socket name_connect; [ httpd_use_openstack ]:True
allow httpd_t ephemeral_port_type:udp_socket name_bind; [ httpd_verify_dns ]:True
allow httpd_t ftp_port_t:tcp_socket name_bind; [ httpd_enable_ftp_server ]:True
allow httpd_t ftp_port_t:tcp_socket name_connect; [ httpd_can_connect_ftp ]:True
allow httpd_t ftp_port_t:tcp_socket name_connect; [ httpd_can_network_relay ]:True
allow httpd_t gds_db_port_t:tcp_socket name_connect; [ httpd_can_network_connect_db ]:True
allow httpd_t git_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t gopher_port_t:tcp_socket name_connect; [ httpd_can_network_relay ]:True
allow httpd_t gssproxy_t:unix_stream_socket connectto;
allow httpd_t http_cache_port_t:tcp_socket name_bind;
allow httpd_t http_cache_port_t:tcp_socket name_connect; [ httpd_can_network_relay ]:True
allow httpd_t http_port_t:tcp_socket name_bind;
allow httpd_t http_port_t:tcp_socket name_connect; [ httpd_can_network_relay ]:True
allow httpd_t http_port_t:tcp_socket name_connect; [ httpd_graceful_shutdown ]:True
allow httpd_t http_port_t:udp_socket name_bind;
allow httpd_t httpd_script_type:unix_stream_socket connectto;
allow httpd_t httpd_sys_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t httpd_sys_script_t:unix_stream_socket connectto;
allow httpd_t httpd_t:netlink_audit_socket { append bind connect create getattr getopt ioctl lock nlmsg_read nlmsg_relay read setattr setopt shutdown write }; [ httpd_mod_auth_pam ]:True
allow httpd_t httpd_t:netlink_route_socket { append bind connect create getattr getopt ioctl lock nlmsg_read read setattr setopt shutdown write };
allow httpd_t httpd_t:tcp_socket { accept append bind connect create getattr getopt ioctl listen lock read setattr setopt shutdown write };
allow httpd_t httpd_t:tcp_socket { accept append bind connect create getattr getopt ioctl listen lock read setattr setopt shutdown write }; [ nis_enabled ]:True
allow httpd_t httpd_t:tcp_socket { append bind connect create getattr getopt ioctl lock read setattr setopt shutdown write }; [ authlogin_nsswitch_use_ldap ]:True
allow httpd_t httpd_t:tcp_socket { append bind connect create getattr getopt ioctl lock read setattr setopt shutdown write }; [ kerberos_enabled ]:True
allow httpd_t httpd_t:tcp_socket { append bind connect create getattr getopt ioctl lock read setattr setopt shutdown write }; [ kerberos_enabled ]:True
allow httpd_t httpd_t:udp_socket { append bind connect create getattr getopt ioctl lock read setattr setopt shutdown write };
allow httpd_t httpd_t:udp_socket { append bind connect create getattr getopt ioctl lock read setattr setopt shutdown write }; [ kerberos_enabled ]:True
allow httpd_t httpd_t:udp_socket { append bind connect create getattr getopt ioctl lock read setattr setopt shutdown write }; [ kerberos_enabled ]:True
allow httpd_t httpd_t:udp_socket { append bind connect create getattr getopt ioctl lock read setattr setopt shutdown write }; [ nis_enabled ]:True
allow httpd_t httpd_t:unix_dgram_socket { append bind connect create getattr getopt ioctl lock read sendto setattr setopt shutdown write };
allow httpd_t httpd_t:unix_stream_socket { accept append bind connect connectto create getattr getopt ioctl listen lock read setattr setopt shutdown write };
allow httpd_t httpd_t:unix_stream_socket { accept append bind connect create getattr getopt ioctl listen lock read setattr setopt shutdown write }; [ nscd_use_shm ]:True
allow httpd_t httpd_t:unix_stream_socket { accept append bind connect create getattr getopt ioctl listen lock read setattr setopt shutdown write }; [ nscd_use_shm ]:True
allow httpd_t httpd_t:unix_stream_socket { append bind connect create getattr getopt ioctl lock read setattr setopt shutdown write }; [ nscd_use_shm ]:False
allow httpd_t httpd_t:unix_stream_socket { append bind connect create getattr getopt ioctl lock read setattr setopt shutdown write }; [ nscd_use_shm ]:False
allow httpd_t httpd_user_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t init_t:unix_stream_socket { connectto getattr sendto };
allow httpd_t jboss_management_port_t:tcp_socket name_bind;
allow httpd_t jboss_messaging_port_t:tcp_socket name_bind;
allow httpd_t kerberos_port_t:tcp_socket name_connect; [ kerberos_enabled ]:True
allow httpd_t kerberos_port_t:tcp_socket { recv_msg send_msg }; [ kerberos_enabled ]:True
allow httpd_t kerberos_port_t:udp_socket recv_msg; [ kerberos_enabled ]:True
allow httpd_t kerberos_port_t:udp_socket send_msg; [ kerberos_enabled ]:True
allow httpd_t keystone_cgi_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t ldap_port_t:tcp_socket name_connect; [ httpd_can_connect_ldap ]:True
allow httpd_t man2html_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t mediawiki_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t memcache_port_t:tcp_socket name_connect; [ httpd_can_network_memcache ]:True
allow httpd_t memcache_port_t:tcp_socket name_connect; [ httpd_can_network_relay ]:True
allow httpd_t memcached_t:unix_stream_socket connectto;
allow httpd_t mojomojo_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t mongod_port_t:tcp_socket name_connect; [ httpd_can_network_connect_db ]:True
allow httpd_t mssql_port_t:tcp_socket name_connect; [ httpd_can_network_connect_db ]:True
allow httpd_t munin_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t mysqld_port_t:tcp_socket name_connect; [ httpd_can_network_connect_db ]:True
allow httpd_t mysqld_port_t:tcp_socket { recv_msg send_msg }; [ httpd_can_network_connect_db ]:True
allow httpd_t mysqld_t:tcp_socket recvfrom; [ httpd_can_network_connect_db ]:True
allow httpd_t mysqld_t:unix_stream_socket connectto;
allow httpd_t mythtv_port_t:tcp_socket name_connect; [ httpd_can_connect_mythtv ]:True
allow httpd_t mythtv_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t nagios_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t netlabel_peer_t:tcp_socket recvfrom; [ httpd_can_network_connect_db ]:True
allow httpd_t netlabel_peer_t:tcp_socket recvfrom; [ httpd_can_network_connect_db ]:True
allow httpd_t node_t:tcp_socket node_bind;
allow httpd_t node_t:tcp_socket node_bind; [ kerberos_enabled ]:True
allow httpd_t node_t:udp_socket node_bind;
allow httpd_t node_t:udp_socket node_bind; [ kerberos_enabled ]:True
allow httpd_t ntop_port_t:tcp_socket name_bind;
allow httpd_t nutups_cgi_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t ocsp_port_t:tcp_socket name_connect; [ kerberos_enabled ]:True
allow httpd_t openshift_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t oracle_port_t:tcp_socket name_connect; [ httpd_can_network_connect_db ]:True
allow httpd_t osapi_compute_port_t:tcp_socket name_connect; [ httpd_use_openstack ]:True
allow httpd_t passenger_t:unix_stream_socket connectto; [ httpd_run_stickshift ]:False
allow httpd_t passenger_t:unix_stream_socket connectto; [ httpd_run_stickshift ]:False
allow httpd_t pcscd_t:unix_stream_socket connectto; [ kerberos_enabled ]:True
allow httpd_t pkcs11proxyd_t:unix_stream_socket connectto;
allow httpd_t pop_port_t:tcp_socket name_connect; [ httpd_can_sendmail ]:True
allow httpd_t port_type:tcp_socket name_connect; [ httpd_can_network_connect ]:True
allow httpd_t port_type:tcp_socket { recv_msg send_msg };
allow httpd_t port_type:udp_socket { recv_msg send_msg };
allow httpd_t postgresql_port_t:tcp_socket name_connect; [ httpd_can_network_connect_db ]:True
allow httpd_t postgresql_port_t:tcp_socket { recv_msg send_msg }; [ httpd_can_network_connect_db ]:True
allow httpd_t postgresql_t:tcp_socket recvfrom; [ httpd_can_network_connect_db ]:True
allow httpd_t postgresql_t:unix_stream_socket connectto;
allow httpd_t preupgrade_port_t:tcp_socket name_bind; [ httpd_run_preupgrade ]:True
allow httpd_t prewikka_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t puppet_port_t:tcp_socket name_bind;
allow httpd_t saslauthd_t:unix_stream_socket connectto; [ httpd_use_sasl ]:True
allow httpd_t slapd_t:unix_stream_socket connectto;
allow httpd_t smokeping_cgi_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t smtp_port_t:tcp_socket name_connect; [ httpd_can_sendmail ]:True
allow httpd_t squid_port_t:tcp_socket name_connect; [ httpd_can_network_relay ]:True
allow httpd_t squid_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t system_dbusd_t:unix_stream_socket connectto;
allow httpd_t thin_t:unix_stream_socket connectto;
allow httpd_t w3c_validator_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t webalizer_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t zabbix_port_t:tcp_socket name_connect; [ httpd_can_connect_zabbix ]:True
allow httpd_t zarafa_server_t:unix_stream_socket connectto;
allow httpd_t zoneminder_script_t:unix_dgram_socket sendto; [ httpd_enable_cgi ]:True
allow httpd_t zoneminder_t:unix_stream_socket connectto;
```
        




In [43]:
p.attribute_summary("domain")


### Domain Attribute Summary: domain

####  Types in Attribute

  * `NetworkManager_t`
  * `abrt_dump_oops_t`
  * `abrt_handle_event_t`
  * `abrt_helper_t`
  * `abrt_retrace_coredump_t`
  * `abrt_retrace_worker_t`
  * `abrt_t`
  * `abrt_upload_watch_t`
  * `abrt_watch_log_t`
  * `accountsd_t`
  * `acct_t`
  * `admin_crontab_t`
  * `afs_bosserver_t`
  * `afs_fsserver_t`
  * `afs_kaserver_t`
  * `afs_ptserver_t`
  * `afs_t`
  * `afs_vlserver_t`
  * `aiccu_t`
  * `aide_t`
  * `ajaxterm_ssh_t`
  * `ajaxterm_t`
  * `alsa_t`
  * `amanda_recover_t`
  * `amanda_t`
  * `amtu_t`
  * `anaconda_t`
  * `anon_sftpd_t`
  * `antivirus_t`
  * `apcupsd_cgi_script_t`
  * `apcupsd_t`
  * `apm_t`
  * `apmd_t`
  * `arpwatch_t`
  * `asterisk_t`
  * `audisp_remote_t`
  * `audisp_t`
  * `auditadm_screen_t`
  * `auditadm_su_t`
  * `auditadm_sudo_t`
  * `auditadm_t`
  * `auditctl_t`
  * `auditd_t`
  * `authconfig_t`
  * `automount_t`
  * `avahi_t`
  * `awstats_script_t`
  * `awstats_t`
  * `bacula_admin_t`
  * `bacula_t`
  * `bacula_unconfined_script_t`
  * `bcfg2_t`
  * `bitlbee_t`
  * `blkmapd_t`
  * `blktap_t`
  * `blueman_t`
  * `bluetooth_helper_t`
  * `bluetooth_t`
  * `boinc_project_t`
  * `boinc_t`
  * `bootloader_t`
  * `brctl_t`
  * `brltty_t`
  * `bugzilla_script_t`
  * `bumblebee_t`
  * `cachefiles_kernel_t`
  * `cachefilesd_t`
  * `calamaris_t`
  * `callweaver_t`
  * `canna_t`
  * `cardmgr_t`
  * `ccs_t`
  * `cdcc_t`
  * `cdrecord_t`
  * `certmaster_t`
  * `certmonger_t`
  * `certmonger_unconfined_t`
  * `certwatch_t`
  * `cfengine_execd_t`
  * `cfengine_monitord_t`
  * `cfengine_serverd_t`
  * `cgclear_t`
  * `cgconfig_t`
  * `cgred_t`
  * `checkpc_t`
  * `checkpolicy_t`
  * `chfn_t`
  * `chkpwd_t`
  * `chrome_sandbox_nacl_t`
  * `chrome_sandbox_t`
  * `chronyd_t`
  * `chroot_user_t`
  * `cinder_api_t`
  * `cinder_backup_t`
  * `cinder_scheduler_t`
  * `cinder_volume_t`
  * `ciped_t`
  * `clogd_t`
  * `cloud_init_t`
  * `cluster_t`
  * `clvmd_t`
  * `cmirrord_t`
  * `cobblerd_t`
  * `cockpit_session_t`
  * `cockpit_ws_t`
  * `collectd_script_t`
  * `collectd_t`
  * `colord_t`
  * `comsat_t`
  * `condor_collector_t`
  * `condor_master_t`
  * `condor_negotiator_t`
  * `condor_procd_t`
  * `condor_schedd_t`
  * `condor_startd_ssh_t`
  * `condor_startd_t`
  * `conman_t`
  * `conman_unconfined_script_t`
  * `consolekit_t`
  * `container_t`
  * `couchdb_t`
  * `courier_authdaemon_t`
  * `courier_pcp_t`
  * `courier_pop_t`
  * `courier_sqwebmail_t`
  * `courier_tcpd_t`
  * `cpucontrol_t`
  * `cpufreqselector_t`
  * `cpuplug_t`
  * `cpuspeed_t`
  * `crack_t`
  * `crond_t`
  * `cronjob_t`
  * `crontab_t`
  * `ctdbd_t`
  * `cups_pdf_t`
  * `cupsd_config_t`
  * `cupsd_lpd_t`
  * `cupsd_t`
  * `cvs_script_t`
  * `cvs_t`
  * `cyphesis_t`
  * `cyrus_t`
  * `dbadm_sudo_t`
  * `dbadm_t`
  * `dbskkd_t`
  * `dcc_client_t`
  * `dcc_dbclean_t`
  * `dccd_t`
  * `dccifd_t`
  * `dccm_t`
  * `dcerpcd_t`
  * `ddclient_t`
  * `deltacloudd_t`
  * `denyhosts_t`
  * `depmod_t`
  * `devicekit_disk_t`
  * `devicekit_power_t`
  * `devicekit_t`
  * `dhcpc_t`
  * `dhcpd_t`
  * `dictd_t`
  * `dirsrv_snmp_t`
  * `dirsrv_t`
  * `dirsrvadmin_script_t`
  * `dirsrvadmin_t`
  * `dirsrvadmin_unconfined_script_t`
  * `disk_munin_plugin_t`
  * `dkim_milter_t`
  * `dlm_controld_t`
  * `dmesg_t`
  * `dmidecode_t`
  * `dnsmasq_t`
  * `dnssec_trigger_t`
  * `dovecot_auth_t`
  * `dovecot_deliver_t`
  * `dovecot_t`
  * `drbd_t`
  * `dspam_script_t`
  * `dspam_t`
  * `entropyd_t`
  * `eventlogd_t`
  * `evtchnd_t`
  * `exim_t`
  * `fail2ban_client_t`
  * `fail2ban_t`
  * `fcoemon_t`
  * `fenced_t`
  * `fetchmail_t`
  * `fingerd_t`
  * `firewalld_t`
  * `firewallgui_t`
  * `firstboot_t`
  * `foghorn_t`
  * `fprintd_t`
  * `freeipmi_bmc_watchdog_t`
  * `freeipmi_ipmidetectd_t`
  * `freeipmi_ipmiseld_t`
  * `freqset_t`
  * `fsadm_t`
  * `fsdaemon_t`
  * `ftpd_t`
  * `ftpdctl_t`
  * `fwupd_t`
  * `games_srv_t`
  * `games_t`
  * `gconfd_t`
  * `gconfdefaultsm_t`
  * `gdomap_t`
  * `gear_t`
  * `geoclue_t`
  * `getty_t`
  * `gfs_controld_t`
  * `git_script_t`
  * `git_session_t`
  * `git_system_t`
  * `gitosis_t`
  * `glance_api_t`
  * `glance_registry_t`
  * `glance_scrubber_t`
  * `glusterd_t`
  * `gnomesystemmm_t`
  * `gpg_agent_t`
  * `gpg_helper_t`
  * `gpg_pinentry_t`
  * `gpg_t`
  * `gpg_web_t`
  * `gpm_t`
  * `gpsd_t`
  * `greylist_milter_t`
  * `groupadd_t`
  * `groupd_t`
  * `gssd_t`
  * `gssproxy_t`
  * `guest_t`
  * `haproxy_t`
  * `hddtemp_t`
  * `hostapd_t`
  * `hostname_t`
  * `hsqldb_t`
  * `httpd_helper_t`
  * `httpd_passwd_t`
  * `httpd_php_t`
  * `httpd_rotatelogs_t`
  * `httpd_suexec_t`
  * `httpd_sys_script_t`
  * `httpd_t`
  * `httpd_unconfined_script_t`
  * `httpd_user_script_t`
  * `hwclock_t`
  * `hwloc_dhwd_t`
  * `hypervkvp_t`
  * `hypervvssd_t`
  * `iceauth_t`
  * `icecast_t`
  * `ifconfig_t`
  * `inetd_child_t`
  * `inetd_t`
  * `init_t`
  * `initrc_t`
  * `innd_t`
  * `insmod_t`
  * `install_t`
  * `iodined_t`
  * `iotop_t`
  * `ipa_dnskey_t`
  * `ipa_helper_t`
  * `ipa_ods_exporter_t`
  * `ipa_otpd_t`
  * `ipmievd_t`
  * `ipsec_mgmt_t`
  * `ipsec_t`
  * `iptables_t`
  * `irc_t`
  * `irqbalance_t`
  * `irssi_t`
  * `iscsid_t`
  * `isnsd_t`
  * `iwhd_t`
  * `jabberd_router_t`
  * `jabberd_t`
  * `jetty_t`
  * `jockey_t`
  * `journalctl_t`
  * `kadmind_t`
  * `kdump_t`
  * `kdumpctl_t`
  * `kdumpgui_t`
  * `keepalived_t`
  * `keepalived_unconfined_script_t`
  * `kernel_t`
  * `keyboardd_t`
  * `keystone_cgi_script_t`
  * `keystone_t`
  * `kismet_t`
  * `klogd_t`
  * `kmscon_t`
  * `kpropd_t`
  * `krb5kdc_t`
  * `ksmtuned_t`
  * `ktalkd_t`
  * `l2tpd_t`
  * `ldconfig_t`
  * `lircd_t`
  * `livecd_t`
  * `lldpad_t`
  * `load_policy_t`
  * `loadkeys_t`
  * `local_login_t`
  * `locate_t`
  * `lockdev_t`
  * `logadm_t`
  * `logrotate_mail_t`
  * `logrotate_t`
  * `logwatch_mail_t`
  * `logwatch_t`
  * `lpd_t`
  * `lpr_t`
  * `lsassd_t`
  * `lsmd_plugin_t`
  * `lsmd_t`
  * `lttng_sessiond_t`
  * `lvm_t`
  * `lwiod_t`
  * `lwregd_t`
  * `lwsmd_t`
  * `mail_munin_plugin_t`
  * `mailman_cgi_t`
  * `mailman_mail_t`
  * `mailman_queue_t`
  * `man2html_script_t`
  * `mandb_t`
  * `mcelog_t`
  * `mdadm_t`
  * `mediawiki_script_t`
  * `memcached_t`
  * `mencoder_t`
  * `minidlna_t`
  * `minissdpd_t`
  * `mip6d_t`
  * `mirrormanager_t`
  * `mock_build_t`
  * `mock_t`
  * `modemmanager_t`
  * `mojomojo_script_t`
  * `mon_procd_t`
  * `mon_statd_t`
  * `mongod_t`
  * `motion_t`
  * `mount_ecryptfs_t`
  * `mount_t`
  * `mozilla_plugin_config_t`
  * `mozilla_plugin_t`
  * `mozilla_t`
  * `mpd_t`
  * `mplayer_t`
  * `mrtg_t`
  * `mscan_t`
  * `munin_script_t`
  * `munin_t`
  * `mysqld_safe_t`
  * `mysqld_t`
  * `mysqlmanagerd_t`
  * `mythtv_script_t`
  * `naemon_t`
  * `nagios_admin_plugin_t`
  * `nagios_checkdisk_plugin_t`
  * `nagios_eventhandler_plugin_t`
  * `nagios_mail_plugin_t`
  * `nagios_openshift_plugin_t`
  * `nagios_script_t`
  * `nagios_services_plugin_t`
  * `nagios_system_plugin_t`
  * `nagios_t`
  * `nagios_unconfined_plugin_t`
  * `named_t`
  * `namespace_init_t`
  * `ncftool_t`
  * `ndc_t`
  * `netlabel_mgmt_t`
  * `netlogond_t`
  * `netutils_t`
  * `neutron_t`
  * `newrole_t`
  * `nfsd_t`
  * `ninfod_t`
  * `nmbd_t`
  * `nova_t`
  * `nrpe_t`
  * `nscd_t`
  * `nsd_crond_t`
  * `nsd_t`
  * `nslcd_t`
  * `ntop_t`
  * `ntpd_t`
  * `numad_t`
  * `nut_upsd_t`
  * `nut_upsdrvctl_t`
  * `nut_upsmon_t`
  * `nutups_cgi_script_t`
  * `nx_server_ssh_t`
  * `nx_server_t`
  * `obex_t`
  * `oddjob_mkhomedir_t`
  * `oddjob_t`
  * `openct_t`
  * `opendnssec_t`
  * `openfortivpn_t`
  * `openhpid_t`
  * `openshift_app_t`
  * `openshift_cgroup_read_t`
  * `openshift_cron_t`
  * `openshift_initrc_t`
  * `openshift_net_read_t`
  * `openshift_script_t`
  * `openshift_t`
  * `opensm_t`
  * `openvpn_t`
  * `openvpn_unconfined_script_t`
  * `openvswitch_t`
  * `openwsman_t`
  * `oracleasm_t`
  * `osad_t`
  * `pads_t`
  * `pam_console_t`
  * `pam_timestamp_t`
  * `passenger_t`
  * `passwd_t`
  * `pcp_pmcd_t`
  * `pcp_pmie_t`
  * `pcp_pmlogger_t`
  * `pcp_pmmgr_t`
  * `pcp_pmproxy_t`
  * `pcp_pmwebd_t`
  * `pcscd_t`
  * `pdns_control_t`
  * `pdns_t`
  * `pegasus_openlmi_account_t`
  * `pegasus_openlmi_admin_t`
  * `pegasus_openlmi_logicalfile_t`
  * `pegasus_openlmi_services_t`
  * `pegasus_openlmi_storage_t`
  * `pegasus_openlmi_system_t`
  * `pegasus_openlmi_unconfined_t`
  * `pegasus_t`
  * `pesign_t`
  * `phc2sys_t`
  * `ping_t`
  * `pingd_t`
  * `piranha_fos_t`
  * `piranha_lvs_t`
  * `piranha_pulse_t`
  * `piranha_web_t`
  * `pkcs11proxyd_t`
  * `pkcs_slotd_t`
  * `pki_ra_t`
  * `pki_tomcat_script_t`
  * `pki_tomcat_t`
  * `pki_tps_t`
  * `plymouth_t`
  * `plymouthd_t`
  * `podsleuth_t`
  * `policykit_auth_t`
  * `policykit_grant_t`
  * `policykit_resolve_t`
  * `policykit_t`
  * `polipo_session_t`
  * `polipo_t`
  * `portmap_helper_t`
  * `portmap_t`
  * `portreserve_t`
  * `postfix_bounce_t`
  * `postfix_cleanup_t`
  * `postfix_local_t`
  * `postfix_map_t`
  * `postfix_master_t`
  * `postfix_pickup_t`
  * `postfix_pipe_t`
  * `postfix_postdrop_t`
  * `postfix_postqueue_t`
  * `postfix_qmgr_t`
  * `postfix_showq_t`
  * `postfix_smtp_t`
  * `postfix_smtpd_t`
  * `postfix_virtual_t`
  * `postgresql_t`
  * `postgrey_t`
  * `pppd_t`
  * `pptp_t`
  * `prelink_cron_system_t`
  * `prelink_t`
  * `prelude_audisp_t`
  * `prelude_correlator_t`
  * `prelude_lml_t`
  * `prelude_t`
  * `preupgrade_t`
  * `prewikka_script_t`
  * `privoxy_t`
  * `procmail_t`
  * `prosody_t`
  * `psad_t`
  * `ptal_t`
  * `ptchown_t`
  * `ptp4l_t`
  * `publicfile_t`
  * `pulseaudio_t`
  * `puppetagent_t`
  * `puppetca_t`
  * `puppetmaster_t`
  * `pwauth_t`
  * `pyicqt_t`
  * `qdiskd_t`
  * `qemu_dm_t`
  * `qmail_clean_t`
  * `qmail_inject_t`
  * `qmail_local_t`
  * `qmail_lspawn_t`
  * `qmail_queue_t`
  * `qmail_remote_t`
  * `qmail_rspawn_t`
  * `qmail_send_t`
  * `qmail_smtpd_t`
  * `qmail_splogger_t`
  * `qmail_start_t`
  * `qmail_tcp_env_t`
  * `qpidd_t`
  * `quota_nld_t`
  * `quota_t`
  * `rabbitmq_t`
  * `racoon_t`
  * `radiusd_t`
  * `radvd_t`
  * `rasdaemon_t`
  * `rdisc_t`
  * `readahead_t`
  * `realmd_consolehelper_t`
  * `realmd_t`
  * `redis_t`
  * `regex_milter_t`
  * `remote_login_t`
  * `restorecond_t`
  * `rhev_agentd_consolehelper_t`
  * `rhev_agentd_t`
  * `rhgb_t`
  * `rhnsd_t`
  * `rhsmcertd_t`
  * `ricci_modcluster_t`
  * `ricci_modclusterd_t`
  * `ricci_modlog_t`
  * `ricci_modrpm_t`
  * `ricci_modservice_t`
  * `ricci_modstorage_t`
  * `ricci_t`
  * `rkt_t`
  * `rlogind_t`
  * `rngd_t`
  * `rolekit_t`
  * `roundup_t`
  * `rpcbind_t`
  * `rpcd_t`
  * `rpm_script_t`
  * `rpm_t`
  * `rshd_t`
  * `rssh_chroot_helper_t`
  * `rssh_t`
  * `rsync_t`
  * `rtas_errd_t`
  * `rtkit_daemon_t`
  * `run_init_t`
  * `rwho_t`
  * `samba_net_t`
  * `samba_unconfined_net_t`
  * `samba_unconfined_script_t`
  * `sambagui_t`
  * `sandbox_min_client_t`
  * `sandbox_min_t`
  * `sandbox_net_client_t`
  * `sandbox_net_t`
  * `sandbox_t`
  * `sandbox_web_client_t`
  * `sandbox_web_t`
  * `sandbox_x_client_t`
  * `sandbox_x_t`
  * `sandbox_xserver_t`
  * `sanlk_resetd_t`
  * `sanlock_t`
  * `saslauthd_t`
  * `sbd_t`
  * `sblim_gatherd_t`
  * `sblim_reposd_t`
  * `sblim_sfcbd_t`
  * `secadm_screen_t`
  * `secadm_su_t`
  * `secadm_sudo_t`
  * `secadm_t`
  * `sectoolm_t`
  * `selinux_munin_plugin_t`
  * `semanage_t`
  * `sendmail_t`
  * `sensord_t`
  * `sepgsql_ranged_proc_t`
  * `sepgsql_trusted_proc_t`
  * `services_munin_plugin_t`
  * `setfiles_mac_t`
  * `setfiles_t`
  * `setkey_t`
  * `setrans_t`
  * `setroubleshoot_fixit_t`
  * `setroubleshootd_t`
  * `setsebool_t`
  * `sftpd_t`
  * `sge_execd_t`
  * `sge_job_ssh_t`
  * `sge_job_t`
  * `sge_shepherd_t`
  * `shorewall_t`
  * `showmount_t`
  * `slapd_t`
  * `slpd_t`
  * `smbcontrol_t`
  * `smbd_t`
  * `smbmount_t`
  * `smokeping_cgi_script_t`
  * `smokeping_t`
  * `smoltclient_t`
  * `smsd_t`
  * `snapperd_t`
  * `snmpd_t`
  * `snort_t`
  * `sosreport_t`
  * `soundd_t`
  * `spamass_milter_t`
  * `spamc_t`
  * `spamd_t`
  * `spamd_update_t`
  * `speech-dispatcher_t`
  * `squid_cron_t`
  * `squid_script_t`
  * `squid_t`
  * `srvsvcd_t`
  * `ssh_keygen_t`
  * `ssh_keysign_t`
  * `ssh_t`
  * `sshd_keygen_t`
  * `sshd_net_t`
  * `sshd_sandbox_t`
  * `sshd_t`
  * `sslh_t`
  * `sssd_selinux_manager_t`
  * `sssd_t`
  * `staff_consolehelper_t`
  * `staff_dbusd_t`
  * `staff_gkeyringd_t`
  * `staff_screen_t`
  * `staff_seunshare_t`
  * `staff_ssh_agent_t`
  * `staff_sudo_t`
  * `staff_t`
  * `staff_wine_t`
  * `stapserver_t`
  * `stunnel_t`
  * `sulogin_t`
  * `svc_multilog_t`
  * `svc_run_t`
  * `svc_start_t`
  * `svirt_kvm_net_t`
  * `svirt_qemu_net_t`
  * `svirt_socket_t`
  * `svirt_t`
  * `svirt_tcg_t`
  * `svnserve_t`
  * `swat_t`
  * `swift_t`
  * `sysadm_dbusd_t`
  * `sysadm_passwd_t`
  * `sysadm_screen_t`
  * `sysadm_seunshare_t`
  * `sysadm_ssh_agent_t`
  * `sysadm_su_t`
  * `sysadm_sudo_t`
  * `sysadm_t`
  * `syslogd_t`
  * `sysstat_t`
  * `system_cronjob_t`
  * `system_dbusd_t`
  * `system_mail_t`
  * `system_munin_plugin_t`
  * `systemd_coredump_t`
  * `systemd_gpt_generator_t`
  * `systemd_hostnamed_t`
  * `systemd_hwdb_t`
  * `systemd_localed_t`
  * `systemd_logger_t`
  * `systemd_logind_t`
  * `systemd_machined_t`
  * `systemd_modules_load_t`
  * `systemd_networkd_t`
  * `systemd_notify_t`
  * `systemd_passwd_agent_t`
  * `systemd_resolved_t`
  * `systemd_rfkill_t`
  * `systemd_sysctl_t`
  * `systemd_timedated_t`
  * `systemd_tmpfiles_t`
  * `targetd_t`
  * `tcpd_t`
  * `tcsd_t`
  * `telepathy_gabble_t`
  * `telepathy_idle_t`
  * `telepathy_logger_t`
  * `telepathy_mission_control_t`
  * `telepathy_msn_t`
  * `telepathy_salut_t`
  * `telepathy_sofiasip_t`
  * `telepathy_stream_engine_t`
  * `telepathy_sunshine_t`
  * `telnetd_t`
  * `tftpd_t`
  * `tgtd_t`
  * `thin_aeolus_configserver_t`
  * `thin_t`
  * `thumb_t`
  * `timemaster_t`
  * `tmpreaper_t`
  * `tomcat_t`
  * `tor_t`
  * `traceroute_t`
  * `tuned_t`
  * `tvtime_t`
  * `udev_t`
  * `ulogd_t`
  * `uml_switch_t`
  * `uml_t`
  * `unconfined_cronjob_t`
  * `unconfined_dbusd_t`
  * `unconfined_mount_t`
  * `unconfined_munin_plugin_t`
  * `unconfined_sendmail_t`
  * `unconfined_service_t`
  * `unconfined_t`
  * `update_modules_t`
  * `updfstab_t`
  * `updpwd_t`
  * `usbmodules_t`
  * `usbmuxd_t`
  * `user_dbusd_t`
  * `user_gkeyringd_t`
  * `user_mail_t`
  * `user_screen_t`
  * `user_seunshare_t`
  * `user_ssh_agent_t`
  * `user_t`
  * `user_wine_t`
  * `useradd_t`
  * `usernetctl_t`
  * `utempter_t`
  * `uucpd_t`
  * `uuidd_t`
  * `uux_t`
  * `varnishd_t`
  * `varnishlog_t`
  * `vdagent_t`
  * `vhostmd_t`
  * `virsh_ssh_t`
  * `virsh_t`
  * `virt_bridgehelper_t`
  * `virt_qemu_ga_t`
  * `virt_qemu_ga_unconfined_t`
  * `virt_qmf_t`
  * `virtd_lxc_t`
  * `virtd_t`
  * `virtlogd_t`
  * `vlock_t`
  * `vmtools_helper_t`
  * `vmtools_t`
  * `vmware_host_t`
  * `vmware_t`
  * `vnstat_t`
  * `vnstatd_t`
  * `vpnc_t`
  * `w3c_validator_script_t`
  * `watchdog_t`
  * `watchdog_unconfined_t`
  * `wdmd_t`
  * `webadm_t`
  * `webalizer_script_t`
  * `webalizer_t`
  * `winbind_helper_t`
  * `winbind_t`
  * `wine_t`
  * `wireshark_t`
  * `wpa_cli_t`
  * `xauth_t`
  * `xdm_t`
  * `xdm_unconfined_t`
  * `xenconsoled_t`
  * `xend_t`
  * `xenstored_t`
  * `xguest_dbusd_t`
  * `xguest_gkeyringd_t`
  * `xguest_t`
  * `xserver_t`
  * `ypbind_t`
  * `yppasswdd_t`
  * `ypserv_t`
  * `ypxfr_t`
  * `zabbix_agent_t`
  * `zabbix_script_t`
  * `zabbix_t`
  * `zarafa_deliver_t`
  * `zarafa_gateway_t`
  * `zarafa_ical_t`
  * `zarafa_indexer_t`
  * `zarafa_monitor_t`
  * `zarafa_server_t`
  * `zarafa_spooler_t`
  * `zebra_t`
  * `zoneminder_script_t`
  * `zoneminder_t`
  * `zos_remote_t`


#### Capability Permissions


```

```
        

#### File Read Permissions


```
allow domain abrt_helper_exec_t:file { execute getattr open read };
allow domain abrt_t:dir { getattr ioctl lock open read search };
allow domain abrt_t:file { getattr ioctl lock open read };
allow domain abrt_t:lnk_file { getattr read };
allow domain abrt_var_run_t:file { getattr ioctl lock open read };
allow domain admin_home_t:lnk_file { getattr read };
allow domain afs_cache_t:file { read write };
allow domain base_ro_file_type:dir { getattr ioctl lock open read search };
allow domain base_ro_file_type:file { getattr ioctl lock open read };
allow domain base_ro_file_type:lnk_file { getattr read };
allow domain bin_t:lnk_file { getattr read };
allow domain bin_t:lnk_file { getattr read }; [ fips_mode ]:True
allow domain cpu_online_t:file { getattr ioctl lock open read };
allow domain device_t:dir { getattr ioctl lock open read search };
allow domain device_t:lnk_file { getattr read };
allow domain devtty_t:chr_file { append getattr ioctl lock open read write };
allow domain etc_t:dir { getattr ioctl lock open read search };
allow domain etc_t:lnk_file { getattr read };
allow domain fonts_cache_t:dir { getattr ioctl lock open read search };
allow domain fonts_cache_t:file { getattr ioctl lock open read };
allow domain fonts_cache_t:lnk_file { getattr read };
allow domain fonts_t:dir { getattr ioctl lock open read search };
allow domain fonts_t:file { getattr ioctl lock open read };
allow domain fonts_t:lnk_file { getattr read };
allow domain ld_so_cache_t:file { getattr ioctl lock open read };
allow domain ld_so_t:file { execute getattr ioctl open read };
allow domain ld_so_t:lnk_file { getattr read };
allow domain lib_t:dir { getattr ioctl lock open read search };
allow domain lib_t:file { execute getattr ioctl lock open read };
allow domain lib_t:lnk_file { getattr read };
allow domain locale_t:dir { getattr ioctl lock open read search };
allow domain locale_t:file { getattr ioctl lock open read };
allow domain locale_t:lnk_file { getattr read };
allow domain machineid_t:file { getattr ioctl lock open read };
allow domain man_cache_t:dir { getattr ioctl lock open read search };
allow domain man_cache_t:file { getattr ioctl lock open read };
allow domain man_cache_t:lnk_file { getattr read };
allow domain man_t:dir { getattr ioctl lock open read search };
allow domain man_t:file { getattr ioctl lock open read };
allow domain man_t:lnk_file { getattr read };
allow domain mandb_cache_t:file { getattr ioctl lock open read };
allow domain null_device_t:chr_file { append getattr ioctl lock open read write };
allow domain prelink_exec_t:file { execute execute_no_trans getattr ioctl lock open read }; [ fips_mode ]:True
allow domain proc_t:lnk_file { getattr read };
allow domain puppet_tmp_t:file { append getattr ioctl lock read write };
allow domain root_t:dir { getattr ioctl lock open read search };
allow domain root_t:lnk_file { getattr ioctl lock read };
allow domain rpm_script_tmp_t:file { getattr ioctl lock open read };
allow domain rpm_script_tmp_t:lnk_file { getattr read };
allow domain security_t:lnk_file { getattr read };
allow domain sysctl_crypto_t:dir { getattr ioctl lock open read search };
allow domain sysctl_crypto_t:file { getattr ioctl lock open read };
allow domain sysctl_kernel_t:dir { getattr ioctl lock open read search }; [ fips_mode ]:True
allow domain sysctl_kernel_t:file { getattr ioctl lock open read }; [ fips_mode ]:True
allow domain sysctl_vm_overcommit_t:file { getattr ioctl lock open read };
allow domain textrel_shlib_t:dir { getattr ioctl lock open read search };
allow domain textrel_shlib_t:file { execmod execute getattr ioctl open read };
allow domain textrel_shlib_t:lnk_file { getattr read };
allow domain tmp_t:lnk_file { getattr read };
allow domain tmpfile:file { append getattr ioctl lock read };
allow domain urandom_device_t:chr_file { getattr ioctl lock open read };
allow domain urandom_device_t:chr_file { getattr ioctl lock open read }; [ global_ssp ]:True
allow domain usr_t:dir { getattr ioctl lock open read search };
allow domain var_run_t:dir { getattr ioctl lock open read search };
allow domain var_run_t:lnk_file { getattr read };
allow domain var_t:lnk_file { getattr read };
allow domain zero_device_t:chr_file { append getattr ioctl lock open read write };
```
        

#### File Write Permissions


```
allow domain abrt_var_cache_t:file { append getattr };
allow domain afs_cache_t:file { read write };
allow domain devtty_t:chr_file { append getattr ioctl lock open read write };
allow domain null_device_t:chr_file { append getattr ioctl lock open read write };
allow domain puppet_tmp_t:file { append getattr ioctl lock read write };
allow domain rkhunter_var_lib_t:file { append getattr ioctl lock open };
allow domain rpm_tmp_t:file { append getattr };
allow domain sosreport_tmp_t:file { append getattr ioctl lock open };
allow domain tmpfile:file { append getattr ioctl lock read };
allow domain zero_device_t:chr_file { append getattr ioctl lock open read write };
```
        


#### Packet Permissions


```
allow domain unlabeled_t:packet { recv send };
```
        

#### Process Permissions


```
allow domain abrt_dump_oops_t:process sigchld;
allow domain abrt_helper_t:process transition;
allow domain abrt_t:process { getattr signull };
allow domain init_t:process { sigchld signull };
allow domain install_t:process sigchld; [ deny_ptrace ]:False
allow domain livecd_t:process sigchld; [ deny_ptrace ]:False
allow domain sysadm_t:process sigchld; [ deny_ptrace ]:False
allow domain unconfined_t:process sigchld;
```
        

#### Socket Permissions


```
allow domain afs_t:udp_socket { read write };
allow domain netlabel_peer_t:dccp_socket recvfrom;
allow domain netlabel_peer_t:tcp_socket recvfrom;
allow domain setrans_t:unix_stream_socket connectto;
allow domain unconfined_domain_type:dccp_socket recvfrom;
allow domain unconfined_domain_type:tcp_socket recvfrom;
allow domain unconfined_service_t:unix_stream_socket { connectto getattr };
```
        



In [44]:
p.file_summary("bin_t")


### File Type Summary: bin_t

####  Type Attributes

  * `base_file_type`
  * `base_ro_file_type`
  * `entry_type`
  * `exec_type`
  * `file_type`
  * `non_auth_file_type`
  * `non_security_file_type`


#### Transitions


```
source
target
tclass
default
filename
cond
```
        

#### File Read Permissions


```
allow NetworkManager_t bin_t:dir { getattr ioctl lock open read search };
allow NetworkManager_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow NetworkManager_t bin_t:lnk_file { getattr read };
allow abrt_dump_oops_t bin_t:dir { getattr ioctl lock open read search };
allow abrt_retrace_coredump_t bin_t:dir { getattr ioctl lock open read search };
allow abrt_retrace_coredump_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow abrt_retrace_coredump_t bin_t:lnk_file { getattr read };
allow abrt_retrace_worker_t bin_t:dir { getattr ioctl lock open read search };
allow abrt_retrace_worker_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow abrt_retrace_worker_t bin_t:lnk_file { getattr read };
allow abrt_t bin_t:dir { getattr ioctl lock open read search };
allow abrt_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow abrt_t bin_t:lnk_file { getattr read };
allow abrt_upload_watch_t bin_t:dir { getattr ioctl lock open read search };
allow abrt_upload_watch_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow abrt_upload_watch_t bin_t:lnk_file { getattr read };
allow abrt_watch_log_t bin_t:dir { getattr ioctl lock open read search };
allow abrt_watch_log_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow abrt_watch_log_t bin_t:lnk_file { getattr read };
allow accountsd_t bin_t:dir { getattr ioctl lock open read search };
allow accountsd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow accountsd_t bin_t:lnk_file { getattr read };
allow acct_t bin_t:dir { getattr ioctl lock open read search };
allow acct_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow acct_t bin_t:lnk_file { getattr read };
allow admin_crontab_t bin_t:dir { getattr ioctl lock open read search };
allow admin_crontab_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow admin_crontab_t bin_t:lnk_file { getattr read };
allow aiccu_t bin_t:dir { getattr ioctl lock open read search };
allow aiccu_t bin_t:lnk_file { getattr read };
allow aide_t bin_t:lnk_file { getattr read };
allow ajaxterm_ssh_t bin_t:dir { getattr ioctl lock open read search };
allow ajaxterm_ssh_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ajaxterm_ssh_t bin_t:lnk_file { getattr read };
allow ajaxterm_t bin_t:dir { getattr ioctl lock open read search };
allow ajaxterm_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ajaxterm_t bin_t:lnk_file { getattr read };
allow alsa_t bin_t:dir { getattr ioctl lock open read search };
allow alsa_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow alsa_t bin_t:lnk_file { getattr read };
allow amanda_recover_t bin_t:dir { getattr ioctl lock open read search };
allow amanda_recover_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow amanda_recover_t bin_t:lnk_file { getattr read };
allow amanda_t bin_t:dir { getattr ioctl lock open read search };
allow amanda_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow amanda_t bin_t:lnk_file { getattr read };
allow anaconda_t bin_t:dir { getattr ioctl lock open read search };
allow anaconda_t bin_t:lnk_file { getattr read };
allow antivirus_domain bin_t:dir { getattr ioctl lock open read search };
allow antivirus_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow antivirus_domain bin_t:lnk_file { getattr read };
allow apcupsd_t bin_t:dir { getattr ioctl lock open read search };
allow apcupsd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow apcupsd_t bin_t:lnk_file { getattr read };
allow apmd_t bin_t:dir { getattr ioctl lock open read search };
allow apmd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow apmd_t bin_t:lnk_file { getattr read };
allow arpwatch_t bin_t:lnk_file { getattr read };
allow asterisk_t bin_t:dir { getattr ioctl lock open read search };
allow asterisk_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow asterisk_t bin_t:lnk_file { getattr read };
allow audisp_remote_t bin_t:dir { getattr ioctl lock open read search };
allow audisp_remote_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow audisp_remote_t bin_t:lnk_file { getattr read };
allow audisp_t bin_t:dir { getattr ioctl lock open read search };
allow audisp_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow audisp_t bin_t:lnk_file { getattr read };
allow auditadm_screen_t bin_t:dir { getattr ioctl lock open read search };
allow auditadm_screen_t bin_t:file { execute getattr open read };
allow auditadm_screen_t bin_t:lnk_file { getattr read };
allow auditadm_su_t bin_t:dir { getattr ioctl lock open read search };
allow auditadm_su_t bin_t:lnk_file { getattr read };
allow auditadm_sudo_t bin_t:dir { getattr ioctl lock open read search };
allow auditadm_sudo_t bin_t:file { execute getattr open read };
allow auditadm_sudo_t bin_t:lnk_file { getattr read };
allow auditadm_t bin_t:dir { getattr ioctl lock open read search };
allow auditadm_t bin_t:lnk_file { getattr read };
allow auditd_t bin_t:dir { getattr ioctl lock open read search };
allow auditd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow auditd_t bin_t:lnk_file { getattr read };
allow automount_t bin_t:dir { getattr ioctl lock open read search };
allow automount_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow automount_t bin_t:lnk_file { getattr read };
allow avahi_t bin_t:dir { getattr ioctl lock open read search };
allow avahi_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow avahi_t bin_t:lnk_file { getattr read };
allow awstats_t bin_t:dir { getattr ioctl lock open read search };
allow awstats_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow awstats_t bin_t:lnk_file { getattr read };
allow bacula_t bin_t:dir { getattr ioctl lock open read search };
allow bacula_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow bacula_t bin_t:lnk_file { getattr read };
allow bcfg2_t bin_t:dir { getattr ioctl lock open read search };
allow bcfg2_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow bcfg2_t bin_t:lnk_file { getattr read };
allow bin_t bin_t:lnk_file { getattr read };
allow blueman_t bin_t:dir { getattr ioctl lock open read search };
allow blueman_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow blueman_t bin_t:lnk_file { getattr read };
allow bluetooth_helper_t bin_t:dir { getattr ioctl lock open read search };
allow bluetooth_helper_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow bluetooth_helper_t bin_t:lnk_file { getattr read };
allow bluetooth_t bin_t:dir { getattr ioctl lock open read search };
allow bluetooth_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow bluetooth_t bin_t:lnk_file { getattr read };
allow boinc_domain bin_t:dir { getattr ioctl lock open read search };
allow boinc_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow boinc_domain bin_t:lnk_file { getattr read };
allow boinc_project_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow boinc_project_t bin_t:lnk_file { getattr read };
allow boinc_t bin_t:lnk_file { getattr read };
allow bootloader_t bin_t:dir { getattr ioctl lock open read search };
allow bootloader_t bin_t:lnk_file { getattr read };
allow brltty_t bin_t:lnk_file { getattr read };
allow bugzilla_script_t bin_t:lnk_file { getattr read };
allow bumblebee_t bin_t:dir { getattr ioctl lock open read search };
allow bumblebee_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow bumblebee_t bin_t:lnk_file { getattr read };
allow calamaris_t bin_t:dir { getattr ioctl lock open read search };
allow calamaris_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow calamaris_t bin_t:lnk_file { getattr read };
allow cardmgr_t bin_t:dir { getattr ioctl lock open read search };
allow cardmgr_t bin_t:lnk_file { getattr read };
allow ccs_t bin_t:dir { getattr ioctl lock open read search };
allow ccs_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ccs_t bin_t:lnk_file { getattr read };
allow cdrecord_t bin_t:dir { getattr ioctl lock open read search };
allow cdrecord_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow cdrecord_t bin_t:lnk_file { getattr read };
allow certmaster_t bin_t:dir { getattr ioctl lock open read search };
allow certmaster_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow certmaster_t bin_t:lnk_file { getattr read };
allow certmonger_t bin_t:dir { getattr ioctl lock open read search };
allow certmonger_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow certmonger_t bin_t:lnk_file { getattr read };
allow certwatch_t bin_t:dir { getattr ioctl lock open read search };
allow certwatch_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow certwatch_t bin_t:lnk_file { getattr read };
allow cfengine_domain bin_t:dir { getattr ioctl lock open read search };
allow cfengine_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow cfengine_domain bin_t:lnk_file { getattr read };
allow checkpc_t bin_t:dir { getattr ioctl lock open read search };
allow checkpc_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow checkpc_t bin_t:lnk_file { getattr read };
allow chfn_t bin_t:dir { getattr ioctl lock open read search };
allow chfn_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow chfn_t bin_t:lnk_file { getattr read };
allow chrome_sandbox_nacl_t bin_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow chrome_sandbox_t bin_t:dir { getattr ioctl lock open read search };
allow chrome_sandbox_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow chrome_sandbox_t bin_t:lnk_file { getattr read };
allow chronyd_t bin_t:dir { getattr ioctl lock open read search };
allow chronyd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow chronyd_t bin_t:lnk_file { getattr read };
allow chroot_user_t bin_t:dir { getattr ioctl lock open read search };
allow chroot_user_t bin_t:lnk_file { getattr read };
allow cinder_api_t bin_t:lnk_file { getattr read };
allow cinder_domain bin_t:dir { getattr ioctl lock open read search };
allow cinder_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow cinder_domain bin_t:lnk_file { getattr read };
allow cinder_volume_t bin_t:lnk_file { getattr read };
allow ciped_t bin_t:dir { getattr ioctl lock open read search };
allow ciped_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ciped_t bin_t:lnk_file { getattr read };
allow cloud_init_t bin_t:dir { getattr ioctl lock open read search };
allow cloud_init_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow cloud_init_t bin_t:lnk_file { getattr read };
allow cluster_t bin_t:dir { getattr ioctl lock open read search };
allow cluster_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow cluster_t bin_t:lnk_file { getattr read };
allow clvmd_t bin_t:dir { getattr ioctl lock open read search };
allow clvmd_t bin_t:lnk_file { getattr read };
allow cobblerd_t bin_t:dir { getattr ioctl lock open read search };
allow cobblerd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow cobblerd_t bin_t:lnk_file { getattr read };
allow cockpit_session_t bin_t:dir { getattr ioctl lock open read search };
allow cockpit_session_t bin_t:lnk_file { getattr read };
allow cockpit_ws_t bin_t:lnk_file { getattr read };
allow collectd_t bin_t:lnk_file { getattr read };
allow colord_t bin_t:dir { getattr ioctl lock open read search };
allow colord_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow colord_t bin_t:lnk_file { getattr read };
allow condor_domain bin_t:dir { getattr ioctl lock open read search };
allow condor_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow condor_domain bin_t:lnk_file { getattr read };
allow condor_master_t bin_t:lnk_file { getattr read };
allow condor_startd_ssh_t bin_t:dir { getattr ioctl lock open read search };
allow condor_startd_ssh_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow condor_startd_ssh_t bin_t:lnk_file { getattr read };
allow confined_admindomain bin_t:dir { getattr ioctl lock open read search };
allow confined_admindomain bin_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow confined_admindomain bin_t:lnk_file { getattr read };
allow conman_t bin_t:dir { getattr ioctl lock open read search };
allow conman_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow conman_t bin_t:lnk_file { getattr read };
allow consolehelper_domain bin_t:dir { getattr ioctl lock open read search };
allow consolehelper_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow consolehelper_domain bin_t:lnk_file { getattr read };
allow consolekit_t bin_t:dir { getattr ioctl lock open read search };
allow consolekit_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow consolekit_t bin_t:lnk_file { getattr read };
allow couchdb_t bin_t:dir { getattr ioctl lock open read search };
allow couchdb_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow couchdb_t bin_t:lnk_file { getattr read };
allow courier_authdaemon_t bin_t:lnk_file { getattr read };
allow courier_domain bin_t:dir { getattr ioctl lock open read search };
allow courier_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow courier_domain bin_t:lnk_file { getattr read };
allow crack_t bin_t:dir { getattr ioctl lock open read search };
allow crack_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow crack_t bin_t:lnk_file { getattr read };
allow crond_t bin_t:dir { getattr ioctl lock open read search };
allow crond_t bin_t:dir { getattr ioctl lock open read search }; [ polyinstantiation_enabled ]:True
allow crond_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow crond_t bin_t:file { execute execute_no_trans getattr ioctl lock open read }; [ polyinstantiation_enabled ]:True
allow crond_t bin_t:lnk_file { getattr read };
allow crond_t bin_t:lnk_file { getattr read }; [ polyinstantiation_enabled ]:True
allow crond_t bin_t:lnk_file { getattr read }; [ polyinstantiation_enabled ]:True
allow cronjob_t bin_t:dir { getattr ioctl lock open read search };
allow crontab_domain bin_t:dir { getattr ioctl lock open read search };
allow crontab_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow crontab_domain bin_t:lnk_file { getattr read };
allow crontab_t bin_t:dir { getattr ioctl lock open read search };
allow crontab_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow crontab_t bin_t:lnk_file { getattr read };
allow ctdbd_t bin_t:dir { getattr ioctl lock open read search };
allow ctdbd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ctdbd_t bin_t:lnk_file { getattr read };
allow cups_domain bin_t:dir { getattr ioctl lock open read search };
allow cups_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow cups_domain bin_t:lnk_file { getattr read };
allow cupsd_config_t bin_t:lnk_file { getattr read };
allow cupsd_t bin_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow cupsd_t bin_t:lnk_file { getattr read };
allow cvs_t bin_t:dir { getattr ioctl lock open read search };
allow cvs_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow cvs_t bin_t:lnk_file { getattr read };
allow cyphesis_t bin_t:lnk_file { getattr read };
allow cyrus_t bin_t:dir { getattr ioctl lock open read search };
allow cyrus_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow cyrus_t bin_t:lnk_file { getattr read };
allow dbadm_sudo_t bin_t:dir { getattr ioctl lock open read search };
allow dbadm_sudo_t bin_t:file { execute getattr open read };
allow dbadm_sudo_t bin_t:lnk_file { getattr read };
allow dbadm_t bin_t:lnk_file { getattr read };
allow dccd_t bin_t:lnk_file { getattr read };
allow ddclient_t bin_t:dir { getattr ioctl lock open read search };
allow ddclient_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ddclient_t bin_t:lnk_file { getattr read };
allow deltacloudd_t bin_t:dir { getattr ioctl lock open read search };
allow deltacloudd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow deltacloudd_t bin_t:lnk_file { getattr read };
allow denyhosts_t bin_t:dir { getattr ioctl lock open read search };
allow denyhosts_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow denyhosts_t bin_t:lnk_file { getattr read };
allow depmod_t bin_t:lnk_file { getattr read };
allow devicekit_disk_t bin_t:dir { getattr ioctl lock open read search };
allow devicekit_disk_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow devicekit_disk_t bin_t:lnk_file { getattr read };
allow devicekit_power_t bin_t:dir { getattr ioctl lock open read search };
allow devicekit_power_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow devicekit_power_t bin_t:lnk_file { getattr read };
allow dhcpc_t bin_t:dir { getattr ioctl lock open read search };
allow dhcpc_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow dhcpc_t bin_t:lnk_file { getattr read };
allow dhcpc_t bin_t:lnk_file { getattr read }; [ dhcpc_exec_iptables ]:True
allow dhcpd_t bin_t:dir { getattr ioctl lock open read search };
allow dhcpd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow dhcpd_t bin_t:lnk_file { getattr read };
allow dirsrv_t bin_t:lnk_file { getattr read };
allow dirsrvadmin_script_t bin_t:lnk_file { getattr read };
allow dirsrvadmin_t bin_t:dir { getattr ioctl lock open read search };
allow dirsrvadmin_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow dirsrvadmin_t bin_t:lnk_file { getattr read };
allow disk_munin_plugin_t bin_t:lnk_file { getattr read };
allow dlm_controld_t bin_t:dir { getattr ioctl lock open read search };
allow dlm_controld_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow dlm_controld_t bin_t:lnk_file { getattr read };
allow dnsmasq_t bin_t:dir { getattr ioctl lock open read search };
allow dnsmasq_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow dnsmasq_t bin_t:lnk_file { getattr read };
allow dnssec_trigger_t bin_t:dir { getattr ioctl lock open read search };
allow dnssec_trigger_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow dnssec_trigger_t bin_t:lnk_file { getattr read };
allow domain bin_t:lnk_file { getattr read };
allow domain bin_t:lnk_file { getattr read }; [ fips_mode ]:True
allow dovecot_auth_t bin_t:dir { getattr ioctl lock open read search };
allow dovecot_auth_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow dovecot_auth_t bin_t:lnk_file { getattr read };
allow dovecot_deliver_t bin_t:lnk_file { getattr read };
allow dovecot_domain bin_t:dir { getattr ioctl lock open read search };
allow dovecot_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow dovecot_domain bin_t:lnk_file { getattr read };
allow dovecot_t bin_t:lnk_file { getattr read };
allow drbd_t bin_t:dir { getattr ioctl lock open read search };
allow drbd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow drbd_t bin_t:lnk_file { getattr read };
allow dspam_script_t bin_t:lnk_file { getattr read };
allow dspam_t bin_t:dir { getattr ioctl lock open read search };
allow dspam_t bin_t:lnk_file { getattr read };
allow exim_t bin_t:lnk_file { getattr read };
allow fail2ban_client_t bin_t:dir { getattr ioctl lock open read search };
allow fail2ban_client_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow fail2ban_client_t bin_t:lnk_file { getattr read };
allow fail2ban_t bin_t:dir { getattr ioctl lock open read search };
allow fail2ban_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow fail2ban_t bin_t:lnk_file { getattr read };
allow fenced_t bin_t:dir { getattr ioctl lock open read search };
allow fenced_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow fenced_t bin_t:lnk_file { getattr read };
allow fenced_t bin_t:lnk_file { getattr read }; [ fenced_can_ssh ]:True
allow fetchmail_t bin_t:dir { getattr ioctl lock open read search };
allow fetchmail_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow fetchmail_t bin_t:lnk_file { getattr read };
allow fingerd_t bin_t:dir { getattr ioctl lock open read search };
allow fingerd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow fingerd_t bin_t:lnk_file { getattr read };
allow firewalld_t bin_t:dir { getattr ioctl lock open read search };
allow firewalld_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow firewalld_t bin_t:lnk_file { getattr read };
allow firewallgui_t bin_t:dir { getattr ioctl lock open read search };
allow firewallgui_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow firewallgui_t bin_t:lnk_file { getattr read };
allow firstboot_t bin_t:dir { getattr ioctl lock open read search };
allow firstboot_t bin_t:lnk_file { getattr read };
allow fprintd_t bin_t:dir { getattr ioctl lock open read search };
allow fprintd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow fprintd_t bin_t:lnk_file { getattr read };
allow fsadm_t bin_t:dir { getattr ioctl lock open read search };
allow fsadm_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow fsadm_t bin_t:lnk_file { getattr read };
allow fsdaemon_t bin_t:dir { getattr ioctl lock open read search };
allow fsdaemon_t bin_t:lnk_file { getattr read };
allow ftpd_t bin_t:dir { getattr ioctl lock open read search };
allow ftpd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ftpd_t bin_t:lnk_file { getattr read };
allow games_t bin_t:dir { getattr ioctl lock open read search };
allow games_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow games_t bin_t:lnk_file { getattr read };
allow gconfdefaultsm_t bin_t:lnk_file { getattr read };
allow gear_t bin_t:dir { getattr ioctl lock open read search };
allow gear_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow gear_t bin_t:lnk_file { getattr read };
allow geoclue_t bin_t:dir { getattr ioctl lock open read search };
allow geoclue_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow geoclue_t bin_t:lnk_file { getattr read };
allow getty_t bin_t:dir { getattr ioctl lock open read search };
allow getty_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow getty_t bin_t:lnk_file { getattr read };
allow gfs_controld_t bin_t:lnk_file { getattr read };
allow git_daemon bin_t:dir { getattr ioctl lock open read search };
allow git_daemon bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow git_daemon bin_t:lnk_file { getattr read };
allow gitosis_t bin_t:dir { getattr ioctl lock open read search };
allow gitosis_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow gitosis_t bin_t:lnk_file { getattr read };
allow gitosis_t bin_t:lnk_file { getattr read }; [ gitosis_can_sendmail ]:True
allow gkeyringd_domain bin_t:lnk_file { getattr read };
allow glance_domain bin_t:dir { getattr ioctl lock open read search };
allow glance_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow glance_domain bin_t:lnk_file { getattr read };
allow glusterd_t bin_t:dir { getattr ioctl lock open read search };
allow glusterd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow glusterd_t bin_t:lnk_file { getattr read };
allow gnomesystemmm_t bin_t:lnk_file { getattr read };
allow gpg_agent_t bin_t:dir { getattr ioctl lock open read search };
allow gpg_agent_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow gpg_agent_t bin_t:lnk_file { getattr read };
allow gpg_pinentry_t bin_t:dir { getattr ioctl lock open read search };
allow gpg_pinentry_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow gpg_pinentry_t bin_t:lnk_file { getattr read };
allow gpg_t bin_t:dir { getattr ioctl lock open read search };
allow gpg_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow gpg_t bin_t:lnk_file { getattr read };
allow greylist_milter_t bin_t:dir { getattr ioctl lock open read search };
allow greylist_milter_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow greylist_milter_t bin_t:lnk_file { getattr read };
allow groupadd_t bin_t:dir { getattr ioctl lock open read search };
allow groupadd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow groupadd_t bin_t:lnk_file { getattr read };
allow gssd_t bin_t:dir { getattr ioctl lock open read search };
allow gssd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow gssd_t bin_t:lnk_file { getattr read };
allow guest_t bin_t:dir { getattr ioctl lock open read search };
allow guest_t bin_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow guest_t bin_t:lnk_file { getattr read };
allow guest_usertype bin_t:dir { getattr ioctl lock open read search };
allow guest_usertype bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow guest_usertype bin_t:lnk_file { getattr read };
allow hsqldb_t bin_t:dir { getattr ioctl lock open read search };
allow hsqldb_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow hsqldb_t bin_t:lnk_file { getattr read };
allow httpd_passwd_t bin_t:dir { getattr ioctl lock open read search };
allow httpd_passwd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow httpd_passwd_t bin_t:lnk_file { getattr read };
allow httpd_script_type bin_t:dir { getattr ioctl lock open read search };
allow httpd_script_type bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow httpd_script_type bin_t:lnk_file { getattr read };
allow httpd_suexec_t bin_t:dir { getattr ioctl lock open read search };
allow httpd_suexec_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow httpd_suexec_t bin_t:lnk_file { getattr read };
allow httpd_suexec_t bin_t:lnk_file { getattr read }; [ httpd_can_sendmail ]:True
allow httpd_sys_script_t bin_t:lnk_file { getattr read }; [ httpd_can_sendmail ]:True
allow httpd_t bin_t:dir { getattr ioctl lock open read search };
allow httpd_t bin_t:dir { getattr ioctl lock open read search }; [ httpd_ssi_exec ]:True
allow httpd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow httpd_t bin_t:lnk_file { getattr read };
allow httpd_t bin_t:lnk_file { getattr read }; [ httpd_can_sendmail ]:True
allow httpd_t bin_t:lnk_file { getattr read }; [ httpd_mod_auth_pam ]:True
allow httpd_t bin_t:lnk_file { getattr read }; [ httpd_run_preupgrade ]:False
allow httpd_t bin_t:lnk_file { getattr read }; [ httpd_run_preupgrade ]:True
allow httpd_t bin_t:lnk_file { getattr read }; [ httpd_ssi_exec ]:True
allow hwclock_t bin_t:dir { getattr ioctl lock open read search };
allow hwclock_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow hwclock_t bin_t:lnk_file { getattr read };
allow hyperv_domain bin_t:dir { getattr ioctl lock open read search };
allow hyperv_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow hyperv_domain bin_t:lnk_file { getattr read };
allow hypervkvp_t bin_t:dir { getattr ioctl lock open read search };
allow hypervkvp_t bin_t:lnk_file { getattr read };
allow ifconfig_t bin_t:dir { getattr ioctl lock open read search };
allow ifconfig_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ifconfig_t bin_t:lnk_file { getattr read };
allow inetd_child_t bin_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow inetd_t bin_t:dir { getattr ioctl lock open read search };
allow inetd_t bin_t:file { execute getattr open read };
allow inetd_t bin_t:lnk_file { getattr read };
allow init_t bin_t:dir { getattr ioctl lock open read search };
allow init_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow init_t bin_t:lnk_file { getattr read };
allow initrc_t bin_t:dir { getattr ioctl lock open read search };
allow initrc_t bin_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow initrc_t bin_t:lnk_file { getattr read };
allow innd_t bin_t:dir { getattr ioctl lock open read search };
allow innd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow innd_t bin_t:lnk_file { getattr read };
allow insmod_t bin_t:dir { getattr ioctl lock open read search };
allow insmod_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow insmod_t bin_t:lnk_file { getattr read };
allow install_t bin_t:lnk_file { getattr read };
allow iodined_t bin_t:dir { getattr ioctl lock open read search };
allow iodined_t bin_t:lnk_file { getattr read };
allow iotop_t bin_t:dir { getattr ioctl lock open read search };
allow iotop_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow iotop_t bin_t:lnk_file { getattr read };
allow ipa_dnskey_t bin_t:dir { getattr ioctl lock open read search };
allow ipa_dnskey_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ipa_dnskey_t bin_t:lnk_file { getattr read };
allow ipa_helper_t bin_t:dir { getattr ioctl lock open read search };
allow ipa_helper_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ipa_helper_t bin_t:lnk_file { getattr read };
allow ipa_ods_exporter_t bin_t:dir { getattr ioctl lock open read search };
allow ipa_ods_exporter_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ipa_ods_exporter_t bin_t:lnk_file { getattr read };
allow ipmievd_t bin_t:dir { getattr ioctl lock open read search };
allow ipmievd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ipmievd_t bin_t:lnk_file { getattr read };
allow ipsec_mgmt_t bin_t:dir { getattr ioctl lock open read search };
allow ipsec_mgmt_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ipsec_mgmt_t bin_t:lnk_file { getattr read };
allow ipsec_t bin_t:dir { getattr ioctl lock open read search };
allow ipsec_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ipsec_t bin_t:lnk_file { getattr read };
allow iptables_t bin_t:dir { getattr ioctl lock open read search };
allow iptables_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow iptables_t bin_t:lnk_file { getattr read };
allow irc_t bin_t:dir { getattr ioctl lock open read search };
allow irc_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow irc_t bin_t:lnk_file { getattr read };
allow irssi_t bin_t:dir { getattr ioctl lock open read search };
allow irssi_t bin_t:lnk_file { getattr read };
allow iscsid_t bin_t:dir { getattr ioctl lock open read search };
allow iscsid_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow iscsid_t bin_t:lnk_file { getattr read };
allow jetty_t bin_t:dir { getattr ioctl lock open read search };
allow jetty_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow jetty_t bin_t:lnk_file { getattr read };
allow jockey_t bin_t:dir { getattr ioctl lock open read search };
allow jockey_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow jockey_t bin_t:lnk_file { getattr read };
allow journalctl_t bin_t:dir { getattr ioctl lock open read search };
allow journalctl_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow journalctl_t bin_t:lnk_file { getattr read };
allow kadmind_t bin_t:dir { getattr ioctl lock open read search };
allow kadmind_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow kadmind_t bin_t:lnk_file { getattr read };
allow kdumpctl_t bin_t:dir { getattr ioctl lock open read search };
allow kdumpctl_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow kdumpctl_t bin_t:lnk_file { getattr read };
allow kdumpgui_t bin_t:dir { getattr ioctl lock open read search };
allow kdumpgui_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow kdumpgui_t bin_t:lnk_file { getattr read };
allow kdumpgui_t bin_t:lnk_file { getattr read }; [ kdumpgui_run_bootloader ]:True
allow keepalived_t bin_t:dir { getattr ioctl lock open read search };
allow keepalived_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow keepalived_t bin_t:lnk_file { getattr read };
allow kernel_t bin_t:dir { getattr ioctl lock open read search };
allow kernel_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow kernel_t bin_t:lnk_file { getattr read };
allow keystone_t bin_t:dir { getattr ioctl lock open read search };
allow keystone_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow keystone_t bin_t:lnk_file { getattr read };
allow kismet_t bin_t:dir { getattr ioctl lock open read search };
allow kismet_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow kismet_t bin_t:lnk_file { getattr read };
allow kmscon_t bin_t:lnk_file { getattr read };
allow kpropd_t bin_t:dir { getattr ioctl lock open read search };
allow kpropd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow kpropd_t bin_t:lnk_file { getattr read };
allow krb5kdc_t bin_t:dir { getattr ioctl lock open read search };
allow krb5kdc_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow krb5kdc_t bin_t:lnk_file { getattr read };
allow ksmtuned_t bin_t:dir { getattr ioctl lock open read search };
allow ksmtuned_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ksmtuned_t bin_t:lnk_file { getattr read };
allow l2tpd_t bin_t:dir { getattr ioctl lock open read search };
allow l2tpd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow l2tpd_t bin_t:lnk_file { getattr read };
allow ldconfig_t bin_t:lnk_file { getattr read };
allow livecd_t bin_t:lnk_file { getattr read };
allow loadkeys_t bin_t:dir { getattr ioctl lock open read search };
allow loadkeys_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow loadkeys_t bin_t:lnk_file { getattr read };
allow local_login_t bin_t:dir { getattr ioctl lock open read search };
allow local_login_t bin_t:file { getattr ioctl lock open read };
allow local_login_t bin_t:lnk_file { getattr read };
allow locate_t bin_t:dir { getattr ioctl lock open read search };
allow locate_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow locate_t bin_t:lnk_file { getattr read };
allow logadm_t bin_t:lnk_file { getattr read };
allow login_pgm bin_t:dir { getattr ioctl lock open read search };
allow login_pgm bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow login_pgm bin_t:lnk_file { getattr read };
allow logrotate_t bin_t:dir { getattr ioctl lock open read search };
allow logrotate_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow logrotate_t bin_t:lnk_file { getattr read };
allow logwatch_mail_t bin_t:lnk_file { getattr read };
allow logwatch_t bin_t:dir { getattr ioctl lock open read search };
allow logwatch_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow logwatch_t bin_t:lnk_file { getattr read };
allow lpd_t bin_t:dir { getattr ioctl lock open read search };
allow lpd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow lpd_t bin_t:lnk_file { getattr read };
allow lsassd_t bin_t:dir { getattr ioctl lock open read search };
allow lsassd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow lsassd_t bin_t:lnk_file { getattr read };
allow lsmd_plugin_t bin_t:dir { getattr ioctl lock open read search };
allow lsmd_plugin_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow lsmd_plugin_t bin_t:lnk_file { getattr read };
allow lsmd_t bin_t:dir { getattr ioctl lock open read search };
allow lsmd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow lsmd_t bin_t:lnk_file { getattr read };
allow lttng_sessiond_t bin_t:dir { getattr ioctl lock open read search };
allow lttng_sessiond_t bin_t:lnk_file { getattr read };
allow lvm_t bin_t:dir { getattr ioctl lock open read search };
allow lvm_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow lvm_t bin_t:lnk_file { getattr read };
allow mail_munin_plugin_t bin_t:lnk_file { getattr read };
allow mailman_domain bin_t:dir { getattr ioctl lock open read search };
allow mailman_queue_t bin_t:lnk_file { getattr read };
allow mailserver_delivery bin_t:lnk_file { getattr read };
allow mandb_t bin_t:dir { getattr ioctl lock open read search };
allow mandb_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow mandb_t bin_t:lnk_file { getattr read };
allow mcelog_t bin_t:dir { getattr ioctl lock open read search };
allow mcelog_t bin_t:dir { getattr ioctl lock open read search }; [ mcelog_exec_scripts ]:True
allow mcelog_t bin_t:dir { getattr ioctl lock open read search }; [ mcelog_exec_scripts ]:True
allow mcelog_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow mcelog_t bin_t:file { execute execute_no_trans getattr ioctl lock open read }; [ mcelog_exec_scripts ]:True
allow mcelog_t bin_t:lnk_file { getattr read };
allow mcelog_t bin_t:lnk_file { getattr read }; [ mcelog_exec_scripts ]:True
allow mcelog_t bin_t:lnk_file { getattr read }; [ mcelog_exec_scripts ]:True
allow mdadm_t bin_t:dir { getattr ioctl lock open read search };
allow mdadm_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow mdadm_t bin_t:lnk_file { getattr read };
allow minidlna_t bin_t:dir { getattr ioctl lock open read search };
allow minidlna_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow minidlna_t bin_t:lnk_file { getattr read };
allow mock_build_t bin_t:dir { getattr ioctl lock open read search };
allow mock_build_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow mock_build_t bin_t:lnk_file { getattr read };
allow mock_t bin_t:dir { getattr ioctl lock open read search };
allow mock_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow mock_t bin_t:lnk_file { getattr read };
allow modemmanager_t bin_t:dir { getattr ioctl lock open read search };
allow modemmanager_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow modemmanager_t bin_t:lnk_file { getattr read };
allow mojomojo_script_t bin_t:lnk_file { getattr read };
allow mongod_t bin_t:dir { getattr ioctl lock open read search };
allow mongod_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow mongod_t bin_t:lnk_file { getattr read };
allow motion_t bin_t:lnk_file { getattr read };
allow mount_t bin_t:dir { getattr ioctl lock open read search };
allow mount_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow mount_t bin_t:lnk_file { getattr read };
allow mozilla_plugin_config_t bin_t:dir { getattr ioctl lock open read search };
allow mozilla_plugin_config_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow mozilla_plugin_config_t bin_t:lnk_file { getattr read };
allow mozilla_plugin_t bin_t:dir { getattr ioctl lock open read search };
allow mozilla_plugin_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow mozilla_plugin_t bin_t:lnk_file { getattr read };
allow mozilla_t bin_t:dir { getattr ioctl lock open read search };
allow mozilla_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow mozilla_t bin_t:lnk_file { getattr read };
allow mpd_t bin_t:dir { getattr ioctl lock open read search };
allow mpd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow mpd_t bin_t:lnk_file { getattr read };
allow mplayer_t bin_t:dir { getattr ioctl lock open read search };
allow mplayer_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow mplayer_t bin_t:lnk_file { getattr read };
allow mrtg_t bin_t:dir { getattr ioctl lock open read search };
allow mrtg_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow mrtg_t bin_t:lnk_file { getattr read };
allow mscan_t bin_t:dir { getattr ioctl lock open read search };
allow mscan_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow mscan_t bin_t:lnk_file { getattr read };
allow munin_plugin_domain bin_t:dir { getattr ioctl lock open read search };
allow munin_plugin_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow munin_plugin_domain bin_t:lnk_file { getattr read };
allow munin_t bin_t:dir { getattr ioctl lock open read search };
allow munin_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow munin_t bin_t:lnk_file { getattr read };
allow mysqld_safe_t bin_t:dir { getattr ioctl lock open read search };
allow mysqld_safe_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow mysqld_safe_t bin_t:lnk_file { getattr read };
allow mysqld_t bin_t:dir { getattr ioctl lock open read search };
allow mysqld_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow mysqld_t bin_t:lnk_file { getattr read };
allow mysqlmanagerd_t bin_t:dir { getattr ioctl lock open read search };
allow mysqlmanagerd_t bin_t:lnk_file { getattr read };
allow nagios_admin_plugin_t bin_t:file { getattr ioctl lock open read };
allow nagios_admin_plugin_t bin_t:lnk_file { getattr read };
allow nagios_checkdisk_plugin_t bin_t:dir { getattr ioctl lock open read search };
allow nagios_checkdisk_plugin_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow nagios_checkdisk_plugin_t bin_t:lnk_file { getattr read };
allow nagios_eventhandler_plugin_t bin_t:dir { getattr ioctl lock open read search };
allow nagios_eventhandler_plugin_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow nagios_eventhandler_plugin_t bin_t:lnk_file { getattr read };
allow nagios_mail_plugin_t bin_t:file { getattr ioctl lock open read };
allow nagios_mail_plugin_t bin_t:lnk_file { getattr read };
allow nagios_openshift_plugin_t bin_t:dir { getattr ioctl lock open read search };
allow nagios_openshift_plugin_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow nagios_openshift_plugin_t bin_t:lnk_file { getattr read };
allow nagios_services_plugin_t bin_t:dir { getattr ioctl lock open read search };
allow nagios_services_plugin_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow nagios_services_plugin_t bin_t:lnk_file { getattr read };
allow nagios_system_plugin_t bin_t:dir { getattr ioctl lock open read search };
allow nagios_system_plugin_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow nagios_system_plugin_t bin_t:lnk_file { getattr read };
allow nagios_t bin_t:dir { getattr ioctl lock open read search };
allow nagios_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow nagios_t bin_t:lnk_file { getattr read };
allow nagios_t bin_t:lnk_file { getattr read }; [ nagios_run_sudo ]:True
allow named_filetrans_domain bin_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow named_t bin_t:lnk_file { getattr read };
allow namespace_init_t bin_t:dir { getattr ioctl lock open read search };
allow namespace_init_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow namespace_init_t bin_t:lnk_file { getattr read };
allow ncftool_t bin_t:dir { getattr ioctl lock open read search };
allow ncftool_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ncftool_t bin_t:lnk_file { getattr read };
allow netlabel_mgmt_t bin_t:dir { getattr ioctl lock open read search };
allow netlabel_mgmt_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow netlabel_mgmt_t bin_t:lnk_file { getattr read };
allow neutron_t bin_t:dir { getattr ioctl lock open read search };
allow neutron_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow neutron_t bin_t:lnk_file { getattr read };
allow newrole_t bin_t:dir { getattr ioctl lock open read search };
allow newrole_t bin_t:dir { getattr ioctl lock open read search }; [ polyinstantiation_enabled ]:True
allow newrole_t bin_t:dir { getattr ioctl lock open read search }; [ secure_mode ]:False
allow newrole_t bin_t:dir { getattr ioctl lock open read search }; [ secure_mode ]:True
allow newrole_t bin_t:file { execute execute_no_trans getattr ioctl lock open read }; [ polyinstantiation_enabled ]:True
allow newrole_t bin_t:lnk_file { getattr read };
allow newrole_t bin_t:lnk_file { getattr read }; [ polyinstantiation_enabled ]:True
allow newrole_t bin_t:lnk_file { getattr read }; [ polyinstantiation_enabled ]:True
allow newrole_t bin_t:lnk_file { getattr read }; [ secure_mode ]:False
allow newrole_t bin_t:lnk_file { getattr read }; [ secure_mode ]:True
allow nfsd_t bin_t:dir { getattr ioctl lock open read search };
allow nfsd_t bin_t:lnk_file { getattr read };
allow nova_domain bin_t:dir { getattr ioctl lock open read search };
allow nova_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow nova_domain bin_t:lnk_file { getattr read };
allow nrpe_t bin_t:dir { getattr ioctl lock open read search };
allow nrpe_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow nrpe_t bin_t:lnk_file { getattr read };
allow nrpe_t bin_t:lnk_file { getattr read }; [ nagios_run_sudo ]:True
allow nscd_t bin_t:lnk_file { getattr read };
allow nsd_crond_t bin_t:dir { getattr ioctl lock open read search };
allow nsd_crond_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow nsd_crond_t bin_t:lnk_file { getattr read };
allow nsd_t bin_t:dir { getattr ioctl lock open read search };
allow nsd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow nsd_t bin_t:lnk_file { getattr read };
allow nslcd_t bin_t:dir { getattr ioctl lock open read search };
allow nslcd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow nslcd_t bin_t:lnk_file { getattr read };
allow ntpd_t bin_t:dir { getattr ioctl lock open read search };
allow ntpd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ntpd_t bin_t:lnk_file { getattr read };
allow nut_upsdrvctl_t bin_t:dir { getattr ioctl lock open read search };
allow nut_upsdrvctl_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow nut_upsdrvctl_t bin_t:lnk_file { getattr read };
allow nut_upsmon_t bin_t:dir { getattr ioctl lock open read search };
allow nut_upsmon_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow nut_upsmon_t bin_t:lnk_file { getattr read };
allow nx_server_ssh_t bin_t:dir { getattr ioctl lock open read search };
allow nx_server_ssh_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow nx_server_ssh_t bin_t:lnk_file { getattr read };
allow nx_server_t bin_t:dir { getattr ioctl lock open read search };
allow nx_server_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow nx_server_t bin_t:lnk_file { getattr read };
allow oddjob_t bin_t:dir { getattr ioctl lock open read search };
allow oddjob_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow oddjob_t bin_t:lnk_file { getattr read };
allow opendnssec_t bin_t:dir { getattr ioctl lock open read search };
allow opendnssec_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow opendnssec_t bin_t:lnk_file { getattr read };
allow openfortivpn_t bin_t:lnk_file { getattr read };
allow openshift_cgroup_read_t bin_t:dir { getattr ioctl lock open read search };
allow openshift_cgroup_read_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow openshift_cgroup_read_t bin_t:lnk_file { getattr read };
allow openshift_cron_t bin_t:dir { getattr ioctl lock open read search };
allow openshift_cron_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow openshift_cron_t bin_t:lnk_file { getattr read };
allow openshift_domain bin_t:dir { getattr ioctl lock open read search };
allow openshift_domain bin_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow openshift_net_read_t bin_t:dir { getattr ioctl lock open read search };
allow openshift_net_read_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow openshift_net_read_t bin_t:lnk_file { getattr read };
allow openshift_t bin_t:lnk_file { getattr read };
allow opensm_t bin_t:dir { getattr ioctl lock open read search };
allow opensm_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow opensm_t bin_t:lnk_file { getattr read };
allow openvpn_t bin_t:dir { getattr ioctl lock open read search };
allow openvpn_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow openvpn_t bin_t:lnk_file { getattr read };
allow openvswitch_t bin_t:dir { getattr ioctl lock open read search };
allow openvswitch_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow openvswitch_t bin_t:lnk_file { getattr read };
allow openwsman_t bin_t:lnk_file { getattr read };
allow oracleasm_t bin_t:dir { getattr ioctl lock open read search };
allow oracleasm_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow oracleasm_t bin_t:lnk_file { getattr read };
allow osad_t bin_t:dir { getattr ioctl lock open read search };
allow osad_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow osad_t bin_t:lnk_file { getattr read };
allow pads_t bin_t:lnk_file { getattr read };
allow passenger_t bin_t:dir { getattr ioctl lock open read search };
allow passenger_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow passenger_t bin_t:lnk_file { getattr read };
allow passwd_t bin_t:dir { getattr ioctl lock open read search };
allow passwd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow passwd_t bin_t:lnk_file { getattr read };
allow pcp_pmcd_t bin_t:dir { getattr ioctl lock open read search };
allow pcp_pmcd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow pcp_pmcd_t bin_t:lnk_file { getattr read };
allow pcp_pmie_t bin_t:dir { getattr ioctl lock open read search };
allow pcp_pmie_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow pcp_pmie_t bin_t:lnk_file { getattr read };
allow pcp_pmlogger_t bin_t:dir { getattr ioctl lock open read search };
allow pcp_pmlogger_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow pcp_pmlogger_t bin_t:lnk_file { getattr read };
allow pcp_pmmgr_t bin_t:dir { getattr ioctl lock open read search };
allow pcp_pmmgr_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow pcp_pmmgr_t bin_t:lnk_file { getattr read };
allow pcp_pmwebd_t bin_t:dir { getattr ioctl lock open read search };
allow pcp_pmwebd_t bin_t:lnk_file { getattr read };
allow pegasus_openlmi_account_t bin_t:lnk_file { getattr read };
allow pegasus_openlmi_admin_t bin_t:lnk_file { getattr read };
allow pegasus_openlmi_domain bin_t:dir { getattr ioctl lock open read search };
allow pegasus_openlmi_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow pegasus_openlmi_domain bin_t:lnk_file { getattr read };
allow pegasus_openlmi_storage_t bin_t:lnk_file { getattr read };
allow pegasus_openlmi_system_t bin_t:lnk_file { getattr read };
allow pegasus_t bin_t:dir { getattr ioctl lock open read search };
allow pegasus_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow pegasus_t bin_t:lnk_file { getattr read };
allow piranha_domain bin_t:dir { getattr ioctl lock open read search };
allow piranha_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow piranha_domain bin_t:lnk_file { getattr read };
allow piranha_lvs_t bin_t:lnk_file { getattr read };
allow piranha_pulse_t bin_t:dir { getattr ioctl lock open read search };
allow piranha_pulse_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow piranha_pulse_t bin_t:lnk_file { getattr read };
allow pki_apache_domain bin_t:dir { getattr ioctl lock open read search };
allow pki_apache_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow pki_apache_domain bin_t:lnk_file { getattr read };
allow pki_ra_t bin_t:lnk_file { getattr read };
allow pki_tomcat_t bin_t:lnk_file { getattr read };
allow plymouth_t bin_t:lnk_file { getattr read };
allow podsleuth_t bin_t:dir { getattr ioctl lock open read search };
allow podsleuth_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow podsleuth_t bin_t:lnk_file { getattr read };
allow policy_manager_domain bin_t:dir { getattr ioctl lock open read search };
allow policy_manager_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow policy_manager_domain bin_t:lnk_file { getattr read };
allow policykit_auth_t bin_t:dir { getattr ioctl lock open read search };
allow policykit_auth_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow policykit_auth_t bin_t:lnk_file { getattr read };
allow policykit_grant_t bin_t:lnk_file { getattr read };
allow policykit_resolve_t bin_t:lnk_file { getattr read };
allow policykit_t bin_t:dir { getattr ioctl lock open read search };
allow policykit_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow policykit_t bin_t:lnk_file { getattr read };
allow polydomain bin_t:dir { getattr ioctl lock open read search }; [ polyinstantiation_enabled ]:True
allow polydomain bin_t:file { execute execute_no_trans getattr ioctl lock open read }; [ polyinstantiation_enabled ]:True
allow polydomain bin_t:lnk_file { getattr read }; [ polyinstantiation_enabled ]:True
allow polydomain bin_t:lnk_file { getattr read }; [ polyinstantiation_enabled ]:True
allow postfix_cleanup_t bin_t:dir { getattr ioctl lock open read search };
allow postfix_cleanup_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow postfix_cleanup_t bin_t:lnk_file { getattr read };
allow postfix_domain bin_t:dir { getattr ioctl lock open read search };
allow postfix_domain bin_t:lnk_file { getattr read };
allow postfix_local_t bin_t:dir { getattr ioctl lock open read search };
allow postfix_local_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow postfix_local_t bin_t:lnk_file { getattr read };
allow postfix_map_t bin_t:dir { getattr ioctl lock open read search };
allow postfix_map_t bin_t:file { getattr ioctl lock open read };
allow postfix_map_t bin_t:lnk_file { getattr read };
allow postfix_master_t bin_t:dir { getattr ioctl lock open read search };
allow postfix_master_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow postfix_master_t bin_t:lnk_file { getattr read };
allow postfix_pipe_t bin_t:dir { getattr ioctl lock open read search };
allow postfix_pipe_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow postfix_pipe_t bin_t:lnk_file { getattr read };
allow postfix_qmgr_t bin_t:dir { getattr ioctl lock open read search };
allow postfix_qmgr_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow postfix_qmgr_t bin_t:lnk_file { getattr read };
allow postfix_smtpd_t bin_t:dir { getattr ioctl lock open read search };
allow postfix_smtpd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow postfix_smtpd_t bin_t:lnk_file { getattr read };
allow postfix_virtual_t bin_t:dir { getattr ioctl lock open read search };
allow postfix_virtual_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow postfix_virtual_t bin_t:lnk_file { getattr read };
allow postgresql_t bin_t:dir { getattr ioctl lock open read search };
allow postgresql_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow postgresql_t bin_t:lnk_file { getattr read };
allow postgresql_t bin_t:lnk_file { getattr read }; [ postgresql_can_rsync ]:True
allow postgrey_t bin_t:lnk_file { getattr read };
allow pppd_t bin_t:dir { getattr ioctl lock open read search };
allow pppd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow pppd_t bin_t:lnk_file { getattr read };
allow pppd_t bin_t:lnk_file { getattr read }; [ pppd_can_insmod ]:True
allow pptp_t bin_t:dir { getattr ioctl lock open read search };
allow pptp_t bin_t:lnk_file { getattr read };
allow prelink_cron_system_t bin_t:dir { getattr ioctl lock open read search };
allow prelink_cron_system_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow prelink_cron_system_t bin_t:lnk_file { getattr read };
allow prelink_t bin_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow prelink_t bin_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow prelude_audisp_t bin_t:lnk_file { getattr read };
allow prelude_correlator_t bin_t:lnk_file { getattr read };
allow prelude_lml_t bin_t:dir { getattr ioctl lock open read search };
allow prelude_lml_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow prelude_lml_t bin_t:lnk_file { getattr read };
allow prelude_t bin_t:lnk_file { getattr read };
allow procmail_t bin_t:dir { getattr ioctl lock open read search };
allow procmail_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow procmail_t bin_t:lnk_file { getattr read };
allow prosody_t bin_t:dir { getattr ioctl lock open read search };
allow prosody_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow prosody_t bin_t:lnk_file { getattr read };
allow psad_t bin_t:dir { getattr ioctl lock open read search };
allow psad_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow psad_t bin_t:lnk_file { getattr read };
allow pulseaudio_t bin_t:dir { getattr ioctl lock open read search };
allow pulseaudio_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow pulseaudio_t bin_t:lnk_file { getattr read };
allow puppetagent_t bin_t:dir { getattr ioctl lock open read search };
allow puppetagent_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow puppetagent_t bin_t:lnk_file { getattr read };
allow puppetca_t bin_t:dir { getattr ioctl lock open read search };
allow puppetca_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow puppetca_t bin_t:lnk_file { getattr read };
allow puppetmaster_t bin_t:dir { getattr ioctl lock open read search };
allow puppetmaster_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow puppetmaster_t bin_t:lnk_file { getattr read };
allow pwauth_t bin_t:lnk_file { getattr read };
allow pyicqt_t bin_t:dir { getattr ioctl lock open read search };
allow pyicqt_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow pyicqt_t bin_t:lnk_file { getattr read };
allow qdiskd_t bin_t:dir { getattr ioctl lock open read search };
allow qdiskd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow qdiskd_t bin_t:lnk_file { getattr read };
allow qmail_clean_t bin_t:lnk_file { getattr read };
allow qmail_inject_t bin_t:lnk_file { getattr read };
allow qmail_local_t bin_t:dir { getattr ioctl lock open read search };
allow qmail_local_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow qmail_local_t bin_t:lnk_file { getattr read };
allow qmail_lspawn_t bin_t:lnk_file { getattr read };
allow qmail_queue_t bin_t:dir { getattr ioctl lock open read search };
allow qmail_queue_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow qmail_queue_t bin_t:lnk_file { getattr read };
allow qmail_remote_t bin_t:lnk_file { getattr read };
allow qmail_rspawn_t bin_t:lnk_file { getattr read };
allow qmail_send_t bin_t:lnk_file { getattr read };
allow qmail_smtpd_t bin_t:lnk_file { getattr read };
allow qmail_splogger_t bin_t:lnk_file { getattr read };
allow qmail_start_t bin_t:lnk_file { getattr read };
allow qmail_tcp_env_t bin_t:lnk_file { getattr read };
allow rabbitmq_t bin_t:dir { getattr ioctl lock open read search };
allow rabbitmq_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow rabbitmq_t bin_t:lnk_file { getattr read };
allow racoon_t bin_t:dir { getattr ioctl lock open read search };
allow racoon_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow racoon_t bin_t:lnk_file { getattr read };
allow radiusd_t bin_t:dir { getattr ioctl lock open read search };
allow radiusd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow radiusd_t bin_t:lnk_file { getattr read };
allow rasdaemon_t bin_t:lnk_file { getattr read };
allow realmd_consolehelper_t bin_t:lnk_file { getattr read };
allow realmd_t bin_t:dir { getattr ioctl lock open read search };
allow realmd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow realmd_t bin_t:lnk_file { getattr read };
allow remote_login_t bin_t:dir { getattr ioctl lock open read search };
allow remote_login_t bin_t:file { getattr ioctl lock open read };
allow remote_login_t bin_t:lnk_file { getattr read };
allow rhev_agentd_consolehelper_t bin_t:lnk_file { getattr read };
allow rhev_agentd_t bin_t:dir { getattr ioctl lock open read search };
allow rhev_agentd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow rhev_agentd_t bin_t:lnk_file { getattr read };
allow rhgb_t bin_t:dir { getattr ioctl lock open read search };
allow rhgb_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow rhgb_t bin_t:lnk_file { getattr read };
allow rhnsd_t bin_t:dir { getattr ioctl lock open read search };
allow rhnsd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow rhnsd_t bin_t:lnk_file { getattr read };
allow rhsmcertd_t bin_t:dir { getattr ioctl lock open read search };
allow rhsmcertd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow rhsmcertd_t bin_t:lnk_file { getattr read };
allow ricci_modcluster_t bin_t:dir { getattr ioctl lock open read search };
allow ricci_modcluster_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ricci_modcluster_t bin_t:lnk_file { getattr read };
allow ricci_modclusterd_t bin_t:dir { getattr ioctl lock open read search };
allow ricci_modclusterd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ricci_modclusterd_t bin_t:lnk_file { getattr read };
allow ricci_modlog_t bin_t:dir { getattr ioctl lock open read search };
allow ricci_modlog_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ricci_modlog_t bin_t:lnk_file { getattr read };
allow ricci_modrpm_t bin_t:dir { getattr ioctl lock open read search };
allow ricci_modrpm_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ricci_modrpm_t bin_t:lnk_file { getattr read };
allow ricci_modservice_t bin_t:dir { getattr ioctl lock open read search };
allow ricci_modservice_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ricci_modservice_t bin_t:lnk_file { getattr read };
allow ricci_modstorage_t bin_t:dir { getattr ioctl lock open read search };
allow ricci_modstorage_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ricci_modstorage_t bin_t:lnk_file { getattr read };
allow ricci_t bin_t:dir { getattr ioctl lock open read search };
allow ricci_t bin_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow ricci_t bin_t:lnk_file { getattr read };
allow rlogind_t bin_t:lnk_file { getattr read };
allow roundup_t bin_t:dir { getattr ioctl lock open read search };
allow roundup_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow roundup_t bin_t:lnk_file { getattr read };
allow rpcbind_t bin_t:dir { getattr ioctl lock open read search };
allow rpcbind_t bin_t:lnk_file { getattr read };
allow rpcd_t bin_t:dir { getattr ioctl lock open read search };
allow rpcd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow rpcd_t bin_t:lnk_file { getattr read };
allow rpm_script_t bin_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow rpm_script_t bin_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow rpm_script_t bin_t:lnk_file { getattr read };
allow rpm_t bin_t:dir { getattr ioctl lock open read search };
allow rpm_t bin_t:lnk_file { getattr read };
allow rshd_t bin_t:lnk_file { getattr read };
allow rssh_t bin_t:lnk_file { getattr read };
allow rsync_t bin_t:lnk_file { getattr read };
allow rsync_t bin_t:lnk_file { getattr read }; [ rsync_client ]:True
allow rtas_errd_t bin_t:dir { getattr ioctl lock open read search };
allow rtas_errd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow rtas_errd_t bin_t:lnk_file { getattr read };
allow run_init_t bin_t:dir { getattr ioctl lock open read search };
allow run_init_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow run_init_t bin_t:lnk_file { getattr read };
allow sambagui_t bin_t:dir { getattr ioctl lock open read search };
allow sambagui_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow sambagui_t bin_t:lnk_file { getattr read };
allow sandbox_domain bin_t:dir { getattr ioctl lock open read search };
allow sandbox_x_domain bin_t:dir { getattr ioctl lock open read search };
allow sandbox_xserver_t bin_t:dir { getattr ioctl lock open read search };
allow sandbox_xserver_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow sandbox_xserver_t bin_t:lnk_file { getattr read };
allow sanlock_t bin_t:lnk_file { getattr read };
allow saslauthd_t bin_t:dir { getattr ioctl lock open read search };
allow saslauthd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow saslauthd_t bin_t:lnk_file { getattr read };
allow sblim_gatherd_t bin_t:dir { getattr ioctl lock open read search };
allow sblim_gatherd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow sblim_gatherd_t bin_t:lnk_file { getattr read };
allow sblim_sfcbd_t bin_t:dir { getattr ioctl lock open read search };
allow sblim_sfcbd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow sblim_sfcbd_t bin_t:lnk_file { getattr read };
allow screen_domain bin_t:dir { getattr ioctl lock open read search };
allow screen_domain bin_t:file { getattr ioctl lock open read };
allow screen_domain bin_t:lnk_file { getattr read };
allow secadm_screen_t bin_t:dir { getattr ioctl lock open read search };
allow secadm_screen_t bin_t:file { execute getattr open read };
allow secadm_screen_t bin_t:lnk_file { getattr read };
allow secadm_su_t bin_t:dir { getattr ioctl lock open read search };
allow secadm_su_t bin_t:lnk_file { getattr read };
allow secadm_sudo_t bin_t:dir { getattr ioctl lock open read search };
allow secadm_sudo_t bin_t:file { execute getattr open read };
allow secadm_sudo_t bin_t:lnk_file { getattr read };
allow secadm_t bin_t:dir { getattr ioctl lock open read search };
allow secadm_t bin_t:lnk_file { getattr read };
allow sectoolm_t bin_t:dir { getattr ioctl lock open read search };
allow sectoolm_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow sectoolm_t bin_t:lnk_file { getattr read };
allow semanage_t bin_t:lnk_file { getattr read };
allow sendmail_t bin_t:dir { getattr ioctl lock open read search };
allow sendmail_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow sendmail_t bin_t:lnk_file { getattr read };
allow services_munin_plugin_t bin_t:lnk_file { getattr read };
allow session_bus_type bin_t:dir { getattr ioctl lock open read search };
allow session_bus_type bin_t:file { getattr ioctl lock open read };
allow session_bus_type bin_t:lnk_file { getattr read };
allow setrans_t bin_t:lnk_file { getattr read };
allow setroubleshoot_fixit_t bin_t:dir { getattr ioctl lock open read search };
allow setroubleshoot_fixit_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow setroubleshoot_fixit_t bin_t:lnk_file { getattr read };
allow setroubleshootd_t bin_t:dir { getattr ioctl lock open read search };
allow setroubleshootd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow setroubleshootd_t bin_t:lnk_file { getattr read };
allow setsebool_t bin_t:lnk_file { getattr read };
allow seunshare_domain bin_t:dir { getattr ioctl lock open read search };
allow seunshare_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow seunshare_domain bin_t:lnk_file { getattr read };
allow sge_domain bin_t:dir { getattr ioctl lock open read search };
allow sge_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow sge_domain bin_t:lnk_file { getattr read };
allow sge_execd_t bin_t:lnk_file { getattr read };
allow sge_job_ssh_t bin_t:dir { getattr ioctl lock open read search };
allow sge_job_ssh_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow sge_job_ssh_t bin_t:lnk_file { getattr read };
allow sge_shepherd_t bin_t:dir { getattr ioctl lock open read search };
allow sge_shepherd_t bin_t:lnk_file { getattr read };
allow shorewall_t bin_t:dir { getattr ioctl lock open read search };
allow shorewall_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow shorewall_t bin_t:lnk_file { getattr read };
allow smbd_t bin_t:dir { getattr ioctl lock open read search };
allow smbd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow smbd_t bin_t:lnk_file { getattr read };
allow smbd_t bin_t:lnk_file { getattr read }; [ samba_domain_controller ]:True
allow smbd_t bin_t:lnk_file { getattr read }; [ samba_domain_controller ]:True
allow smbd_t bin_t:lnk_file { getattr read }; [ samba_domain_controller ]:True
allow smbmount_t bin_t:dir { getattr ioctl lock open read search };
allow smbmount_t bin_t:lnk_file { getattr read };
allow smokeping_cgi_script_t bin_t:lnk_file { getattr read };
allow smokeping_t bin_t:dir { getattr ioctl lock open read search };
allow smokeping_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow smokeping_t bin_t:lnk_file { getattr read };
allow smoltclient_t bin_t:dir { getattr ioctl lock open read search };
allow smoltclient_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow smoltclient_t bin_t:lnk_file { getattr read };
allow smsd_t bin_t:dir { getattr ioctl lock open read search };
allow smsd_t bin_t:lnk_file { getattr read };
allow snapperd_t bin_t:dir { getattr ioctl lock open read search };
allow snapperd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow snapperd_t bin_t:lnk_file { getattr read };
allow snmpd_t bin_t:dir { getattr ioctl lock open read search };
allow snmpd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow snmpd_t bin_t:lnk_file { getattr read };
allow sosreport_t bin_t:dir { getattr ioctl lock open read search };
allow sosreport_t bin_t:lnk_file { getattr read };
allow soundd_t bin_t:lnk_file { getattr read };
allow spamass_milter_t bin_t:dir { getattr ioctl lock open read search };
allow spamass_milter_t bin_t:lnk_file { getattr read };
allow spamc_t bin_t:dir { getattr ioctl lock open read search };
allow spamc_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow spamc_t bin_t:lnk_file { getattr read };
allow spamd_t bin_t:dir { getattr ioctl lock open read search };
allow spamd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow spamd_t bin_t:lnk_file { getattr read };
allow spamd_update_t bin_t:dir { getattr ioctl lock open read search };
allow spamd_update_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow spamd_update_t bin_t:lnk_file { getattr read };
allow squid_cron_t bin_t:dir { getattr ioctl lock open read search };
allow squid_cron_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow squid_cron_t bin_t:lnk_file { getattr read };
allow squid_t bin_t:dir { getattr ioctl lock open read search };
allow squid_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow squid_t bin_t:lnk_file { getattr read };
allow ssh_keygen_t bin_t:dir { getattr ioctl lock open read search };
allow ssh_keygen_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ssh_keygen_t bin_t:lnk_file { getattr read };
allow ssh_t bin_t:dir { getattr ioctl lock open read search };
allow ssh_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ssh_t bin_t:lnk_file { getattr read };
allow sshd_keygen_t bin_t:dir { getattr ioctl lock open read search };
allow sshd_keygen_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow sshd_keygen_t bin_t:lnk_file { getattr read };
allow sshd_t bin_t:dir { getattr ioctl lock open read search };
allow sshd_t bin_t:dir { getattr ioctl lock open read search }; [ ssh_sysadm_login ]:True
allow sshd_t bin_t:lnk_file { getattr read };
allow sshd_t bin_t:lnk_file { getattr read }; [ ssh_sysadm_login ]:True
allow sssd_selinux_manager_t bin_t:lnk_file { getattr read };
allow sssd_t bin_t:dir { getattr ioctl lock open read search };
allow sssd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow sssd_t bin_t:lnk_file { getattr read };
allow staff_consolehelper_t bin_t:lnk_file { getattr read };
allow staff_dbusd_t bin_t:dir { getattr ioctl lock open read search };
allow staff_dbusd_t bin_t:file { execute getattr open read };
allow staff_dbusd_t bin_t:lnk_file { getattr read };
allow staff_gkeyringd_t bin_t:dir { getattr ioctl lock open read search };
allow staff_gkeyringd_t bin_t:file { execute getattr open read };
allow staff_gkeyringd_t bin_t:lnk_file { getattr read };
allow staff_screen_t bin_t:dir { getattr ioctl lock open read search };
allow staff_screen_t bin_t:file { execute getattr open read };
allow staff_screen_t bin_t:lnk_file { getattr read };
allow staff_seunshare_t bin_t:dir { getattr ioctl lock open read search };
allow staff_seunshare_t bin_t:file { execute getattr open read };
allow staff_seunshare_t bin_t:lnk_file { getattr read };
allow staff_ssh_agent_t bin_t:dir { getattr ioctl lock open read search };
allow staff_ssh_agent_t bin_t:file { execute getattr open read };
allow staff_ssh_agent_t bin_t:lnk_file { getattr read };
allow staff_sudo_t bin_t:dir { getattr ioctl lock open read search };
allow staff_sudo_t bin_t:file { execute getattr open read };
allow staff_sudo_t bin_t:lnk_file { getattr read };
allow staff_t bin_t:dir { getattr ioctl lock open read search };
allow staff_t bin_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow staff_t bin_t:lnk_file { getattr read };
allow staff_t bin_t:lnk_file { getattr read }; [ pppd_for_user ]:True
allow staff_usertype bin_t:dir { getattr ioctl lock open read search };
allow staff_usertype bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow staff_usertype bin_t:lnk_file { getattr read };
allow staff_wine_t bin_t:file { execute getattr open read };
allow staff_wine_t bin_t:lnk_file { getattr read };
allow stapserver_t bin_t:dir { getattr ioctl lock open read search };
allow stapserver_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow stapserver_t bin_t:lnk_file { getattr read };
allow stunnel_t bin_t:dir { getattr ioctl lock open read search };
allow stunnel_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow stunnel_t bin_t:lnk_file { getattr read };
allow su_domain_type bin_t:lnk_file { getattr read };
allow sudodomain bin_t:dir { getattr ioctl lock open read search };
allow sudodomain bin_t:lnk_file { getattr read };
allow svc_run_t bin_t:dir { getattr ioctl lock open read search };
allow svc_run_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow svc_run_t bin_t:lnk_file { getattr read };
allow svc_start_t bin_t:dir { getattr ioctl lock open read search };
allow svc_start_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow svc_start_t bin_t:lnk_file { getattr read };
allow svirt_sandbox_domain bin_t:dir { getattr ioctl lock open read search };
allow swat_t bin_t:lnk_file { getattr read };
allow swift_t bin_t:dir { getattr ioctl lock open read search };
allow swift_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow swift_t bin_t:lnk_file { getattr read };
allow sysadm_dbusd_t bin_t:dir { getattr ioctl lock open read search };
allow sysadm_dbusd_t bin_t:file { execute getattr open read };
allow sysadm_dbusd_t bin_t:lnk_file { getattr read };
allow sysadm_passwd_t bin_t:dir { getattr ioctl lock open read search };
allow sysadm_passwd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow sysadm_passwd_t bin_t:lnk_file { getattr read };
allow sysadm_screen_t bin_t:dir { getattr ioctl lock open read search };
allow sysadm_screen_t bin_t:file { execute getattr open read };
allow sysadm_screen_t bin_t:lnk_file { getattr read };
allow sysadm_seunshare_t bin_t:dir { getattr ioctl lock open read search };
allow sysadm_seunshare_t bin_t:file { execute getattr open read };
allow sysadm_seunshare_t bin_t:lnk_file { getattr read };
allow sysadm_ssh_agent_t bin_t:dir { getattr ioctl lock open read search };
allow sysadm_ssh_agent_t bin_t:file { execute getattr open read };
allow sysadm_ssh_agent_t bin_t:lnk_file { getattr read };
allow sysadm_su_t bin_t:dir { getattr ioctl lock open read search };
allow sysadm_su_t bin_t:lnk_file { getattr read };
allow sysadm_sudo_t bin_t:dir { getattr ioctl lock open read search };
allow sysadm_sudo_t bin_t:file { execute getattr open read };
allow sysadm_sudo_t bin_t:lnk_file { getattr read };
allow sysadm_t bin_t:dir { getattr ioctl lock open read search };
allow sysadm_t bin_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow sysadm_t bin_t:lnk_file { getattr read };
allow sysadm_usertype bin_t:dir { getattr ioctl lock open read search };
allow sysadm_usertype bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow sysadm_usertype bin_t:lnk_file { getattr read };
allow syslogd_t bin_t:dir { getattr ioctl lock open read search };
allow syslogd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow syslogd_t bin_t:lnk_file { getattr read };
allow sysstat_t bin_t:dir { getattr ioctl lock open read search };
allow sysstat_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow sysstat_t bin_t:lnk_file { getattr read };
allow system_cronjob_t bin_t:dir { getattr ioctl lock open read search };
allow system_cronjob_t bin_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow system_cronjob_t bin_t:lnk_file { getattr read };
allow system_cronjob_t bin_t:lnk_file { getattr read }; [ cron_can_relabel ]:True
allow system_dbusd_t bin_t:dir { getattr ioctl lock open read search };
allow system_dbusd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow system_dbusd_t bin_t:lnk_file { getattr read };
allow system_mail_t bin_t:dir { getattr ioctl lock open read search };
allow system_mail_t bin_t:lnk_file { getattr read };
allow system_munin_plugin_t bin_t:lnk_file { getattr read };
allow systemd_logind_t bin_t:lnk_file { getattr read };
allow systemd_timedated_t bin_t:dir { getattr ioctl lock open read search };
allow systemd_timedated_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow systemd_timedated_t bin_t:lnk_file { getattr read };
allow targetd_t bin_t:dir { getattr ioctl lock open read search };
allow targetd_t bin_t:lnk_file { getattr read };
allow tcpd_t bin_t:dir { getattr ioctl lock open read search };
allow tcpd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow tcpd_t bin_t:lnk_file { getattr read };
allow telepathy_domain bin_t:dir { getattr ioctl lock open read search };
allow telepathy_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow telepathy_domain bin_t:lnk_file { getattr read };
allow telepathy_msn_t bin_t:lnk_file { getattr read };
allow telnetd_t bin_t:lnk_file { getattr read };
allow thin_domain bin_t:dir { getattr ioctl lock open read search };
allow thin_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow thin_domain bin_t:lnk_file { getattr read };
allow thumb_t bin_t:dir { getattr ioctl lock open read search };
allow thumb_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow thumb_t bin_t:lnk_file { getattr read };
allow timemaster_t bin_t:lnk_file { getattr read };
allow tmpreaper_t bin_t:dir { getattr ioctl lock open read search };
allow tmpreaper_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow tmpreaper_t bin_t:lnk_file { getattr read };
allow tomcat_domain bin_t:dir { getattr ioctl lock open read search };
allow tomcat_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow tomcat_domain bin_t:lnk_file { getattr read };
allow tuned_t bin_t:dir { getattr ioctl lock open read search };
allow tuned_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow tuned_t bin_t:lnk_file { getattr read };
allow udev_t bin_t:dir { getattr ioctl lock open read search };
allow udev_t bin_t:lnk_file { getattr read };
allow uml_t bin_t:dir { getattr ioctl lock open read search };
allow uml_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow uml_t bin_t:lnk_file { getattr read };
allow unconfined_dbusd_t bin_t:dir { getattr ioctl lock open read search };
allow unconfined_dbusd_t bin_t:file { execute getattr open read };
allow unconfined_dbusd_t bin_t:lnk_file { getattr read };
allow unconfined_domain_type bin_t:lnk_file { getattr read };
allow unconfined_login_domain bin_t:dir { getattr ioctl lock open read search }; [ unconfined_login ]:True
allow unconfined_login_domain bin_t:lnk_file { getattr read }; [ unconfined_login ]:True
allow unconfined_service_t bin_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow unconfined_t bin_t:dir { getattr ioctl lock open read search };
allow unconfined_t bin_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow unconfined_t bin_t:lnk_file { getattr read };
allow unconfined_usertype bin_t:dir { getattr ioctl lock open read search };
allow unconfined_usertype bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow unconfined_usertype bin_t:lnk_file { getattr read };
allow update_modules_t bin_t:dir { getattr ioctl lock open read search };
allow update_modules_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow update_modules_t bin_t:lnk_file { getattr read };
allow updfstab_t bin_t:dir { getattr ioctl lock open read search };
allow updfstab_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow updfstab_t bin_t:lnk_file { getattr read };
allow user_dbusd_t bin_t:dir { getattr ioctl lock open read search };
allow user_dbusd_t bin_t:file { execute getattr open read };
allow user_dbusd_t bin_t:lnk_file { getattr read };
allow user_gkeyringd_t bin_t:dir { getattr ioctl lock open read search };
allow user_gkeyringd_t bin_t:file { execute getattr open read };
allow user_gkeyringd_t bin_t:lnk_file { getattr read };
allow user_mail_domain bin_t:dir { getattr ioctl lock open read search };
allow user_mail_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow user_mail_domain bin_t:lnk_file { getattr read };
allow user_screen_t bin_t:dir { getattr ioctl lock open read search };
allow user_screen_t bin_t:file { execute getattr open read };
allow user_screen_t bin_t:lnk_file { getattr read };
allow user_seunshare_t bin_t:dir { getattr ioctl lock open read search };
allow user_seunshare_t bin_t:file { execute getattr open read };
allow user_seunshare_t bin_t:lnk_file { getattr read };
allow user_ssh_agent_t bin_t:dir { getattr ioctl lock open read search };
allow user_ssh_agent_t bin_t:file { execute getattr open read };
allow user_ssh_agent_t bin_t:lnk_file { getattr read };
allow user_t bin_t:dir { getattr ioctl lock open read search };
allow user_t bin_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow user_t bin_t:lnk_file { getattr read };
allow user_t bin_t:lnk_file { getattr read }; [ pppd_for_user ]:True
allow user_t bin_t:lnk_file { getattr read }; [ selinuxuser_ping ]:True
allow user_t bin_t:lnk_file { getattr read }; [ selinuxuser_ping ]:True
allow user_usertype bin_t:dir { getattr ioctl lock open read search };
allow user_usertype bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow user_usertype bin_t:lnk_file { getattr read };
allow user_wine_t bin_t:file { execute getattr open read };
allow user_wine_t bin_t:lnk_file { getattr read };
allow useradd_t bin_t:dir { getattr ioctl lock open read search };
allow useradd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow useradd_t bin_t:lnk_file { getattr read };
allow usernetctl_t bin_t:dir { getattr ioctl lock open read search };
allow usernetctl_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow usernetctl_t bin_t:lnk_file { getattr read };
allow uucpd_t bin_t:dir { getattr ioctl lock open read search };
allow uucpd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow uucpd_t bin_t:lnk_file { getattr read };
allow uux_t bin_t:dir { getattr ioctl lock open read search };
allow uux_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow uux_t bin_t:lnk_file { getattr read };
allow varnishd_t bin_t:dir { getattr ioctl lock open read search };
allow varnishd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow varnishd_t bin_t:lnk_file { getattr read };
allow vhostmd_t bin_t:dir { getattr ioctl lock open read search };
allow vhostmd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow vhostmd_t bin_t:lnk_file { getattr read };
allow virsh_ssh_t bin_t:dir { getattr ioctl lock open read search };
allow virsh_ssh_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow virsh_ssh_t bin_t:lnk_file { getattr read };
allow virsh_t bin_t:dir { getattr ioctl lock open read search };
allow virsh_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow virsh_t bin_t:lnk_file { getattr read };
allow virt_domain bin_t:dir { getattr ioctl lock open read search };
allow virt_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow virt_domain bin_t:lnk_file { getattr read };
allow virt_qemu_ga_t bin_t:dir { getattr ioctl lock open read search };
allow virt_qemu_ga_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow virt_qemu_ga_t bin_t:lnk_file { getattr read };
allow virtd_lxc_t bin_t:dir { getattr ioctl lock open read search };
allow virtd_lxc_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow virtd_lxc_t bin_t:lnk_file { getattr read };
allow virtd_t bin_t:dir { getattr ioctl lock open read search };
allow virtd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow virtd_t bin_t:lnk_file { getattr read };
allow vlock_t bin_t:dir { getattr ioctl lock open read search };
allow vlock_t bin_t:lnk_file { getattr read };
allow vmtools_helper_t bin_t:dir { getattr ioctl lock open read search };
allow vmtools_helper_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow vmtools_helper_t bin_t:lnk_file { getattr read };
allow vmtools_t bin_t:dir { getattr ioctl lock open read search };
allow vmtools_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow vmtools_t bin_t:lnk_file { getattr read };
allow vmware_host_t bin_t:dir { getattr ioctl lock open read search };
allow vmware_host_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow vmware_host_t bin_t:lnk_file { getattr read };
allow vmware_t bin_t:dir { getattr ioctl lock open read search };
allow vmware_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow vmware_t bin_t:lnk_file { getattr read };
allow vpnc_t bin_t:dir { getattr ioctl lock open read search };
allow vpnc_t bin_t:lnk_file { getattr read };
allow watchdog_t bin_t:dir { getattr ioctl lock open read search };
allow watchdog_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow watchdog_t bin_t:lnk_file { getattr read };
allow wdmd_t bin_t:dir { getattr ioctl lock open read search };
allow wdmd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow wdmd_t bin_t:lnk_file { getattr read };
allow webadm_t bin_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow webadm_t bin_t:lnk_file { getattr read };
allow webadm_usertype bin_t:dir { getattr ioctl lock open read search };
allow webadm_usertype bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow webadm_usertype bin_t:lnk_file { getattr read };
allow winbind_helper_t bin_t:dir { getattr ioctl lock open read search };
allow winbind_helper_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow winbind_helper_t bin_t:lnk_file { getattr read };
allow winbind_t bin_t:dir { getattr ioctl lock open read search };
allow winbind_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow winbind_t bin_t:lnk_file { getattr read };
allow wireshark_t bin_t:dir { getattr ioctl lock open read search };
allow wireshark_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow wireshark_t bin_t:lnk_file { getattr read };
allow xdm_t bin_t:dir { getattr ioctl lock open read search };
allow xdm_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow xdm_t bin_t:lnk_file { getattr read };
allow xenconsoled_t bin_t:lnk_file { getattr read };
allow xend_t bin_t:dir { getattr ioctl lock open read search };
allow xend_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow xend_t bin_t:lnk_file { getattr read };
allow xguest_dbusd_t bin_t:dir { getattr ioctl lock open read search };
allow xguest_dbusd_t bin_t:file { execute getattr open read };
allow xguest_dbusd_t bin_t:lnk_file { getattr read };
allow xguest_gkeyringd_t bin_t:dir { getattr ioctl lock open read search };
allow xguest_gkeyringd_t bin_t:file { execute getattr open read };
allow xguest_gkeyringd_t bin_t:lnk_file { getattr read };
allow xguest_t bin_t:dir { getattr ioctl lock open read search };
allow xguest_t bin_t:file { entrypoint execute execute_no_trans getattr ioctl lock open read };
allow xguest_t bin_t:lnk_file { getattr read };
allow xguest_usertype bin_t:dir { getattr ioctl lock open read search };
allow xguest_usertype bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow xguest_usertype bin_t:lnk_file { getattr read };
allow xserver_t bin_t:dir { getattr ioctl lock open read search };
allow xserver_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow xserver_t bin_t:lnk_file { getattr read };
allow yppasswdd_t bin_t:dir { getattr ioctl lock open read search };
allow yppasswdd_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow yppasswdd_t bin_t:lnk_file { getattr read };
allow ypserv_t bin_t:dir { getattr ioctl lock open read search };
allow ypserv_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow ypserv_t bin_t:lnk_file { getattr read };
allow zabbix_agent_t bin_t:lnk_file { getattr read };
allow zabbix_domain bin_t:dir { getattr ioctl lock open read search };
allow zabbix_domain bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow zabbix_domain bin_t:lnk_file { getattr read };
allow zabbix_script_t bin_t:lnk_file { getattr read };
allow zabbix_t bin_t:lnk_file { getattr read };
allow zoneminder_t bin_t:dir { getattr ioctl lock open read search };
allow zoneminder_t bin_t:file { execute execute_no_trans getattr ioctl lock open read };
allow zoneminder_t bin_t:lnk_file { getattr read };
allow zoneminder_t bin_t:lnk_file { getattr read }; [ zoneminder_run_sudo ]:True
```
        

#### File Write Permissions


```
allow cupsd_t bin_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow named_filetrans_domain bin_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow prelink_t bin_t:dir { add_name getattr ioctl lock open read remove_name search write };
allow prelink_t bin_t:lnk_file { append create getattr ioctl link lock read rename setattr unlink write };
allow rpm_script_t bin_t:dir { add_name getattr ioctl lock open read remove_name search write };
```
        



In [45]:
p.packet_summary("dns_client_packet_t")


### Packet Type Summary: dns_client_packet_t

####  Type Attributes

  * `client_packet_type`
  * `packet_type`


#### Packet Permissions


```
allow abrt_retrace_coredump_t dns_client_packet_t:packet { recv send };
allow abrt_retrace_worker_t dns_client_packet_t:packet { recv send };
allow afs_t dns_client_packet_t:packet { recv send };
allow aiccu_t dns_client_packet_t:packet { recv send };
allow ajaxterm_t dns_client_packet_t:packet { recv send };
allow apcupsd_cgi_script_t dns_client_packet_t:packet { recv send };
allow apcupsd_t dns_client_packet_t:packet { recv send };
allow audisp_remote_t dns_client_packet_t:packet { recv send };
allow audisp_t dns_client_packet_t:packet { recv send };
allow auditd_t dns_client_packet_t:packet { recv send };
allow awstats_t dns_client_packet_t:packet { recv send };
allow bacula_admin_t dns_client_packet_t:packet { recv send };
allow bacula_t dns_client_packet_t:packet { recv send };
allow blueman_t dns_client_packet_t:packet { recv send };
allow boinc_domain dns_client_packet_t:packet { recv send };
allow brltty_t dns_client_packet_t:packet { recv send };
allow bumblebee_t dns_client_packet_t:packet { recv send };
allow ccs_t dns_client_packet_t:packet { recv send };
allow cfengine_domain dns_client_packet_t:packet { recv send };
allow chrome_sandbox_t dns_client_packet_t:packet { recv send };
allow cloud_init_t dns_client_packet_t:packet { recv send };
allow cobblerd_t dns_client_packet_t:packet { recv send };
allow collectd_t dns_client_packet_t:packet { recv send };
allow condor_domain dns_client_packet_t:packet { recv send };
allow conman_t dns_client_packet_t:packet { recv send };
allow cupsd_t dns_client_packet_t:packet { recv send };
allow cyphesis_t dns_client_packet_t:packet { recv send };
allow ddclient_t dns_client_packet_t:packet { recv send };
allow denyhosts_t dns_client_packet_t:packet { recv send };
allow dirsrv_snmp_t dns_client_packet_t:packet { recv send };
allow dirsrv_t dns_client_packet_t:packet { recv send };
allow dkim_milter_t dns_client_packet_t:packet { recv send };
allow dnssec_trigger_t dns_client_packet_t:packet { recv send };
allow drbd_t dns_client_packet_t:packet { recv send };
allow fetchmail_t dns_client_packet_t:packet { recv send };
allow firewalld_t dns_client_packet_t:packet { recv send };
allow firstboot_t dns_client_packet_t:packet { recv send };
allow freeipmi_domain dns_client_packet_t:packet { recv send };
allow fsdaemon_t dns_client_packet_t:packet { recv send };
allow games_t dns_client_packet_t:packet { recv send };
allow gear_t dns_client_packet_t:packet { recv send };
allow geoclue_t dns_client_packet_t:packet { recv send };
allow glance_domain dns_client_packet_t:packet { recv send };
allow haproxy_t dns_client_packet_t:packet { recv send };
allow hostname_t dns_client_packet_t:packet { recv send };
allow hypervkvp_t dns_client_packet_t:packet { recv send };
allow ifconfig_t dns_client_packet_t:packet { recv send };
allow ipa_otpd_t dns_client_packet_t:packet { recv send };
allow iptables_t dns_client_packet_t:packet { recv send };
allow kpropd_t dns_client_packet_t:packet { recv send };
allow l2tpd_t dns_client_packet_t:packet { recv send };
allow lircd_t dns_client_packet_t:packet { recv send };
allow mojomojo_script_t dns_client_packet_t:packet { recv send };
allow mongod_t dns_client_packet_t:packet { recv send };
allow nagios_mail_plugin_t dns_client_packet_t:packet { recv send };
allow netlogond_t dns_client_packet_t:packet { recv send };
allow ninfod_t dns_client_packet_t:packet { recv send };
allow nsswitch_domain dns_client_packet_t:packet { recv send };
allow nutups_cgi_script_t dns_client_packet_t:packet { recv send };
allow openvpn_t dns_client_packet_t:packet { recv send };
allow openvswitch_t dns_client_packet_t:packet { recv send };
allow pads_t dns_client_packet_t:packet { recv send };
allow pcscd_t dns_client_packet_t:packet { recv send };
allow piranha_lvs_t dns_client_packet_t:packet { recv send };
allow podsleuth_t dns_client_packet_t:packet { recv send };
allow prelude_audisp_t dns_client_packet_t:packet { recv send };
allow prelude_correlator_t dns_client_packet_t:packet { recv send };
allow qpidd_t dns_client_packet_t:packet { recv send };
allow rabbitmq_t dns_client_packet_t:packet { recv send };
allow realmd_t dns_client_packet_t:packet { recv send };
allow redis_t dns_client_packet_t:packet { recv send };
allow rhgb_t dns_client_packet_t:packet { recv send };
allow rhsmcertd_t dns_client_packet_t:packet { recv send };
allow ricci_t dns_client_packet_t:packet { recv send };
allow rkt_t dns_client_packet_t:packet { recv send };
allow roundup_t dns_client_packet_t:packet { recv send };
allow rpcbind_t dns_client_packet_t:packet { recv send };
allow rwho_t dns_client_packet_t:packet { recv send };
allow sblim_gatherd_t dns_client_packet_t:packet { recv send };
allow sge_domain dns_client_packet_t:packet { recv send };
allow showmount_t dns_client_packet_t:packet { recv send };
allow slpd_t dns_client_packet_t:packet { recv send };
allow smokeping_cgi_script_t dns_client_packet_t:packet { recv send };
allow smsd_t dns_client_packet_t:packet { recv send };
allow snort_t dns_client_packet_t:packet { recv send };
allow spamc_t dns_client_packet_t:packet { recv send };
allow squid_script_t dns_client_packet_t:packet { recv send };
allow sslh_t dns_client_packet_t:packet { recv send };
allow sssd_t dns_client_packet_t:packet { recv send };
allow svnserve_t dns_client_packet_t:packet { recv send };
allow tomcat_domain dns_client_packet_t:packet { recv send };
allow ulogd_t dns_client_packet_t:packet { recv send };
allow virsh_t dns_client_packet_t:packet { recv send };
allow virt_qemu_ga_t dns_client_packet_t:packet { recv send };
allow vmware_host_t dns_client_packet_t:packet { recv send };
allow vmware_t dns_client_packet_t:packet { recv send };
allow w3c_validator_script_t dns_client_packet_t:packet { recv send };
allow watchdog_t dns_client_packet_t:packet { recv send };
allow webalizer_t dns_client_packet_t:packet { recv send };
allow xend_t dns_client_packet_t:packet { recv send };
allow xguest_t dns_client_packet_t:packet { recv send };
allow zabbix_agent_t dns_client_packet_t:packet { recv send };
allow zarafa_server_t dns_client_packet_t:packet { recv send };
```
        



# Policy Source

Find a type definition

In [46]:
# note the use of print to make this look nice
print(ps.type_def("kernel_t"))

./kernel/kernel.te:51:type kernel_t, can_load_kernmodule;



In [47]:
print(ps.attr_def("domain"))

./contrib/abrt.te:404:		attribute domain;
./kernel/domain.te:40:attribute domain;
./kernel/unlabelednet.te:7:    attribute domain;



In [48]:
ps.genfscon("selinuxfs")

'./kernel/selinux.te:37:genfscon selinuxfs / gen_context(system_u:object_r:security_t,s0)\n'

In [49]:
print(ps.file_contexts("httpd_exec_t"))

./contrib/apache.fc:49:/usr/bin/mongrel_rails		--	gen_context(system_u:object_r:httpd_exec_t,s0)
./contrib/apache.fc:53:/usr/lib/apache-ssl/.+		--	gen_context(system_u:object_r:httpd_exec_t,s0)
./contrib/apache.fc:63:/usr/sbin/apache(2)?		--	gen_context(system_u:object_r:httpd_exec_t,s0)
./contrib/apache.fc:64:/usr/sbin/apache-ssl(2)?	--	gen_context(system_u:object_r:httpd_exec_t,s0)
./contrib/apache.fc:65:/usr/sbin/cherokee		--	gen_context(system_u:object_r:httpd_exec_t,s0)
./contrib/apache.fc:66:/usr/sbin/httpd\.event		--	gen_context(system_u:object_r:httpd_exec_t,s0)
./contrib/apache.fc:67:/usr/sbin/httpd(\.worker)?	--	gen_context(system_u:object_r:httpd_exec_t,s0)
./contrib/apache.fc:68:/usr/sbin/htcacheclean      --  gen_context(system_u:object_r:httpd_exec_t,s0)
./contrib/apache.fc:69:/usr/sbin/lighttpd		--	gen_context(system_u:object_r:httpd_exec_t,s0)
./contrib/apache.fc:70:/usr/sbin/nginx         --  gen_context(system_u:object_r:httpd_exec_t,s0)
./contrib/apache.fc:71:/usr/sb

Search for rules (this is just grep really)

In [50]:
print(ps.rules_search("allow sshd_t"))

./services/ssh.if:912:    allow sshd_t $1:process dyntransition;
./services/ssh.if:914:    allow sshd_t $1:process { getattr sigkill sigstop signull signal };
./services/ssh.te:269:allow sshd_t self:netlink_route_socket r_netlink_socket_perms;
./services/ssh.te:270:allow sshd_t self:key { search link write };
./services/ssh.te:271:allow sshd_t self:process setcurrent;
./services/ssh.te:273:allow sshd_t sshd_keytab_t:file read_file_perms;
./services/ssh.te:428:		allow sshd_t ptyfile:chr_file relabelto;
./services/ssh.te:441:		allow sshd_t userpty_type:chr_file { relabelto rw_inherited_chr_file_perms setattr_chr_file_perms };
./services/ssh.te:552:allow sshd_t sshd_sandbox_t:process signal;
./services/ssh.te:563:allow sshd_t sshd_net_t:process signal;



Show an entire module

In [51]:
print(ps.get_module("services/ssh.te"))

policy_module(ssh, 2.4.2)

########################################
#
# Declarations
#

## <desc>
##      <p>
##      allow host key based authentication
##      </p>
## </desc>
gen_tunable(ssh_keysign, false)

## <desc>
##	<p>
##	Allow ssh logins as sysadm_r:sysadm_t
##	</p>
## </desc>
gen_tunable(ssh_sysadm_login, false)

## <desc>
## <p>
## Allow ssh with chroot env to read and write files 
## in the user home directories
## </p>
## </desc>
gen_tunable(ssh_chroot_rw_homedirs, false)

attribute ssh_dyntransition_domain;
attribute ssh_server;
attribute ssh_agent_type;

ssh_dyntransition_domain_template(chroot_user_t)
ssh_dyntransition_domain_template(sshd_sandbox_t)
ssh_dyntransition_domain_template(sshd_net_t)

type ssh_keygen_t;
type ssh_keygen_exec_t;
init_system_domain(ssh_keygen_t, ssh_keygen_exec_t)

type ssh_keygen_tmp_t;
files_tmp_file(ssh_keygen_tmp_t)

type sshd_keygen_t;
type sshd_keygen_exec_t;
init_daemon_domain(sshd_keygen_t, sshd_keygen_exec_t)

type sshd_keygen_unit_fi